### Importing necessary libraries

In [13]:
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
import os
import dill

# Cricbuzz Scraping

In [4]:
def get_match_data(match_id):
  payload = {}
  headers = {
    'x-apihub-key': 'NFrLWAL6waj1iYNyxjW594CFWp0KHzlZVJIMOAaEX7t3OOdbfk',
    'x-apihub-host': 'Cricbuzz-Official-Cricket-API.allthingsdev.co',
    'x-apihub-endpoint': '5f260335-c228-4005-9eec-318200ca48d6'
  }
  try:
    match_number_string = str(match_id)
    url = "https://Cricbuzz-Official-Cricket-API.proxy-production.allthingsdev.co/match/"+match_number_string+"/scorecard"
    response = requests.request("GET", url, headers=headers, data=payload)
    match = response.json()
    match_url = match['appindex']['weburl']
    return match_url, match
  except:
    return None,None

In [ ]:
matches = []
match_number = 8598
while True:
  match_url,match_data = get_match_scorecard(match_number)
  if match_number == None:
    print(match_number,": unsuccessful")
  else:
    matches.append(match_data)
  match_number += 1




8598 http://www.cricbuzz.com/live-cricket-scorecard/8598/aus-vs-eng-1st-test-england-in-australia-test-series-1877
8599 http://www.cricbuzz.com/live-cricket-scorecard/8599/aus-vs-eng-2nd-test-england-in-australia-test-series-1877
8600 http://www.cricbuzz.com/live-cricket-scorecard/8600/aus-vs-eng-only-test-england-in-australia-test-match-1879
8601 http://www.cricbuzz.com/live-cricket-scorecard/8601/eng-vs-aus-only-test-australia-in-england-test-match-1880
8602 http://www.cricbuzz.com/live-cricket-scorecard/8602/aus-vs-eng-1st-test-england-in-australia-test-series-1881-82
8603 http://www.cricbuzz.com/live-cricket-scorecard/8603/aus-vs-eng-2nd-test-england-in-australia-test-series-1881-82
8604 http://www.cricbuzz.com/live-cricket-scorecard/8604/aus-vs-eng-3rd-test-england-in-australia-test-series-1881-82
8605 http://www.cricbuzz.com/live-cricket-scorecard/8605/aus-vs-eng-4th-test-england-in-australia-test-series-1881-82
8606 http://www.cricbuzz.com/live-cricket-scorecard/8606/eng-vs-aus-

In [ ]:
match_url1,match_data1 = get_match_scorecard(117371)

In [ ]:
match_url2,match_data2 = get_match_scorecard(114960)

In [22]:
import pandas as pd
from typing import List, Dict, Tuple, Any

class Score:
    
    def __init__(self, match_number: int):
        self.match_number = match_number
        self.match_url, self.match_data = self.get_match_data(match_number)
        (
            self.full_player_list,
            self.player_list,
            self.winner,
            self.man_of_the_match,
            self.catchers,
            self.stumpers,
            self.main_runouters,
            self.secondary_runouters,
            self.bowled,
            self.lbw,
            self.innings_list,
            self.batsmen_list,
            self.bowlers_info
        ) = self._parse_json(self.match_data)
        
    def get_match_data(self,match_id):
        payload = {}
        headers = {
            'x-apihub-key': 'NFrLWAL6waj1iYNyxjW594CFWp0KHzlZVJIMOAaEX7t3OOdbfk',
            'x-apihub-host': 'Cricbuzz-Official-Cricket-API.allthingsdev.co',
            'x-apihub-endpoint': '5f260335-c228-4005-9eec-318200ca48d6'
        }
        try:
            match_number_string = str(match_id)
            url = "https://Cricbuzz-Official-Cricket-API.proxy-production.allthingsdev.co/match/"+match_number_string+"/scorecard"
            response = requests.request("GET", url, headers=headers, data=payload)
            match = response.json()
            match_url = match['appindex']['weburl']
            return match_url, match
        except:
            return None,None

    def find_full_name(self,team,short_name):
        try:
            if "Varun Chak" in short_name:
                return "Varun Chakaravarthy"
            if "Reddy" in short_name and "Nitish" in short_name:
                return "Nitish Reddy"
            player = ""
            for player in team:
                if len(player)>len(short_name):
                    count = 0
                    parts = short_name.split(' ')
                    parts_dup = []
                    for part in parts:
                        word = ""
                        for letter_number in range(len(part)):
                            char = part[letter_number]
                            word += char
                            if letter_number == len(part)-1:
                                parts_dup.append(word)
                                break
                            if part[letter_number+1].isupper():
                                parts_dup.append(word)
                                word = ""
                    parts = parts_dup
                    for part in parts:
                        if part not in player:
                            count += 1
                    if count == 0:
                        #team_copy.append(player)
                        return player
                else:
                    count = 0
                    parts = player.split(' ')
                    parts_dup = []
                    for part in parts:
                        word = ""
                        for letter_number in range(len(part)):
                            char = part[letter_number]
                            word += char
                            if letter_number == len(part)-1:
                                parts_dup.append(word)
                                break
                            if part[letter_number+1].isupper():
                                parts_dup.append(word)
                                word = ""
                    parts = parts_dup
                    for part in parts:
                        if part not in short_name:
                            count += 1
                    if count == 0:
                        return player
                    
            if player == "":
                try:
                    best_match, score, _ = process.extractOne(short_name, team)
                    if score > 70:
                        return best_match
                    else:
                        print(short_name,"not found")
                except:
                    print(short_name,"not found")
                    pass
        except:
            return None

    def _parse_dismissal(self, outdec: str) -> Dict[str, Any]:
        outdec = outdec.strip()
        if not outdec or outdec == 'not out':
            return {
                'mode': 'not out',
                'catcher': '',
                'stumper': '',
                'main_ro': '',
                'secondary_ro': '',
                'bowler_bowled': '',
                'bowler_lbw': ''
            }

        result = {
            'mode': 'other',
            'catcher': '',
            'stumper': '',
            'main_ro': '',
            'secondary_ro': '',
            'bowler_bowled': '',
            'bowler_lbw': ''
        }

        # c and b X → caught by X (but NOT bowled)
        if outdec.startswith('c and b '):
            x = outdec.replace('c and b ', '').strip()
            result.update({'mode': 'caught', 'catcher': x})
            return result

        # c X b Y
        if outdec.startswith('c ') and ' b ' in outdec:
            parts = outdec.split(' b ', 1)
            fielder = parts[0].replace('c ', '').strip()
            bowler = parts[1].strip()
            result.update({'mode': 'caught', 'catcher': fielder})
            return result

        # st X b Y
        if outdec.startswith('st ') and ' b ' in outdec:
            parts = outdec.split(' b ', 1)
            fielder = parts[0].replace('st ', '').strip()
            bowler = parts[1].strip()
            result.update({'mode': 'stumped', 'stumper': fielder})
            return result

        # b X (pure bowled — must NOT contain 'lbw')
        if outdec.startswith('b ') and 'lbw' not in outdec.lower():
            bowler = outdec.replace('b ', '').strip()
            result.update({'mode': 'bowled', 'bowler_bowled': bowler})
            return result

        # lbw b X
        if 'lbw b ' in outdec.lower():
            bowler = outdec.lower().split('lbw b ')[-1].strip()
            # Fallback if case mismatch
            if not bowler:
                bowler = outdec.split(' b ')[-1].strip()
            result.update({'mode': 'lbw', 'bowler_lbw': bowler})
            return result

        # run out (X) or run out (X/Y)
        if 'run out' in outdec.lower():
            import re
            f_match = re.search(r'\(([^)]*)\)', outdec)

            main = None
            sec = None

            if f_match:
                raw = f_match.group(1).strip()

                # If raw is not empty, process it
                if raw:
                    parts = [p.strip() for p in raw.split('/') if p.strip()]

                    if len(parts) >= 2:
                        # Take only the last two
                        main, sec = parts[-2], parts[-1]
                    elif len(parts) == 1:
                        main = parts[0]
                    # If 0 parts → leave main/sec = None
                # If raw is empty → leave main/sec = None

            # Build result dict only with fields that exist
            result.update({'mode': 'run out'})
            if main is not None:
                result['main_ro'] = main
            if sec is not None:
                result['secondary_ro'] = sec

            return result


        # retd out, obstructing, etc. → ignore for dismissal lists
        return result

    def _parse_json(self, match_Dict: [str, Any]) -> Tuple[
        List[str], Dict[str, List[str]], str, str,
        List[str], List[str], List[str], List[str],
        List[str], List[str],
        List[str], pd.DataFrame, pd.DataFrame
    ]:
        # Lists — NO deduplication, preserve order & frequency
        catchers, stumpers = [], []
        main_runouters, secondary_runouters = [], []
        bowled, lbw = [], []

        innings_list = []
        player_list = {}
        batsmen_rows, bowlers_rows = [], []
        
        dots_info,man_of_the_match = self.match_dots_mom()
        print(dots_info,man_of_the_match)

        for idx, inn in enumerate(self.match_data.get('scorecard', [])):
            inn_num = idx + 1
            bat_team = inn['batteamname']
            innings_list.append(bat_team)

            batsmen_names, bowler_names = [], []

            # --- Batsmen ---
            for b in inn.get('batsman', []):
                if b['balls'] == 0 and not b.get('outdec'):
                    continue
                name = b['name'].strip()
                batsmen_names.append(name)

                outdec = b.get('outdec', 'not out') or 'not out'
                dis = self._parse_dismissal(outdec)

                # Append exactly once per dismissal — no dedup
                if dis['catcher']:
                    catchers.append(dis['catcher'])
                if dis['stumper']:
                    stumpers.append(dis['stumper'])
                if dis['main_ro'] != '' or dis['secondary_ro'] != '':
                    main_runouters.append(dis['main_ro'])
                    secondary_runouters.append(dis['secondary_ro'])
                if dis['bowler_bowled']:
                    bowled.append(dis['bowler_bowled'])
                if dis['bowler_lbw']:
                    lbw.append(dis['bowler_lbw'])

                sr = float(b['strkrate']) if b['strkrate'] and b['strkrate'] != '0' else 0.0
                batsmen_rows.append({
                    'Innings Number': inn_num,
                    'Innings Name': bat_team,
                    'Batsman': name,
                    'Dismissal': outdec,
                    'Runs': int(b['runs']),
                    'Balls': int(b['balls']),
                    '4s': int(b['fours']),
                    '6s': int(b['sixes']),
                    'Strike Rate': sr
                })

            # --- Bowlers ---
            for blr in inn.get('bowler', []):
                print(blr)
                name = blr['name'].strip()
                bowler_names.append(name)

                # Overs: preserve as X.Y (e.g., '2.2' → 2.2, not 2.333)
                o_str = blr['overs']
                try:
                    if '.' in o_str:
                        whole, frac = o_str.split('.')
                        # Clamp fractional part to 1 digit (0–5)
                        frac = frac[:1]
                        # Ensure valid ball count (0–5)
                        if frac.isdigit() and 0 <= int(frac) <= 5:
                            overs_val = float(f"{whole}.{frac}")
                        else:
                            overs_val = float(whole)
                    else:
                        overs_val = float(o_str)
                except:
                    overs_val = 0.0
                
                bowlers_rows.append({
                    'Innings Number': inn_num,
                    'Innings Name': bat_team,
                    'Bowler': name,
                    'Overs': overs_val,
                    'Maidens': int(blr['maidens']),
                    'Runs': int(blr['runs']),
                    'Wickets': int(blr['wickets']),
                    'Economy': float(blr['economy']),
                    '0s': dots_info.get(name, {}).get('dots', 0)
                })

            all_players = list(set(batsmen_names + bowler_names))
            player_list[bat_team] = all_players

        full_player_list = [p for team in player_list.values() for p in team]

        batsmen_df = pd.DataFrame(batsmen_rows) if batsmen_rows else pd.DataFrame(columns=[
            'Innings Number', 'Innings Name', 'Batsman', 'Dismissal',
            'Runs', 'Balls', '4s', '6s', 'Strike Rate'
        ])
        bowlers_df = pd.DataFrame(bowlers_rows) if bowlers_rows else pd.DataFrame(columns=[
            'Innings Number', 'Innings Name', 'Bowler',
            'Overs', 'Maidens', 'Runs', 'Wickets', 'Economy','0s'
        ])

        # Winner
        status = self.match_data.get('status', '')
        winner = ""
        if 'won by' in status:
            winner = status.split(' won by ')[0].strip()
            for team in innings_list:
                if winner in team or team in winner:
                    winner = team
                    break
        elif ' won the' in status:
            winner = status.split(' won the')[0].split('(')[1].strip()
        

        return (
            full_player_list,
            player_list,
            winner,
            man_of_the_match,
            catchers,
            stumpers,
            main_runouters,
            secondary_runouters,
            bowled,
            lbw,
            innings_list,
            batsmen_df,
            bowlers_df
        )

    def printing_scorecard(self):
        print("Player List:")
        print(self.player_list)
        print()
        for innings in self.innings_list:
            print("-" * 140)
            print(f"{innings}:")
            print()

            print("Batsmen:")
            batsmen_inn = self.batsmen_list[self.batsmen_list['Innings Name'] == innings]
            if not batsmen_inn.empty:
                df = batsmen_inn.drop(columns=['Innings Number', 'Innings Name']).copy()
                print(df.to_string(index=False))
            else:
                print("(No data)")
            print()

            print("Bowlers:")
            bowlers_inn = self.bowlers_info[self.bowlers_info['Innings Name'] == innings]
            if not bowlers_inn.empty:
                df = bowlers_inn.drop(columns=['Innings Number', 'Innings Name']).copy()
                # Format Overs to show exactly one decimal place (e.g., 2.2, 4.0)
                df['Overs'] = df['Overs'].apply(lambda x: f"{x:.1f}")
                print(df.to_string(index=False))
            else:
                print("(No data)")
            print()

        print("Catchers:")
        print(self.catchers)
        print()

        print("Stumpings:")
        print(self.stumpers)
        print()

        print("Main Run Outs:")
        print(self.main_runouters)
        print()

        print("Secondary Run Outs:")
        print(self.secondary_runouters)
        print()

        print("Bowled:")
        print(self.bowled)
        print()

        print("LBW:")
        print(self.lbw)
        print()
        print("-" * 140)
        print("Winner: ", self.winner)
        print()
        print("Man of the Match: ", self.man_of_the_match)
        
    def match_dots_mom(self):
        def calculate_dots(over_summary):
            over = over_summary.split(' ')
            dots = 0
            for ball in over:
                if ball == '0' or ball == 'W' or 'L' in ball or 'B' in ball:
                    dots += 1
            return dots

        payload = {}
        headers = {
            'x-apihub-key': 'NFrLWAL6waj1iYNyxjW594CFWp0KHzlZVJIMOAaEX7t3OOdbfk',
            'x-apihub-host': 'Cricbuzz-Official-Cricket-API.allthingsdev.co',
            'x-apihub-endpoint': '5db6b2f0-86b9-44b5-bfc4-8c2888acd4de'
        }

        match_number_string = str(self.match_number)
        url = "https://Cricbuzz-Official-Cricket-API.proxy-production.allthingsdev.co/match/"+match_number_string+"/overs"
        response = requests.request("GET", url, headers=headers, data=payload)
        overs_score = response.json()  

        bowlers_info = {}
        for over in overs_score['overseplist']['oversep']:
            bowler_names = over['ovrbowlnames']
            over_summary = over['oversummary']
            if len(bowler_names) == 1:
                bowler = bowler_names[0]
                if bowler not in bowlers_info.keys():
                    bowlers_info[bowler] = {'dots': 0}
                    dots = calculate_dots(over_summary)
                    bowlers_info[bowler]['dots'] += dots
                else:
                    dots = calculate_dots(over_summary)
                    bowlers_info[bowler]['dots'] += dots
            else:
                for bowler in bowler_names:
                    x=2 #fill this later
        
        man_of_the_match = overs_score["matchheaders"]["momplayers"]["player"][0]["name"]

        return bowlers_info,man_of_the_match

{
  "scorecard": [
    {
      "inningsid": 1,
      "batsman": [
        {
          "id": 8520,
          "balls": 5,
          "runs": 4,
          "fours": 1,
          "sixes": 0,
          "strkrate": "80",
          "name": "de Kock",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": True,
          "outdec": "c Jitesh Sharma b Hazlewood",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": True,
          "playingxichange": ""
        },
        {
          "id": 2276,
          "balls": 26,
          "runs": 44,
          "fours": 5,
          "sixes": 3,
          "strkrate": "169.23",
          "name": "Narine",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "c Jitesh Sharma b Rasikh Salam",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": True,
          "playingxichange": ""
        },
        {
          "id": 1447,
          "balls": 31,
          "runs": 56,
          "fours": 6,
          "sixes": 4,
          "strkrate": "180.65",
          "name": "Ajinkya Rahane",
          "nickname": "",
          "iscaptain": True,
          "iskeeper": False,
          "outdec": "c Rasikh Salam b Krunal Pandya",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 10917,
          "balls": 7,
          "runs": 6,
          "fours": 1,
          "sixes": 0,
          "strkrate": "85.71",
          "name": "Venkatesh Iyer",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "b Krunal Pandya",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 22566,
          "balls": 22,
          "runs": 30,
          "fours": 2,
          "sixes": 1,
          "strkrate": "136.36",
          "name": "Angkrish Raghuvanshi",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "c Jitesh Sharma b Yash Dayal",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "MOUT",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 10896,
          "balls": 10,
          "runs": 12,
          "fours": 1,
          "sixes": 0,
          "strkrate": "120",
          "name": "Rinku Singh",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "b Krunal Pandya",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 7736,
          "balls": 3,
          "runs": 4,
          "fours": 1,
          "sixes": 0,
          "strkrate": "133.33",
          "name": "Russell",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "b Suyash Sharma",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": True,
          "playingxichange": ""
        },
        {
          "id": 12337,
          "balls": 9,
          "runs": 6,
          "fours": 0,
          "sixes": 0,
          "strkrate": "66.67",
          "name": "Ramandeep Singh",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "not out",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 24729,
          "balls": 6,
          "runs": 5,
          "fours": 1,
          "sixes": 0,
          "strkrate": "83.33",
          "name": "Harshit Rana",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "c Jitesh Sharma b Hazlewood",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 13143,
          "balls": 1,
          "runs": 1,
          "fours": 0,
          "sixes": 0,
          "strkrate": "100",
          "name": "Spencer Johnson",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "not out",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": True,
          "playingxichange": ""
        },
        {
          "id": 12926,
          "balls": 0,
          "runs": 0,
          "fours": 0,
          "sixes": 0,
          "strkrate": "0",
          "name": "Varun Chakaravarthy",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 15861,
          "balls": 0,
          "runs": 0,
          "fours": 0,
          "sixes": 0,
          "strkrate": "0",
          "name": "Vaibhav Arora",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "MIN",
          "isoverseas": False,
          "playingxichange": ""
        }
      ],
      "bowler": [
        {
          "id": 6258,
          "overs": "4",
          "maidens": 0,
          "wickets": 2,
          "runs": 22,
          "economy": "5.5",
          "name": "Hazlewood",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "dots": 0,
          "balls": 40,
          "rpb": 0.55,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": True,
          "playingxichange": ""
        },
        {
          "id": 14172,
          "overs": "3",
          "maidens": 0,
          "wickets": 1,
          "runs": 25,
          "economy": "8.3",
          "name": "Yash Dayal",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "dots": 0,
          "balls": 30,
          "rpb": 0.83,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 14274,
          "overs": "3",
          "maidens": 0,
          "wickets": 1,
          "runs": 35,
          "economy": "11.7",
          "name": "Rasikh Salam",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "dots": 0,
          "balls": 30,
          "rpb": 1.17,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 11311,
          "overs": "4",
          "maidens": 0,
          "wickets": 3,
          "runs": 29,
          "economy": "7.2",
          "name": "Krunal Pandya",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "dots": 0,
          "balls": 40,
          "rpb": 0.72,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 36487,
          "overs": "4",
          "maidens": 0,
          "wickets": 1,
          "runs": 47,
          "economy": "11.8",
          "name": "Suyash Sharma",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "dots": 0,
          "balls": 40,
          "rpb": 1.18,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "ispremiumfree": False,
          "inmatchchange": "MOUT",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 10045,
          "overs": "2",
          "maidens": 0,
          "wickets": 0,
          "runs": 14,
          "economy": "7",
          "name": "Livingstone",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "dots": 0,
          "balls": 20,
          "rpb": 0.7,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": True,
          "playingxichange": ""
        }
      ],
      "fow": {
        "fow": [
          {
            "batsmanid": 8520,
            "batsmanname": "de Kock",
            "overnbr": 0.5,
            "runs": 4,
            "ballnbr": 5
          },
          {
            "batsmanid": 2276,
            "batsmanname": "Narine",
            "overnbr": 9.6,
            "runs": 107,
            "ballnbr": 60
          },
          {
            "batsmanid": 1447,
            "batsmanname": "Ajinkya Rahane",
            "overnbr": 10.3,
            "runs": 109,
            "ballnbr": 63
          },
          {
            "batsmanid": 10917,
            "batsmanname": "Venkatesh Iyer",
            "overnbr": 12.1,
            "runs": 125,
            "ballnbr": 73
          },
          {
            "batsmanid": 10896,
            "batsmanname": "Rinku Singh",
            "overnbr": 14.6,
            "runs": 145,
            "ballnbr": 90
          },
          {
            "batsmanid": 7736,
            "batsmanname": "Russell",
            "overnbr": 15.4,
            "runs": 150,
            "ballnbr": 94
          },
          {
            "batsmanid": 22566,
            "batsmanname": "Angkrish Raghuvanshi",
            "overnbr": 18.5,
            "runs": 168,
            "ballnbr": 113
          },
          {
            "batsmanid": 24729,
            "batsmanname": "Harshit Rana",
            "overnbr": 19.5,
            "runs": 173,
            "ballnbr": 119
          }
        ]
      },
      "extras": {
        "legbyes": 2,
        "byes": 0,
        "wides": 4,
        "noballs": 0,
        "penalty": 0,
        "total": 6
      },
      "score": 174,
      "wickets": 8,
      "overs": 20,
      "runrate": 8.7,
      "batteamname": "Kolkata Knight Riders",
      "batteamsname": "KKR",
      "isdeclared": False,
      "isfollowon": False,
      "ballnbr": 120,
      "rpb": 1.45,
      "partnership": {
        "partnership": [
          {
            "id": 0,
            "bat1id": 8520,
            "bat1name": "de Kock",
            "bat1runs": 4,
            "bat1fours": 1,
            "bat1sixes": 0,
            "bat2id": 2276,
            "bat2name": "Narine",
            "bat2runs": 0,
            "bat2fours": 0,
            "bat2sixes": 0,
            "totalruns": 4,
            "totalballs": 5,
            "bat1balls": 0,
            "bat2balls": 0,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 0,
            "bat1twos": 0,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 0,
            "bat1sixers": 0,
            "bat2ones": 0,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          },
          {
            "id": 0,
            "bat1id": 1447,
            "bat1name": "Rahane",
            "bat1runs": 55,
            "bat1fours": 6,
            "bat1sixes": 4,
            "bat2id": 2276,
            "bat2name": "Narine",
            "bat2runs": 44,
            "bat2fours": 5,
            "bat2sixes": 3,
            "totalruns": 103,
            "totalballs": 55,
            "bat1balls": 0,
            "bat2balls": 0,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 0,
            "bat1twos": 0,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 0,
            "bat1sixers": 0,
            "bat2ones": 0,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          },
          {
            "id": 0,
            "bat1id": 1447,
            "bat1name": "Rahane",
            "bat1runs": 1,
            "bat1fours": 0,
            "bat1sixes": 0,
            "bat2id": 10917,
            "bat2name": "Venkatesh Iyer",
            "bat2runs": 1,
            "bat2fours": 0,
            "bat2sixes": 0,
            "totalruns": 2,
            "totalballs": 3,
            "bat1balls": 0,
            "bat2balls": 0,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 0,
            "bat1twos": 0,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 0,
            "bat1sixers": 0,
            "bat2ones": 0,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          },
          {
            "id": 0,
            "bat1id": 22566,
            "bat1name": "Angkrish Raghuvanshi",
            "bat1runs": 10,
            "bat1fours": 2,
            "bat1sixes": 0,
            "bat2id": 10917,
            "bat2name": "Venkatesh Iyer",
            "bat2runs": 5,
            "bat2fours": 1,
            "bat2sixes": 0,
            "totalruns": 16,
            "totalballs": 10,
            "bat1balls": 0,
            "bat2balls": 0,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 0,
            "bat1twos": 0,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 0,
            "bat1sixers": 0,
            "bat2ones": 0,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          },
          {
            "id": 0,
            "bat1id": 22566,
            "bat1name": "Angkrish Raghuvanshi",
            "bat1runs": 8,
            "bat1fours": 0,
            "bat1sixes": 0,
            "bat2id": 10896,
            "bat2name": "Rinku Singh",
            "bat2runs": 12,
            "bat2fours": 1,
            "bat2sixes": 0,
            "totalruns": 20,
            "totalballs": 17,
            "bat1balls": 0,
            "bat2balls": 0,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 0,
            "bat1twos": 0,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 0,
            "bat1sixers": 0,
            "bat2ones": 0,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          },
          {
            "id": 0,
            "bat1id": 22566,
            "bat1name": "Angkrish Raghuvanshi",
            "bat1runs": 1,
            "bat1fours": 0,
            "bat1sixes": 0,
            "bat2id": 7736,
            "bat2name": "Russell",
            "bat2runs": 4,
            "bat2fours": 1,
            "bat2sixes": 0,
            "totalruns": 5,
            "totalballs": 4,
            "bat1balls": 0,
            "bat2balls": 0,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 0,
            "bat1twos": 0,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 0,
            "bat1sixers": 0,
            "bat2ones": 0,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          },
          {
            "id": 0,
            "bat1id": 22566,
            "bat1name": "Angkrish Raghuvanshi",
            "bat1runs": 11,
            "bat1fours": 0,
            "bat1sixes": 1,
            "bat2id": 12337,
            "bat2name": "Ramandeep Singh ",
            "bat2runs": 6,
            "bat2fours": 0,
            "bat2sixes": 0,
            "totalruns": 18,
            "totalballs": 19,
            "bat1balls": 0,
            "bat2balls": 0,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 0,
            "bat1twos": 0,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 0,
            "bat1sixers": 0,
            "bat2ones": 0,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          },
          {
            "id": 0,
            "bat1id": 24729,
            "bat1name": "Harshit Rana",
            "bat1runs": 5,
            "bat1fours": 1,
            "bat1sixes": 0,
            "bat2id": 12337,
            "bat2name": "Ramandeep Singh ",
            "bat2runs": 0,
            "bat2fours": 0,
            "bat2sixes": 0,
            "totalruns": 5,
            "totalballs": 6,
            "bat1balls": 0,
            "bat2balls": 0,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 0,
            "bat1twos": 0,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 0,
            "bat1sixers": 0,
            "bat2ones": 0,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          },
          {
            "id": 0,
            "bat1id": 13143,
            "bat1name": "Spencer Johnson",
            "bat1runs": 1,
            "bat1fours": 0,
            "bat1sixes": 0,
            "bat2id": 12337,
            "bat2name": "Ramandeep Singh ",
            "bat2runs": 0,
            "bat2fours": 0,
            "bat2sixes": 0,
            "totalruns": 1,
            "totalballs": 1,
            "bat1balls": 0,
            "bat2balls": 0,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 0,
            "bat1twos": 0,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 0,
            "bat1sixers": 0,
            "bat2ones": 0,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          }
        ]
      }
    },
    {
      "inningsid": 2,
      "batsman": [
        {
          "id": 10479,
          "balls": 31,
          "runs": 56,
          "fours": 9,
          "sixes": 2,
          "strkrate": "180.65",
          "name": "Philip Salt",
          "nickname": "Phil Salt",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "c Spencer Johnson b Varun Chakaravarthy",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": True,
          "playingxichange": ""
        },
        {
          "id": 1413,
          "balls": 36,
          "runs": 59,
          "fours": 4,
          "sixes": 3,
          "strkrate": "163.89",
          "name": "Virat Kohli",
          "nickname": "Kohli",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "not out",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 13088,
          "balls": 10,
          "runs": 10,
          "fours": 1,
          "sixes": 0,
          "strkrate": "100",
          "name": "Devdutt Padikkal",
          "nickname": "Padikkal",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "c Ramandeep Singh b Narine",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "MIN",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 10636,
          "balls": 16,
          "runs": 34,
          "fours": 5,
          "sixes": 1,
          "strkrate": "212.5",
          "name": "Rajat Patidar",
          "nickname": "Rajat Patidar",
          "iscaptain": True,
          "iskeeper": False,
          "outdec": "c Rinku Singh b Vaibhav Arora",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 10045,
          "balls": 5,
          "runs": 15,
          "fours": 2,
          "sixes": 1,
          "strkrate": "300",
          "name": "Liam Livingstone",
          "nickname": "Livingstone",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "not out",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": True,
          "playingxichange": ""
        },
        {
          "id": 10214,
          "balls": 0,
          "runs": 0,
          "fours": 0,
          "sixes": 0,
          "strkrate": "0",
          "name": "Jitesh Sharma",
          "nickname": "Jitesh Sharma",
          "iscaptain": False,
          "iskeeper": True,
          "outdec": "",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 13169,
          "balls": 0,
          "runs": 0,
          "fours": 0,
          "sixes": 0,
          "strkrate": "0",
          "name": "Tim David",
          "nickname": "Tim David",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": True,
          "playingxichange": ""
        },
        {
          "id": 11311,
          "balls": 0,
          "runs": 0,
          "fours": 0,
          "sixes": 0,
          "strkrate": "0",
          "name": "Krunal Pandya",
          "nickname": "Krunal Pandya",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 14274,
          "balls": 0,
          "runs": 0,
          "fours": 0,
          "sixes": 0,
          "strkrate": "0",
          "name": "Rasikh Dar Salam",
          "nickname": "Rasikh Salam",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 36487,
          "balls": 0,
          "runs": 0,
          "fours": 0,
          "sixes": 0,
          "strkrate": "0",
          "name": "Suyash Sharma",
          "nickname": "Suyash Sharma",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "MOUT",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 6258,
          "balls": 0,
          "runs": 0,
          "fours": 0,
          "sixes": 0,
          "strkrate": "0",
          "name": "Josh Hazlewood",
          "nickname": "Hazlewood",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": True,
          "playingxichange": ""
        },
        {
          "id": 14172,
          "balls": 0,
          "runs": 0,
          "fours": 0,
          "sixes": 0,
          "strkrate": "0",
          "name": "Yash Dayal",
          "nickname": "Yash Dayal",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        }
      ],
      "bowler": [
        {
          "id": 15861,
          "overs": "3",
          "maidens": 0,
          "wickets": 1,
          "runs": 42,
          "economy": "14",
          "name": "Vaibhav Arora",
          "nickname": "Vaibhav Arora",
          "iscaptain": False,
          "iskeeper": False,
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "dots": 0,
          "balls": 30,
          "rpb": 1.4,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "ispremiumfree": False,
          "inmatchchange": "MIN",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 13143,
          "overs": "2.2",
          "maidens": 0,
          "wickets": 0,
          "runs": 31,
          "economy": "13.3",
          "name": "Spencer Johnson",
          "nickname": "Spencer Johnson",
          "iscaptain": False,
          "iskeeper": False,
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "dots": 0,
          "balls": 22,
          "rpb": 1.41,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": True,
          "playingxichange": ""
        },
        {
          "id": 12926,
          "overs": "4",
          "maidens": 0,
          "wickets": 1,
          "runs": 43,
          "economy": "10.8",
          "name": "Varun Chakaravarthy",
          "nickname": "Varun Chakaravarthy",
          "iscaptain": False,
          "iskeeper": False,
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "dots": 0,
          "balls": 40,
          "rpb": 1.07,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 24729,
          "overs": "3",
          "maidens": 0,
          "wickets": 0,
          "runs": 32,
          "economy": "10.7",
          "name": "Harshit Rana",
          "nickname": "Harshit Rana",
          "iscaptain": False,
          "iskeeper": False,
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "dots": 0,
          "balls": 30,
          "rpb": 1.07,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 2276,
          "overs": "4",
          "maidens": 0,
          "wickets": 1,
          "runs": 27,
          "economy": "6.8",
          "name": "Sunil Narine",
          "nickname": "Narine",
          "iscaptain": False,
          "iskeeper": False,
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "dots": 0,
          "balls": 40,
          "rpb": 0.68,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": True,
          "playingxichange": ""
        }
      ],
      "fow": {
        "fow": [
          {
            "batsmanid": 10479,
            "batsmanname": "Philip Salt",
            "overnbr": 8.3,
            "runs": 95,
            "ballnbr": 51
          },
          {
            "batsmanid": 13088,
            "batsmanname": "Devdutt Padikkal",
            "overnbr": 11.4,
            "runs": 118,
            "ballnbr": 70
          },
          {
            "batsmanid": 10636,
            "batsmanname": "Rajat Patidar",
            "overnbr": 15.3,
            "runs": 162,
            "ballnbr": 93
          }
        ]
      },
      "extras": {
        "legbyes": 2,
        "byes": 0,
        "wides": 1,
        "noballs": 0,
        "penalty": 0,
        "total": 3
      },
      "score": 177,
      "wickets": 3,
      "overs": 16.2,
      "runrate": 10.84,
      "batteamname": "Royal Challengers Bengaluru",
      "batteamsname": "RCB",
      "isdeclared": False,
      "isfollowon": False,
      "ballnbr": 98,
      "rpb": 1.81,
      "partnership": {
        "partnership": [
          {
            "id": 0,
            "bat1id": 10479,
            "bat1name": "Phil Salt",
            "bat1runs": 56,
            "bat1fours": 9,
            "bat1sixes": 2,
            "bat2id": 1413,
            "bat2name": "Kohli",
            "bat2runs": 36,
            "bat2fours": 3,
            "bat2sixes": 2,
            "totalruns": 95,
            "totalballs": 51,
            "bat1balls": 31,
            "bat2balls": 20,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 6,
            "bat1twos": 1,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 9,
            "bat1sixers": 2,
            "bat2ones": 8,
            "bat2twos": 2,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 3,
            "bat2sixers": 2
          },
          {
            "id": 0,
            "bat1id": 13088,
            "bat1name": "Padikkal",
            "bat1runs": 10,
            "bat1fours": 1,
            "bat1sixes": 0,
            "bat2id": 1413,
            "bat2name": "Kohli",
            "bat2runs": 13,
            "bat2fours": 0,
            "bat2sixes": 1,
            "totalruns": 23,
            "totalballs": 19,
            "bat1balls": 10,
            "bat2balls": 9,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 6,
            "bat1twos": 0,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 1,
            "bat1sixers": 0,
            "bat2ones": 7,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 1
          },
          {
            "id": 0,
            "bat1id": 10636,
            "bat1name": "Rajat Patidar",
            "bat1runs": 34,
            "bat1fours": 5,
            "bat1sixes": 1,
            "bat2id": 1413,
            "bat2name": "Kohli",
            "bat2runs": 10,
            "bat2fours": 1,
            "bat2sixes": 0,
            "totalruns": 44,
            "totalballs": 23,
            "bat1balls": 16,
            "bat2balls": 7,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 4,
            "bat1twos": 2,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 5,
            "bat1sixers": 1,
            "bat2ones": 6,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 1,
            "bat2sixers": 0
          },
          {
            "id": 0,
            "bat1id": 10045,
            "bat1name": "Livingstone",
            "bat1runs": 15,
            "bat1fours": 2,
            "bat1sixes": 1,
            "bat2id": 1413,
            "bat2name": "Kohli",
            "bat2runs": 0,
            "bat2fours": 0,
            "bat2sixes": 0,
            "totalruns": 15,
            "totalballs": 5,
            "bat1balls": 5,
            "bat2balls": 0,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 1,
            "bat1twos": 0,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 2,
            "bat1sixers": 1,
            "bat2ones": 0,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          }
        ]
      }
    }
  ],
  "ismatchcomplete": True,
  "appindex": {
    "seotitle": "Cricket scorecard - KKR vs RCB 1st Match,Indian Premier League 2025 | Cricbuzz.com",
    "weburl": "http://www.cricbuzz.com/live-cricket-scorecard/114960/kkr-vs-rcb-1st-match-indian-premier-league-2025"
  },
  "status": "Royal Challengers Bengaluru won by 7 wkts",
  "responselastupdated": 1765220266
}

In [14]:
names = ['Ruturaj Gaikwad', 'Andre Siddarth C', 'Shaik Rasheed', 'Rahul Tripathi', 'Ayush Mhatre', 'Dewald Brevis', 'Shivam Dube', 'Rachin Ravindra', 'Deepak Hooda', 'Vijay Shankar', 'Ramakrishna Ghosh', 'Ravindra Jadeja', 'Anshul Kamboj', 'Jamie Overton', 'Sam Curran', 'Ravichandran Ashwin', 'Devon Conway', 'MS Dhoni', 'Vansh Bedi', 'Urvil Patel', 'Kamlesh Nagarkoti', 'Shreyas Gopal', 'Matheesha Pathirana', 'Mukesh Choudhary', 'Nathan Ellis', 'Noor Ahmad', 'Khaleel Ahmed', 'Vaibhav Suryavanshi', 'Shimron Hetmyer', 'Yashasvi Jaiswal', 'Shubham Dubey', 'Riyan Parag', 'Wanindu Hasaranga', 'Sanju Samson', 'Dhruv Jurel', 'Kunal Singh Rathore', 'Lhuan-dre Pretorius', 'Yudhvir Singh Charak', 'Tushar Deshpande', 'Kumar Kartikeya', 'Akash Madhwal', 'Kwena Maphaka', 'Maheesh Theekshana', 'Fazalhaq Farooqi', 'Ashok Sharma', 'Jofra Archer', 'Nandre Burger', 'Manish Pandey', 'Ajinkya Rahane', 'Rinku Singh', 'Angkrish Raghuvanshi', 'Anukul Roy', 'Ramandeep Singh', 'Venkatesh Iyer', 'Moeen Ali', 'Sunil Narine', 'Andre Russell', 'Quinton de Kock', 'Rahmanullah Gurbaz', 'Luvnith Sisodia', 'Varun Chakaravarthy', 'Mayank Markande', 'Vaibhav Arora', 'Harshit Rana', 'Anrich Nortje', 'Spencer Johnson', 'Chetan Sakariya', 'Shivam Shukla', 'Atharva Taide', 'Travis Head', 'Abhinav Manohar', 'Sachin Baby', 'Aniket Verma', 'Nitish Kumar Reddy', 'Abhishek Sharma', 'Kamindu Mendis', 'Wiaan Mulder', 'Harsh Dubey', 'Heinrich Klaasen', 'Ishan Kishan', 'Zeeshan Ansari', 'Pat Cummins', 'Mohammed Shami', 'Harshal Patel', 'Rahul Chahar', 'Simarjeet Singh', 'Eshan Malinga', 'Jaydev Unadkat', 'Virat Kohli', 'Rajat Patidar', 'Swastik Chikara', 'Tim David', 'Mayank Agarwal', 'Krunal Pandya', 'Liam Livingstone', 'Manoj Bhandage', 'Romario Shepherd', 'Swapnil Singh', 'Mohit Rathee', 'Philip Salt', 'Jitesh Sharma', 'Tim Seifert', 'Josh Hazlewood', 'Bhuvneshwar Kumar', 'Rasikh Dar Salam', 'Suyash Sharma', 'Yash Dayal', 'Nuwan Thushara', 'Abhinandan Singh', 'Blessing Muzarabani', 'Faf du Plessis', 'Karun Nair', 'Sameer Rizvi', 'Sediqullah Atal', 'Ashutosh Sharma', 'Tripurana Vijay', 'Axar Patel', 'Darshan Nalkande', 'Ajay Jadav Mandal', 'Manvanth Kumar L', 'Madhav Tiwari', 'Tristan Stubbs', 'Abishek Porel', 'Donovan Ferreira', 'KL Rahul', 'Vipraj Nigam', 'Kuldeep Yadav', 'Dushmantha Chameera', 'Mohit Sharma', 'T Natarajan', 'Mukesh Kumar', 'Mustafizur Rahman', 'Nehal Wadhera', 'Harnoor Singh', 'Shreyas Iyer', 'Pyla Avinash', 'Priyansh Arya', 'Musheer Khan', 'Marcus Stoinis', 'Aaron Hardie', 'Suryansh Shedge', 'Shashank Singh', 'Mitchell Owen', 'Praveen Dubey', 'Azmatullah Omarzai', 'Prabhsimran Singh', 'Josh Inglis', 'Vishnu Vinod', 'Harpreet Brar', 'Arshdeep Singh', 'Yuzvendra Chahal', 'Vijaykumar Vyshak', 'Kuldeep Sen', 'Yash Thakur', 'Xavier Bartlett', 'Kyle Jamieson', 'Rohit Sharma', 'Suryakumar Yadav', 'Tilak Varma', 'Naman Dhir', 'Bevon Jacobs', 'Hardik Pandya', 'Raj Bawa', 'Charith Asalanka', 'Mitchell Santner', 'Arjun Tendulkar', 'Krishnan Shrijith', 'Robin Minz', 'Jonny Bairstow', 'Jasprit Bumrah', 'Ashwani Kumar', 'Reece Topley', 'Karn Sharma', 'Trent Boult', 'Satyanarayana Raju', 'Deepak Chahar', 'Mujeeb Ur Rahman', 'Raghu Sharma', 'Richard Gleeson', 'Sai Sudharsan', 'Shubman Gill', 'Shahrukh Khan', 'Rahul Tewatia', 'Nishant Sindhu', 'Sherfane Rutherford', 'Mahipal Lomror', 'Dasun Shanaka', 'Rashid Khan', 'Ravisrinivasan Sai Kishore', 'Arshad Khan', 'Jayant Yadav', 'Karim Janat', 'Washington Sundar', 'Kumar Kushagra', 'Anuj Rawat', 'Kusal Mendis', 'Gerald Coetzee', 'Manav Suthar', 'Gurnoor Brar', 'Ishant Sharma', 'Kulwant Khejroliya', 'Prasidh Krishna', 'Mohammed Siraj', 'Himmat Singh', 'David Miller', 'Aiden Markram', 'Ayush Badoni', 'Mitchell Marsh', 'Abdul Samad', 'Arshin Kulkarni', 'Yuvraj Chaudhary', 'Shahbaz Ahmed', 'RS Hangargekar', 'Shardul Thakur', 'Matthew Breetzke', 'Nicholas Pooran', 'Aryan Juyal', 'Rishabh Pant', 'Ravi Bishnoi', 'Akash Deep', 'Manimaran Siddharth', 'Shamar Joseph', 'Avesh Khan', 'Prince Yadav', 'Akash Maharaj Singh', 'Digvesh Singh Rathi', 'William ORourke']

names2 = ['Ravichandran Smaran','Mitchell Owen','Ayush Mhatre','Dasun Shanaka','Shardul Thakur','Dhruv Jurel','Nitish Reddy','Travis Head', 'Varun Chakaravarthy', 'Rahul Chahar', 'Mukesh Choudhary', 'Harshit Rana', 'Ishant Sharma', 'Jaydev Unadkat', 'Mukesh Kumar', 'Abdul Samad', 'Riyan Parag', 'Khaleel Ahmed', 'Avesh Khan', 'Faf du Plessis', 'Arjun Tendulkar', 'Mohammed Shami', 'Shivam Dube', 'Lockie Ferguson', 'Josh Hazlewood', 'Prabhsimran Singh', 'Rishabh Pant', 'Corbin Bosch', 'Mohammed Siraj', 'Prasidh Krishna', 'Marcus Stoinis', 'Harpreet Brar', 'Rahmanullah Gurbaz', 'Rashid Khan', 'Washington Sundar', 'Hardik Pandya', 'Heinrich Klaasen', 'Rinku Singh', 'Nehal Wadhera', 'Romario Shepherd', 'Manav Suthar', 'Vijaykumar Vyshak', 'Himmat Singh', 'Ayush Badoni', 'Liam Livingstone', 'Nathan Ellis', 'Moeen Ali', 'Karn Sharma', 'Yashasvi Jaiswal', 'Shimron Hetmyer', 'Axar Patel', 'Mayank Yadav', 'Abhinav Manohar', 'Ashutosh Sharma', 'Rachin Ravindra', 'Shahrukh Khan', 'Anrich Nortje', 'Mayank Markande', 'Yuzvendra Chahal', 'Tushar Deshpande', 'Noor Ahmad', 'Kagiso Rabada', 'Marco Jansen', 'Virat Kohli', 'Abhishek Sharma', 'Jitesh Sharma', 'Harnoor Singh', 'Bhuvneshwar Kumar', 'Abishek Porel', 'Angkrish Raghuvanshi', 'Kuldeep Yadav', 'David Miller', 'Anuj Rawat', 'Josh Inglis', 'Kumar Kartikeya', 'Akash Deep', 'Rahul Tewatia', 'Ramandeep Singh', 'Sherfane Rutherford', 'Glenn Maxwell', 'Sandeep Sharma', 'Suryakumar Yadav', 'Shamar Joseph', 'Pat Cummins', 'Quinton de Kock', 'Ravichandran Ashwin', 'Jasprit Bumrah', 'Sai Sudharsan', 'Shreyas Iyer', 'Swastik Chikara', 'Rajvardhan Hangargekar', 'Manoj Bhandage', 'Nitish Rana', 'Rasikh Salam Dar', 'Deepak Chahar', 'MS Dhoni', 'Aaron Hardie', 'Priyansh Arya', 'Phil Salt', 'Sameer Rizvi', 'Mitchell Santner', 'Manish Pandey', 'Suyash Sharma', 'Kamlesh Nagarkoti', 'Will Jacks', 'Azmatullah Omarzai', 'Adam Zampa', 'Spencer Johnson', 'Jamie Overton', 'Shashank Singh', 'Rovman Powell', 'Suryansh Shedge', 'Maheesh Theekshana', 'Ruturaj Gaikwad', 'Shubman Gill', 'Mohit Sharma', 'Sai Kishore', 'Raj Bawa', 'Ishan Kishan', 'Mitchell Marsh', 'Nitish Reddy', 'Karim Janat', 'Yash Dayal', 'Bevon Jacobs', 'Ryan Rickelton', 'Rajat Patidar', 'Tristan Stubbs', 'Gerald Coetzee', 'Glenn Phillips', 'Tim David', 'Ravi Bishnoi', 'Donovan Ferreira', 'Jayant Yadav', 'Trent Boult', 'Jofra Archer', 'Akash Madhwal', 'Darshan Nalkande', 'Kwena Maphaka', 'KL Rahul', 'Arshdeep Singh', 'Aiden Markram', 'Sachin Baby', 'Dushmantha Chameera', 'Naman Dhir', 'Karun Nair', 'Wanindu Hasaranga', 'Arshad Khan', 'Devdutt Padikkal', 'Robin Minz', 'Shahbaz Ahmed', 'Mohsin Khan', 'Krunal Pandya', 'Ravindra Jadeja', 'Mitchell Starc', 'Sanju Samson', 'Jos Buttler', 'Atharva Taide', 'Musheer Khan', 'Devon Conway', 'Venkatesh Iyer', 'Andre Russell', 'Sunil Narine', 'Chetan Sakariya', 'T Natarajan', 'Ajinkya Rahane', 'Shreyas Gopal', 'Tilak Varma', 'Vijay Shankar', 'Shubham Dubey', 'Anukul Roy', 'Deepak Hooda', 'Harshal Patel', 'Rahul Tripathi', 'Lungi Ngidi', 'Matheesha Pathirana', 'Vaibhav Arora', 'Nicholas Pooran', 'Jake Fraser-McGurk', 'Sam Curran', 'Rohit Sharma', 'Mujeeb ur Rahman', 'Anshul Kamboj', 'Mahipal Lomror']

missing = [x for x in names2 if find_full_name(names,x) == None]

print(missing)

['Ravichandran Smaran', 'Lockie Ferguson', 'Corbin Bosch', 'Mayank Yadav', 'Kagiso Rabada', 'Marco Jansen', 'Glenn Maxwell', 'Sandeep Sharma', 'Rajvardhan Hangargekar', 'Nitish Rana', 'Will Jacks', 'Adam Zampa', 'Rovman Powell', 'Ryan Rickelton', 'Glenn Phillips', 'Devdutt Padikkal', 'Mohsin Khan', 'Mitchell Starc', 'Jos Buttler', 'Lungi Ngidi', 'Jake Fraser-McGurk', 'Mujeeb ur Rahman']


In [10]:
#test code
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
import os
import dill

names = ['Ruturaj Gaikwad', 'Andre Siddarth C', 'Shaik Rasheed', 'Rahul Tripathi', 'Ayush Mhatre', 'Dewald Brevis', 'Shivam Dube', 'Rachin Ravindra', 'Deepak Hooda', 'Vijay Shankar', 'Ramakrishna Ghosh', 'Ravindra Jadeja', 'Anshul Kamboj', 'Jamie Overton', 'Sam Curran', 'Ravichandran Ashwin', 'Devon Conway', 'MS Dhoni', 'Vansh Bedi', 'Urvil Patel', 'Kamlesh Nagarkoti', 'Shreyas Gopal', 'Matheesha Pathirana', 'Mukesh Choudhary', 'Nathan Ellis', 'Noor Ahmad', 'Khaleel Ahmed', 'Vaibhav Suryavanshi', 'Shimron Hetmyer', 'Yashasvi Jaiswal', 'Shubham Dubey', 'Riyan Parag', 'Wanindu Hasaranga', 'Sanju Samson', 'Dhruv Jurel', 'Kunal Singh Rathore', 'Lhuan-dre Pretorius', 'Yudhvir Singh Charak', 'Tushar Deshpande', 'Kumar Kartikeya', 'Akash Madhwal', 'Kwena Maphaka', 'Maheesh Theekshana', 'Fazalhaq Farooqi', 'Ashok Sharma', 'Jofra Archer', 'Nandre Burger', 'Manish Pandey', 'Ajinkya Rahane', 'Rinku Singh', 'Angkrish Raghuvanshi', 'Anukul Roy', 'Ramandeep Singh', 'Venkatesh Iyer', 'Moeen Ali', 'Sunil Narine', 'Andre Russell', 'Quinton de Kock', 'Rahmanullah Gurbaz', 'Luvnith Sisodia', 'Varun Chakaravarthy', 'Mayank Markande', 'Vaibhav Arora', 'Harshit Rana', 'Anrich Nortje', 'Spencer Johnson', 'Chetan Sakariya', 'Shivam Shukla', 'Atharva Taide', 'Travis Head', 'Abhinav Manohar', 'Sachin Baby', 'Aniket Verma', 'Nitish Kumar Reddy', 'Abhishek Sharma', 'Kamindu Mendis', 'Wiaan Mulder', 'Harsh Dubey', 'Heinrich Klaasen', 'Ishan Kishan', 'Zeeshan Ansari', 'Pat Cummins', 'Mohammed Shami', 'Harshal Patel', 'Rahul Chahar', 'Simarjeet Singh', 'Eshan Malinga', 'Jaydev Unadkat', 'Virat Kohli', 'Rajat Patidar', 'Swastik Chikara', 'Tim David', 'Mayank Agarwal', 'Krunal Pandya', 'Liam Livingstone', 'Manoj Bhandage', 'Romario Shepherd', 'Swapnil Singh', 'Mohit Rathee', 'Philip Salt', 'Jitesh Sharma', 'Tim Seifert', 'Josh Hazlewood', 'Bhuvneshwar Kumar', 'Rasikh Dar Salam', 'Suyash Sharma', 'Yash Dayal', 'Nuwan Thushara', 'Abhinandan Singh', 'Blessing Muzarabani', 'Faf du Plessis', 'Karun Nair', 'Sameer Rizvi', 'Sediqullah Atal', 'Ashutosh Sharma', 'Tripurana Vijay', 'Axar Patel', 'Darshan Nalkande', 'Ajay Jadav Mandal', 'Manvanth Kumar L', 'Madhav Tiwari', 'Tristan Stubbs', 'Abishek Porel', 'Donovan Ferreira', 'KL Rahul', 'Vipraj Nigam', 'Kuldeep Yadav', 'Dushmantha Chameera', 'Mohit Sharma', 'T Natarajan', 'Mukesh Kumar', 'Mustafizur Rahman', 'Nehal Wadhera', 'Harnoor Singh', 'Shreyas Iyer', 'Pyla Avinash', 'Priyansh Arya', 'Musheer Khan', 'Marcus Stoinis', 'Aaron Hardie', 'Suryansh Shedge', 'Shashank Singh', 'Mitchell Owen', 'Praveen Dubey', 'Azmatullah Omarzai', 'Prabhsimran Singh', 'Josh Inglis', 'Vishnu Vinod', 'Harpreet Brar', 'Arshdeep Singh', 'Yuzvendra Chahal', 'Vijaykumar Vyshak', 'Kuldeep Sen', 'Yash Thakur', 'Xavier Bartlett', 'Kyle Jamieson', 'Rohit Sharma', 'Suryakumar Yadav', 'Tilak Varma', 'Naman Dhir', 'Bevon Jacobs', 'Hardik Pandya', 'Raj Bawa', 'Charith Asalanka', 'Mitchell Santner', 'Arjun Tendulkar', 'Krishnan Shrijith', 'Robin Minz', 'Jonny Bairstow', 'Jasprit Bumrah', 'Ashwani Kumar', 'Reece Topley', 'Karn Sharma', 'Trent Boult', 'Satyanarayana Raju', 'Deepak Chahar', 'Mujeeb Ur Rahman', 'Raghu Sharma', 'Richard Gleeson', 'Sai Sudharsan', 'Shubman Gill', 'Shahrukh Khan', 'Rahul Tewatia', 'Nishant Sindhu', 'Sherfane Rutherford', 'Mahipal Lomror', 'Dasun Shanaka', 'Rashid Khan', 'Ravisrinivasan Sai Kishore', 'Arshad Khan', 'Jayant Yadav', 'Karim Janat', 'Washington Sundar', 'Kumar Kushagra', 'Anuj Rawat', 'Kusal Mendis', 'Gerald Coetzee', 'Manav Suthar', 'Gurnoor Brar', 'Ishant Sharma', 'Kulwant Khejroliya', 'Prasidh Krishna', 'Mohammed Siraj', 'Himmat Singh', 'David Miller', 'Aiden Markram', 'Ayush Badoni', 'Mitchell Marsh', 'Abdul Samad', 'Arshin Kulkarni', 'Yuvraj Chaudhary', 'Shahbaz Ahmed', 'RS Hangargekar', 'Shardul Thakur', 'Matthew Breetzke', 'Nicholas Pooran', 'Aryan Juyal', 'Rishabh Pant', 'Ravi Bishnoi', 'Akash Deep', 'Manimaran Siddharth', 'Shamar Joseph', 'Avesh Khan', 'Prince Yadav', 'Akash Maharaj Singh', 'Digvesh Singh Rathi', 'William ORourke']
roles = ['BAT', 'BAT', 'BAT', 'BAT', 'BAT', 'BAT', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'WK', 'WK', 'WK', 'WK', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BAT', 'BAT', 'BAT', 'BAT', 'AR', 'AR', 'WK', 'WK', 'WK', 'WK', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BAT', 'BAT', 'BAT', 'BAT', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'WK', 'WK', 'WK', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BAT', 'BAT', 'BAT', 'BAT', 'BAT', 'AR', 'AR', 'AR', 'AR', 'AR', 'WK', 'WK', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BAT', 'BAT', 'BAT', 'BAT', 'BAT', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'WK', 'WK', 'WK', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BAT', 'BAT', 'BAT', 'BAT', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'WK', 'WK', 'WK', 'WK', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BAT', 'BAT', 'BAT', 'BAT', 'BAT', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'WK', 'WK', 'WK', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BAT', 'BAT', 'BAT', 'BAT', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'WK', 'WK', 'WK', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BAT', 'BAT', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'WK', 'WK', 'WK', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BAT', 'BAT', 'BAT', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'WK', 'WK', 'WK', 'WK', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL']
squads = {'Chennai Super Kings': ['Ruturaj Gaikwad', 'Andre Siddarth C', 'Shaik Rasheed', 'Rahul Tripathi', 'Ayush Mhatre', 'Dewald Brevis', 'Shivam Dube', 'Rachin Ravindra', 'Deepak Hooda', 'Vijay Shankar', 'Ramakrishna Ghosh', 'Ravindra Jadeja', 'Anshul Kamboj', 'Jamie Overton', 'Sam Curran', 'Ravichandran Ashwin', 'Devon Conway', 'MS Dhoni', 'Vansh Bedi', 'Urvil Patel', 'Kamlesh Nagarkoti', 'Shreyas Gopal', 'Matheesha Pathirana', 'Mukesh Choudhary', 'Nathan Ellis', 'Noor Ahmad', 'Khaleel Ahmed'], 'Rajasthan Royals': ['Vaibhav Suryavanshi', 'Shimron Hetmyer', 'Yashasvi Jaiswal', 'Shubham Dubey', 'Riyan Parag', 'Wanindu Hasaranga', 'Sanju Samson', 'Dhruv Jurel', 'Kunal Singh Rathore', 'Lhuan-dre Pretorius', 'Yudhvir Singh Charak', 'Tushar Deshpande', 'Kumar Kartikeya', 'Akash Madhwal', 'Kwena Maphaka', 'Maheesh Theekshana', 'Fazalhaq Farooqi', 'Ashok Sharma', 'Jofra Archer', 'Nandre Burger'], 'Kolkata Knight Riders': ['Manish Pandey', 'Ajinkya Rahane', 'Rinku Singh', 'Angkrish Raghuvanshi', 'Anukul Roy', 'Ramandeep Singh', 'Venkatesh Iyer', 'Moeen Ali', 'Sunil Narine', 'Andre Russell', 'Quinton de Kock', 'Rahmanullah Gurbaz', 'Luvnith Sisodia', 'Varun Chakaravarthy', 'Mayank Markande', 'Vaibhav Arora', 'Harshit Rana', 'Anrich Nortje', 'Spencer Johnson', 'Chetan Sakariya', 'Shivam Shukla'], 'Sunrisers Hyderabad': ['Atharva Taide', 'Travis Head', 'Abhinav Manohar', 'Sachin Baby', 'Aniket Verma', 'Nitish Kumar Reddy', 'Abhishek Sharma', 'Kamindu Mendis', 'Wiaan Mulder', 'Harsh Dubey', 'Heinrich Klaasen', 'Ishan Kishan', 'Zeeshan Ansari', 'Pat Cummins', 'Mohammed Shami', 'Harshal Patel', 'Rahul Chahar', 'Simarjeet Singh', 'Eshan Malinga', 'Jaydev Unadkat'], 'Royal Challengers Bengaluru': ['Virat Kohli', 'Rajat Patidar', 'Swastik Chikara', 'Tim David', 'Mayank Agarwal', 'Krunal Pandya', 'Liam Livingstone', 'Manoj Bhandage', 'Romario Shepherd', 'Swapnil Singh', 'Mohit Rathee', 'Philip Salt', 'Jitesh Sharma', 'Tim Seifert', 'Josh Hazlewood', 'Bhuvneshwar Kumar', 'Rasikh Dar Salam', 'Suyash Sharma', 'Yash Dayal', 'Nuwan Thushara', 'Abhinandan Singh', 'Blessing Muzarabani'], 'Delhi Capitals': ['Faf du Plessis', 'Karun Nair', 'Sameer Rizvi', 'Sediqullah Atal', 'Ashutosh Sharma', 'Tripurana Vijay', 'Axar Patel', 'Darshan Nalkande', 'Ajay Jadav Mandal', 'Manvanth Kumar L', 'Madhav Tiwari', 'Tristan Stubbs', 'Abishek Porel', 'Donovan Ferreira', 'KL Rahul', 'Vipraj Nigam', 'Kuldeep Yadav', 'Dushmantha Chameera', 'Mohit Sharma', 'T Natarajan', 'Mukesh Kumar', 'Mustafizur Rahman'], 'Punjab Kings': ['Nehal Wadhera', 'Harnoor Singh', 'Shreyas Iyer', 'Pyla Avinash', 'Priyansh Arya', 'Musheer Khan', 'Marcus Stoinis', 'Aaron Hardie', 'Suryansh Shedge', 'Shashank Singh', 'Mitchell Owen', 'Praveen Dubey', 'Azmatullah Omarzai', 'Prabhsimran Singh', 'Josh Inglis', 'Vishnu Vinod', 'Harpreet Brar', 'Arshdeep Singh', 'Yuzvendra Chahal', 'Vijaykumar Vyshak', 'Kuldeep Sen', 'Yash Thakur', 'Xavier Bartlett', 'Kyle Jamieson'], 'Mumbai Indians': ['Rohit Sharma', 'Suryakumar Yadav', 'Tilak Varma', 'Naman Dhir', 'Bevon Jacobs', 'Hardik Pandya', 'Raj Bawa', 'Charith Asalanka', 'Mitchell Santner', 'Arjun Tendulkar', 'Krishnan Shrijith', 'Robin Minz', 'Jonny Bairstow', 'Jasprit Bumrah', 'Ashwani Kumar', 'Reece Topley', 'Karn Sharma', 'Trent Boult', 'Satyanarayana Raju', 'Deepak Chahar', 'Mujeeb Ur Rahman', 'Raghu Sharma', 'Richard Gleeson'], 'Gujarat Titans': ['Sai Sudharsan', 'Shubman Gill', 'Shahrukh Khan', 'Rahul Tewatia', 'Nishant Sindhu', 'Sherfane Rutherford', 'Mahipal Lomror', 'Dasun Shanaka', 'Rashid Khan', 'Ravisrinivasan Sai Kishore', 'Arshad Khan', 'Jayant Yadav', 'Karim Janat', 'Washington Sundar', 'Kumar Kushagra', 'Anuj Rawat', 'Kusal Mendis', 'Gerald Coetzee', 'Manav Suthar', 'Gurnoor Brar', 'Ishant Sharma', 'Kulwant Khejroliya', 'Prasidh Krishna', 'Mohammed Siraj'], 'Lucknow Super Giants': ['Himmat Singh', 'David Miller', 'Aiden Markram', 'Ayush Badoni', 'Mitchell Marsh', 'Abdul Samad', 'Arshin Kulkarni', 'Yuvraj Chaudhary', 'Shahbaz Ahmed', 'RS Hangargekar', 'Shardul Thakur', 'Matthew Breetzke', 'Nicholas Pooran', 'Aryan Juyal', 'Rishabh Pant', 'Ravi Bishnoi', 'Akash Deep', 'Manimaran Siddharth', 'Shamar Joseph', 'Avesh Khan', 'Prince Yadav', 'Akash Maharaj Singh', 'Digvesh Singh Rathi', 'William ORourke']}
team_names_sf = ["KKR","GT","MI","CSK","RR","RCB","PBKS","DC","SRH","LSG"]
team_names_ff = ["Kolkata Knight Riders", "Gujarat Titans", "Mumbai Indians", "Chennai Super Kings","Rajasthan Royals","Royal Challengers Bengaluru", "Punjab Kings","Delhi Capitals","Sunrisers Hyderabad","Lucknow Super Giants"]

def find_full_name(team,short_name):
    try:
        if "Varun Chak" in short_name:
            return "Varun Chakaravarthy"
        if "Reddy" in short_name and "Nitish" in short_name:
            return "Nitish Reddy"
        player = ""
        for player in team:
            if len(player)>len(short_name):
                count = 0
                parts = short_name.split(' ')
                parts_dup = []
                for part in parts:
                    word = ""
                    for letter_number in range(len(part)):
                        char = part[letter_number]
                        word += char
                        if letter_number == len(part)-1:
                            parts_dup.append(word)
                            break
                        if part[letter_number+1].isupper():
                            parts_dup.append(word)
                            word = ""
                parts = parts_dup
                for part in parts:
                    if part not in player:
                        count += 1
                if count == 0:
                    #team_copy.append(player)
                    return player
            else:
                count = 0
                parts = player.split(' ')
                parts_dup = []
                for part in parts:
                    word = ""
                    for letter_number in range(len(part)):
                        char = part[letter_number]
                        word += char
                        if letter_number == len(part)-1:
                            parts_dup.append(word)
                            break
                        if part[letter_number+1].isupper():
                            parts_dup.append(word)
                            word = ""
                parts = parts_dup
                for part in parts:
                    if part not in short_name:
                        count += 1
                if count == 0:
                    return player
                
        if player == "":
            try:
                best_match, score, _ = process.extractOne(short_name, team)
                if score > 70:
                    return best_match
                else:
                    print(short_name,"not found")
            except:
                print(short_name,"not found")
                pass
    except:
        print(short_name,"not found")
        return None


class Score:
    
    def __init__(self, match_number: int):
        self.match_number = match_number
        self.match_url, self.match_data = self.get_match_data(match_number)
        (
            self.full_player_list,
            self.player_list,
            self.winner,
            self.man_of_the_match,
            self.catchers,
            self.stumpers,
            self.main_runouters,
            self.secondary_runouters,
            self.bowled,
            self.lbw,
            self.innings_list,
            self.batsmen_list,
            self.bowlers_info
        ) = self._parse_json(self.match_data)
        
    def get_match_data(self,match_id):
        payload = {}
        headers = {
            'x-apihub-key': 'NFrLWAL6waj1iYNyxjW594CFWp0KHzlZVJIMOAaEX7t3OOdbfk',
            'x-apihub-host': 'Cricbuzz-Official-Cricket-API.allthingsdev.co',
            'x-apihub-endpoint': '5f260335-c228-4005-9eec-318200ca48d6'
        }
        try:
            match_number_string = str(match_id)
            url = "https://Cricbuzz-Official-Cricket-API.proxy-production.allthingsdev.co/match/"+match_number_string+"/scorecard"
            response = requests.request("GET", url, headers=headers, data=payload)
            match = response.json()
            match_url = match['appindex']['weburl']
            return match_url, match
        except:
            return None,None
    
    #def player_list_generator(self):

    def _parse_dismissal(self, outdec: str) -> Dict[str, Any]:
        outdec = outdec.strip()
        if not outdec or outdec == 'not out':
            return {
                'mode': 'not out',
                'catcher': '',
                'stumper': '',
                'main_ro': '',
                'secondary_ro': '',
                'bowler_bowled': '',
                'bowler_lbw': ''
            }

        result = {
            'mode': 'other',
            'catcher': '',
            'stumper': '',
            'main_ro': '',
            'secondary_ro': '',
            'bowler_bowled': '',
            'bowler_lbw': ''
        }

        # c and b X → caught by X (but NOT bowled)
        if outdec.startswith('c and b '):
            x = outdec.replace('c and b ', '').strip()
            result.update({'mode': 'caught', 'catcher': x})
            return result

        # c X b Y
        if outdec.startswith('c ') and ' b ' in outdec:
            parts = outdec.split(' b ', 1)
            fielder = parts[0].replace('c ', '').strip()
            bowler = parts[1].strip()
            result.update({'mode': 'caught', 'catcher': fielder})
            return result

        # st X b Y
        if outdec.startswith('st ') and ' b ' in outdec:
            parts = outdec.split(' b ', 1)
            fielder = parts[0].replace('st ', '').strip()
            bowler = parts[1].strip()
            result.update({'mode': 'stumped', 'stumper': fielder})
            return result

        # b X (pure bowled — must NOT contain 'lbw')
        if outdec.startswith('b ') and 'lbw' not in outdec.lower():
            bowler = outdec.replace('b ', '').strip()
            result.update({'mode': 'bowled', 'bowler_bowled': bowler})
            return result

        # lbw b X
        if 'lbw b ' in outdec.lower():
            bowler = outdec.lower().split('lbw b ')[-1].strip()
            # Fallback if case mismatch
            if not bowler:
                bowler = outdec.split(' b ')[-1].strip()
            result.update({'mode': 'lbw', 'bowler_lbw': bowler})
            return result

        # run out (X) or run out (X/Y)
        if 'run out' in outdec.lower():
            import re
            f_match = re.search(r'\(([^)]*)\)', outdec)

            main = None
            sec = None

            if f_match:
                raw = f_match.group(1).strip()

                # If raw is not empty, process it
                if raw:
                    parts = [p.strip() for p in raw.split('/') if p.strip()]

                    if len(parts) >= 2:
                        # Take only the last two
                        main, sec = parts[-2], parts[-1]
                    elif len(parts) == 1:
                        main = parts[0]
                    # If 0 parts → leave main/sec = None
                # If raw is empty → leave main/sec = None

            # Build result dict only with fields that exist
            result.update({'mode': 'run out'})
            if main is not None:
                result['main_ro'] = main
            if sec is not None:
                result['secondary_ro'] = sec

            return result


        # retd out, obstructing, etc. → ignore for dismissal lists
        return result

    def _parse_json(self, match_Dict: [str, Any]) -> Tuple[
        List[str], Dict[str, List[str]], str, str,
        List[str], List[str], List[str], List[str],
        List[str], List[str],
        List[str], pd.DataFrame, pd.DataFrame
    ]:
        # Lists — NO deduplication, preserve order & frequency
        catchers, stumpers = [], []
        main_runouters, secondary_runouters = [], []
        bowled, lbw = [], []

        innings_list = []
        player_list = {}
        batsmen_rows, bowlers_rows = [], []
        
        dots_info,man_of_the_match,player_list,full_player_list = self.match_dots_mom()

        for idx, inn in enumerate(self.match_data.get('scorecard', [])):
            inn_num = idx + 1
            bat_team = inn['batteamname']
            teams_known = list(player_list.keys())
            bowl_team = next((t for t in teams_known if t != bat_team), "")
            
            bat_players = player_list[bat_team]
            bowl_players = player_list[bowl_team]
            
            innings_list.append(bat_team)


            # --- Batsmen ---
            for b in inn.get('batsman', []):
                if b['balls'] == 0 and not b.get('outdec'):
                    continue
                name = b['name'].strip()
                print(name)
                name = find_full_name(bat_players,name)
                print(name)

                outdec = b.get('outdec', 'not out') or 'not out'
                dis = self._parse_dismissal(outdec)
                

                # Append exactly once per dismissal — no dedup
                if dis['catcher']:
                    dismisser = dis['catcher']
                    dismisser = find_full_name(bowl_players,dismisser)
                    catchers.append(dismisser)
                if dis['stumper']:
                    dismisser = dis['stumper']
                    dismisser = find_full_name(bowl_players,dismisser)
                    stumpers.append(dismisser)
                if dis['main_ro'] != '' or dis['secondary_ro'] != '':
                    dismisser = dis['main_ro']
                    dismisser = find_full_name(bowl_players,dismisser)
                    main_runouters.append(dismisser)
                    dismisser = dis['secondary_ro']
                    dismisser = find_full_name(bowl_players,dismisser)
                    secondary_runouters.append(dismisser)
                if dis['bowler_bowled']:
                    dismisser = dis['bowler_bowled']
                    dismisser = find_full_name(bowl_players,dismisser)
                    bowled.append(dismisser)
                if dis['bowler_lbw']:
                    dismisser = dis['bowler_lbw']
                    dismisser = find_full_name(bowl_players,dismisser)
                    lbw.append(dismisser)

                sr = float(b['strkrate']) if b['strkrate'] and b['strkrate'] != '0' else 0.0
                batsmen_rows.append({
                    'Innings Number': inn_num,
                    'Innings Name': bat_team,
                    'Batsman': name,
                    'Dismissal': outdec,
                    'Runs': int(b['runs']),
                    'Balls': int(b['balls']),
                    '4s': int(b['fours']),
                    '6s': int(b['sixes']),
                    'Strike Rate': sr
                })

            # --- Bowlers ---
            for blr in inn.get('bowler', []):
                name = blr['name'].strip()
                print(name)
                name = find_full_name(bowl_players,name)
                print(name)

                # Overs: preserve as X.Y (e.g., '2.2' → 2.2, not 2.333)
                o_str = blr['overs']
                try:
                    if '.' in o_str:
                        whole, frac = o_str.split('.')
                        # Clamp fractional part to 1 digit (0–5)
                        frac = frac[:1]
                        # Ensure valid ball count (0–5)
                        if frac.isdigit() and 0 <= int(frac) <= 5:
                            overs_val = float(f"{whole}.{frac}")
                        else:
                            overs_val = float(whole)
                    else:
                        overs_val = float(o_str)
                except:
                    overs_val = 0.0
                
                bowlers_rows.append({
                    'Innings Number': inn_num,
                    'Innings Name': bat_team,
                    'Bowler': name,
                    'Overs': overs_val,
                    'Maidens': int(blr['maidens']),
                    'Runs': int(blr['runs']),
                    'Wickets': int(blr['wickets']),
                    'Economy': float(blr['economy']),
                    '0s': dots_info.get(name, {}).get('dots', 0)
                })


        batsmen_df = pd.DataFrame(batsmen_rows) if batsmen_rows else pd.DataFrame(columns=[
            'Innings Number', 'Innings Name', 'Batsman', 'Dismissal',
            'Runs', 'Balls', '4s', '6s', 'Strike Rate'
        ])
        bowlers_df = pd.DataFrame(bowlers_rows) if bowlers_rows else pd.DataFrame(columns=[
            'Innings Number', 'Innings Name', 'Bowler',
            'Overs', 'Maidens', 'Runs', 'Wickets', 'Economy','0s'
        ])

        # Winner
        status = self.match_data.get('status', '')
        winner = ""
        if 'won by' in status:
            winner = status.split(' won by ')[0].strip()
            for team in innings_list:
                if winner in team or team in winner:
                    winner = team
                    break
        elif ' won the' in status:
            winner = status.split(' won the')[0].split('(')[1].strip()
        

        return (
            full_player_list,
            player_list,
            winner,
            man_of_the_match,
            catchers,
            stumpers,
            main_runouters,
            secondary_runouters,
            bowled,
            lbw,
            innings_list,
            batsmen_df,
            bowlers_df
        )

    def printing_scorecard(self):
        print("Player List:")
        print(self.player_list)
        print()
        for innings in self.innings_list:
            print("-" * 140)
            print(f"{innings}:")
            print()

            print("Batsmen:")
            batsmen_inn = self.batsmen_list[self.batsmen_list['Innings Name'] == innings]
            if not batsmen_inn.empty:
                df = batsmen_inn.drop(columns=['Innings Number', 'Innings Name']).copy()
                print(df.to_string(index=False))
            else:
                print("(No data)")
            print()

            print("Bowlers:")
            bowlers_inn = self.bowlers_info[self.bowlers_info['Innings Name'] == innings]
            if not bowlers_inn.empty:
                df = bowlers_inn.drop(columns=['Innings Number', 'Innings Name']).copy()
                # Format Overs to show exactly one decimal place (e.g., 2.2, 4.0)
                df['Overs'] = df['Overs'].apply(lambda x: f"{x:.1f}")
                print(df.to_string(index=False))
            else:
                print("(No data)")
            print()

        print("Catchers:")
        print(self.catchers)
        print()

        print("Stumpings:")
        print(self.stumpers)
        print()

        print("Main Run Outs:")
        print(self.main_runouters)
        print()

        print("Secondary Run Outs:")
        print(self.secondary_runouters)
        print()

        print("Bowled:")
        print(self.bowled)
        print()

        print("LBW:")
        print(self.lbw)
        print()
        print("-" * 140)
        print("Winner: ", self.winner)
        print()
        print("Man of the Match: ", self.man_of_the_match)
        
    def match_dots_mom(self):
        def calculate_dots(over_summary):
            over = over_summary.split(' ')
            dots = 0
            for ball in over:
                if ball == '0' or ball == 'W' or 'L' in ball or 'B' in ball:
                    dots += 1
            return dots

        payload = {}
        headers = {
            'x-apihub-key': 'NFrLWAL6waj1iYNyxjW594CFWp0KHzlZVJIMOAaEX7t3OOdbfk',
            'x-apihub-host': 'Cricbuzz-Official-Cricket-API.allthingsdev.co',
            'x-apihub-endpoint': '5db6b2f0-86b9-44b5-bfc4-8c2888acd4de'
        }

        match_number_string = str(self.match_number)
        url = "https://Cricbuzz-Official-Cricket-API.proxy-production.allthingsdev.co/match/"+match_number_string+"/overs"
        response = requests.request("GET", url, headers=headers, data=payload)
        overs_score = response.json()  

        bowlers_info = {}
        for over in overs_score['overseplist']['oversep']:
            bowler_names = over['ovrbowlnames']
            over_summary = over['oversummary']
            if len(bowler_names) == 1:
                bowler = bowler_names[0]
                if bowler not in bowlers_info.keys():
                    bowlers_info[bowler] = {'dots': 0}
                    dots = calculate_dots(over_summary)
                    bowlers_info[bowler]['dots'] += dots
                else:
                    dots = calculate_dots(over_summary)
                    bowlers_info[bowler]['dots'] += dots
            else:
                for bowler in bowler_names:
                    x=2 #fill this later
        
        man_of_the_match = overs_score["matchheaders"]["momplayers"]["player"][0]["name"]
        
        team1 = overs_score["matchheaders"]["team1"]["teamname"]
        team2 = overs_score["matchheaders"]["team2"]["teamname"]
        
        player_list = {}
        player_list[team1] = squads[team1]
        player_list[team2] = squads[team2]
        
        full_player_list = squads[team1]
        full_player_list.extend(squads[team2])

        return bowlers_info,man_of_the_match,player_list,full_player_list
    
    import requests
import dill

class Series:
    def __init__(
        self,
        series_number: int = 9237,
        database_name: str = "IPL2025.pkl",
    ):
        """
        End behaviour matches your old Series:
        - self.match_objects: dict[match_id] -> Score instance
        - self.match_links: list of match_ids in this series
        - loads/stores objects via dill in database_name
        """
        self.series_number = series_number
        self.database_name = database_name

        self.base_url = (
            "https://Cricbuzz-Official-Cricket-API.proxy-production.allthingsdev.co/series/"
        )

        self.match_objects = {}
        self.match_links = []

        # Get all match IDs for this series
        match_links = self.match_id_generator()

        # Try loading existing DB
        try:
            with open(self.database_name, "rb") as file:
                stored = dill.load(file)
        except Exception:
            stored = {}

        match_objects = stored
        match_links_list = list(stored.keys())

        # Mirror your original logic:
        if len(match_links) >= len(match_links_list):
            if len(match_links) != 0:
                for match in match_links:
                    # Same condition: new match or last match (to refresh)
                    if match not in match_links_list or match == match_links[-1]:
                        print("Attempting to scrape:", match)
                        # Score now takes match_id instead of URL
                        match_object = Score(match)
                        print("Scraping Successful")
                        match_object.printing_scorecard()
                        match_objects[match] = match_object
                        print("Added:", match)

                # After loop, same success/fail logic
                if len(list(match_objects.keys())) == len(match_links):
                    self.match_links = match_links
                    self.match_objects = match_objects
                    with open(self.database_name, "wb") as file:
                        dill.dump(match_objects, file)
                    print("LOADING SUCCESSFUL")
                else:
                    print("LOADING FAILED")
                    print("No. of match objects", len(match_objects))
                    print("Number of extracted links", len(match_links))
                    self.match_objects = match_objects
                    self.match_links = match_links
                    print("Missing Links:")
                    for match_id in match_links:
                        if match_id not in list(match_objects.keys()):
                            print(match_id)
                    if len(match_objects) > len(stored):
                        with open(self.database_name, "wb") as file:
                            dill.dump(match_objects, file)
        else:
            print("DATA UP TO DATE")
            self.match_objects = match_objects
            self.match_links = match_links

    def match_id_generator(self):
        url = f"{self.base_url}{self.series_number}"
        headers = {
            "x-apihub-key": "NFrLWAL6waj1iYNyxjW594CFWp0KHzlZVJIMOAaEX7t3OOdbfk",
            "x-apihub-host": "Cricbuzz-Official-Cricket-API.allthingsdev.co",
            "x-apihub-endpoint": "661c6b89-b558-41fa-9553-d0aca64fcb6f",
        }

        try:
            resp = requests.get(url, headers=headers, timeout=15)
            resp.raise_for_status()
            series_data = resp.json()
        except Exception as e:
            print("Error fetching series data:", e)
            return []

        match_ids = []
        for block in series_data.get("matchDetails", []):
            mmap = block.get("matchDetailsMap", {})
            match_list = mmap.get("match", [])
            for m in match_list:
                info = m.get("matchInfo", {})
                match_id = info.get("matchId")
                if match_id:
                    match_ids.append(match_id)

        return match_ids


In [11]:
# Ensure you have:
# def get_match_scorecard(mid): ...

# Then:
score = Score(115192)
score.printing_scorecard()

Player List:
{'Punjab Kings': ['Nehal Wadhera', 'Harnoor Singh', 'Shreyas Iyer', 'Pyla Avinash', 'Priyansh Arya', 'Musheer Khan', 'Marcus Stoinis', 'Aaron Hardie', 'Suryansh Shedge', 'Shashank Singh', 'Mitchell Owen', 'Praveen Dubey', 'Azmatullah Omarzai', 'Prabhsimran Singh', 'Josh Inglis', 'Vishnu Vinod', 'Harpreet Brar', 'Arshdeep Singh', 'Yuzvendra Chahal', 'Vijaykumar Vyshak', 'Kuldeep Sen', 'Yash Thakur', 'Xavier Bartlett', 'Kyle Jamieson', 'Virat Kohli', 'Rajat Patidar', 'Swastik Chikara', 'Tim David', 'Mayank Agarwal', 'Krunal Pandya', 'Liam Livingstone', 'Manoj Bhandage', 'Romario Shepherd', 'Swapnil Singh', 'Mohit Rathee', 'Philip Salt', 'Jitesh Sharma', 'Tim Seifert', 'Josh Hazlewood', 'Bhuvneshwar Kumar', 'Rasikh Dar Salam', 'Suyash Sharma', 'Yash Dayal', 'Nuwan Thushara', 'Abhinandan Singh', 'Blessing Muzarabani'], 'Royal Challengers Bengaluru': ['Virat Kohli', 'Rajat Patidar', 'Swastik Chikara', 'Tim David', 'Mayank Agarwal', 'Krunal Pandya', 'Liam Livingstone', 'Manoj Bh

In [ ]:
# import requests
class Series:
   series_number = 9237
   url = "https://Cricbuzz-Official-Cricket-API.proxy-production.allthingsdev.co/series/"+str(series_number)

   payload = {}
   headers = {
      'x-apihub-key': 'NFrLWAL6waj1iYNyxjW594CFWp0KHzlZVJIMOAaEX7t3OOdbfk',
      'x-apihub-host': 'Cricbuzz-Official-Cricket-API.allthingsdev.co',
      'x-apihub-endpoint': '661c6b89-b558-41fa-9553-d0aca64fcb6f'
   }

   response = requests.request("GET", url, headers=headers, data=payload)

   series_data = response.json()
   for match in series_data['matchDetails']:
      matchinf = match['matchDetailsMap']['match'][0]['matchInfo']
      matchid = matchinf['matchId']
      status = matchinf['state']
      print(matchid)
      print(status)
      score = Score(matchid)
      score.printing_scorecard()
      print()

114960
complete
Player List:
{'Kolkata Knight Riders': ['Spencer Johnson', 'Ajinkya Rahane', 'Hazlewood', 'Krunal Pandya', 'Livingstone', 'Angkrish Raghuvanshi', 'Rinku Singh', 'Yash Dayal', 'Russell', 'de Kock', 'Narine', 'Ramandeep Singh', 'Rasikh Salam', 'Suyash Sharma', 'Venkatesh Iyer', 'Harshit Rana'], 'Royal Challengers Bengaluru': ['Sunil Narine', 'Spencer Johnson', 'Rajat Patidar', 'Devdutt Padikkal', 'Harshit Rana', 'Liam Livingstone', 'Varun Chakaravarthy', 'Virat Kohli', 'Vaibhav Arora', 'Philip Salt']}

--------------------------------------------------------------------------------------------------------------------------------------------
Kolkata Knight Riders:

Batsmen:
             Batsman                      Dismissal  Runs  Balls  4s  6s  Strike Rate
             de Kock    c Jitesh Sharma b Hazlewood     4      5   1   0        80.00
              Narine c Jitesh Sharma b Rasikh Salam    44     26   5   3       169.23
      Ajinkya Rahane c Rasikh Salam b Krunal P

AttributeError: 'NoneType' object has no attribute 'get'

In [11]:
import requests
import dill

class Series:
    def __init__(
        self,
        series_number: int = 9237,
        database_name: str = "IPL2025.pkl",
    ):
        """
        End behaviour matches your old Series:
        - self.match_objects: dict[match_id] -> Score instance
        - self.match_links: list of match_ids in this series
        - loads/stores objects via dill in database_name
        """
        self.series_number = series_number
        self.database_name = database_name

        self.base_url = (
            "https://Cricbuzz-Official-Cricket-API.proxy-production.allthingsdev.co/series/"
        )

        self.match_objects = {}
        self.match_links = []

        # Get all match IDs for this series
        match_links = self.match_id_generator()

        # Try loading existing DB
        try:
            with open(self.database_name, "rb") as file:
                stored = dill.load(file)
        except Exception:
            stored = {}

        match_objects = stored
        match_links_list = list(stored.keys())

        # Mirror your original logic:
        if len(match_links) >= len(match_links_list):
            if len(match_links) != 0:
                for match in match_links:
                    # Same condition: new match or last match (to refresh)
                    if match not in match_links_list or match == match_links[-1]:
                        print("Attempting to scrape:", match)
                        # Score now takes match_id instead of URL
                        match_object = Score(match)
                        print("Scraping Successful")
                        match_object.printing_scorecard()
                        match_objects[match] = match_object
                        print("Added:", match)

                # After loop, same success/fail logic
                if len(list(match_objects.keys())) == len(match_links):
                    self.match_links = match_links
                    self.match_objects = match_objects
                    with open(self.database_name, "wb") as file:
                        dill.dump(match_objects, file)
                    print("LOADING SUCCESSFUL")
                else:
                    print("LOADING FAILED")
                    print("No. of match objects", len(match_objects))
                    print("Number of extracted links", len(match_links))
                    self.match_objects = match_objects
                    self.match_links = match_links
                    print("Missing Links:")
                    for match_id in match_links:
                        if match_id not in list(match_objects.keys()):
                            print(match_id)
                    if len(match_objects) > len(stored):
                        with open(self.database_name, "wb") as file:
                            dill.dump(match_objects, file)
        else:
            print("DATA UP TO DATE")
            self.match_objects = match_objects
            self.match_links = match_links

    def match_id_generator(self):
        url = f"{self.base_url}{self.series_number}"
        headers = {
            "x-apihub-key": "NFrLWAL6waj1iYNyxjW594CFWp0KHzlZVJIMOAaEX7t3OOdbfk",
            "x-apihub-host": "Cricbuzz-Official-Cricket-API.allthingsdev.co",
            "x-apihub-endpoint": "661c6b89-b558-41fa-9553-d0aca64fcb6f",
        }

        try:
            resp = requests.get(url, headers=headers, timeout=15)
            resp.raise_for_status()
            series_data = resp.json()
        except Exception as e:
            print("Error fetching series data:", e)
            return []

        match_ids = []
        for block in series_data.get("matchDetails", []):
            mmap = block.get("matchDetailsMap", {})
            match_list = mmap.get("match", [])
            for m in match_list:
                info = m.get("matchInfo", {})
                match_id = info.get("matchId")
                if match_id:
                    match_ids.append(match_id)

        return match_ids


In [12]:
ipl25 = Series()

Attempting to scrape: 114960
Scraping Successful
Player List:
{'Kolkata Knight Riders': ['Rasikh Salam', 'Krunal Pandya', 'Ajinkya Rahane', 'Rinku Singh', 'Russell', 'Suyash Sharma', 'Venkatesh Iyer', 'Hazlewood', 'Angkrish Raghuvanshi', 'Narine', 'de Kock', 'Harshit Rana', 'Yash Dayal', 'Livingstone', 'Ramandeep Singh', 'Spencer Johnson'], 'Royal Challengers Bengaluru': ['Rajat Patidar', 'Devdutt Padikkal', 'Philip Salt', 'Virat Kohli', 'Liam Livingstone', 'Varun Chakaravarthy', 'Sunil Narine', 'Harshit Rana', 'Vaibhav Arora', 'Spencer Johnson']}

--------------------------------------------------------------------------------------------------------------------------------------------
Kolkata Knight Riders:

Batsmen:
             Batsman                      Dismissal  Runs  Balls  4s  6s  Strike Rate
             de Kock    c Jitesh Sharma b Hazlewood     4      5   1   0        80.00
              Narine c Jitesh Sharma b Rasikh Salam    44     26   5   3       169.23
      Ajinkya

In [ ]:
for key in match_data.keys():
  print(key,match_data[key])

print()

for innings in match_data['scorecard']:
  for key in innings.keys():
    print(key,innings[key])
    if key in ['batsman','bowler']:
      for j in innings[key]:
        print(j)
  print()

scorecard [{'inningsid': 1, 'batsman': [{'id': 8520, 'balls': 5, 'runs': 4, 'fours': 1, 'sixes': 0, 'strkrate': '80', 'name': 'de Kock', 'nickname': '', 'iscaptain': False, 'iskeeper': True, 'outdec': 'c Jitesh Sharma b Hazlewood', 'videotype': '', 'videourl': '', 'videoid': 0, 'planid': 0, 'imageid': 0, 'premiumvideourl': '', 'iscbplusfree': False, 'ispremiumfree': False, 'inmatchchange': '', 'isoverseas': True, 'playingxichange': ''}, {'id': 2276, 'balls': 26, 'runs': 44, 'fours': 5, 'sixes': 3, 'strkrate': '169.23', 'name': 'Narine', 'nickname': '', 'iscaptain': False, 'iskeeper': False, 'outdec': 'c Jitesh Sharma b Rasikh Salam', 'videotype': '', 'videourl': '', 'videoid': 0, 'planid': 0, 'imageid': 0, 'premiumvideourl': '', 'iscbplusfree': False, 'ispremiumfree': False, 'inmatchchange': '', 'isoverseas': True, 'playingxichange': ''}, {'id': 1447, 'balls': 31, 'runs': 56, 'fours': 6, 'sixes': 4, 'strkrate': '180.65', 'name': 'Ajinkya Rahane', 'nickname': '', 'iscaptain': True, 'isk

In [37]:
class Squads:
    
    def __init__(self,series_id: int = 9237):
        self.series_id = series_id
        self.series_url = "https://Cricbuzz-Official-Cricket-API.proxy-production.allthingsdev.co/series/"+str(self.series_id)+"/squads"
        self.names,self.roles,self.squads = self.squad_generator()
        
    def squad_generator(self):
        payload = {}
        headers = {
        'x-apihub-key': 'NFrLWAL6waj1iYNyxjW594CFWp0KHzlZVJIMOAaEX7t3OOdbfk',
        'x-apihub-host': 'Cricbuzz-Official-Cricket-API.allthingsdev.co',
        'x-apihub-endpoint': '038d223b-aca5-4096-8eb1-184dd0c09513'
        }

        response = requests.request("GET", self.series_url, headers=headers, data=payload)
        squad_names = {}
        squad_data = response.json()
        squad_id_data = squad_data["squads"]
        
        for squad in squad_id_data:
            if "squadId" in squad.keys():
                team = squad["squadType"]
                squad_id = squad["squadId"]
                squad_names[squad_id] = team
        
        squads = {}
        names = []
        roles = []
        
        for squad_id in squad_names.keys():
            squad_url = self.series_url+ "/" + str(squad_id)
            team = squad_names[squad_id]
            squads[team] = []

            payload = {}
            headers = {
            'x-apihub-key': 'NFrLWAL6waj1iYNyxjW594CFWp0KHzlZVJIMOAaEX7t3OOdbfk',
            'x-apihub-host': 'Cricbuzz-Official-Cricket-API.allthingsdev.co',
            'x-apihub-endpoint': 'c4b3ccd2-0bb1-4d94-98c9-b31f389480be'
            }

            response = requests.request("GET", squad_url, headers=headers, data=payload)

            current_squad_data = response.json()
            
            for player in current_squad_data["player"]:
                if 'id' in player.keys():
                    name = player["name"]
                    role = player["role"]
                    if role == "Batsman":
                        role = "BAT"
                    elif "Allrounder" in role:
                        role = "AR"
                    elif role == "WK-Batsman":
                        role = "WK"
                    elif role == "Bowler":
                        role = "BOWL"
                    names.append(name)
                    roles.append(role)
                    squads[team].append(name)
        
        return names,roles,squads
    
ipl25_squads = Squads()
names = ipl25_squads.names
roles = ipl25_squads.roles
player_list = ipl25_squads.squads
print(names)
print(roles)
print(player_list)


['Ruturaj Gaikwad', 'Andre Siddarth C', 'Shaik Rasheed', 'Rahul Tripathi', 'Ayush Mhatre', 'Dewald Brevis', 'Shivam Dube', 'Rachin Ravindra', 'Deepak Hooda', 'Vijay Shankar', 'Ramakrishna Ghosh', 'Ravindra Jadeja', 'Anshul Kamboj', 'Jamie Overton', 'Sam Curran', 'Ravichandran Ashwin', 'Devon Conway', 'MS Dhoni', 'Vansh Bedi', 'Urvil Patel', 'Kamlesh Nagarkoti', 'Shreyas Gopal', 'Matheesha Pathirana', 'Mukesh Choudhary', 'Nathan Ellis', 'Noor Ahmad', 'Khaleel Ahmed', 'Vaibhav Suryavanshi', 'Shimron Hetmyer', 'Yashasvi Jaiswal', 'Shubham Dubey', 'Riyan Parag', 'Wanindu Hasaranga', 'Sanju Samson', 'Dhruv Jurel', 'Kunal Singh Rathore', 'Lhuan-dre Pretorius', 'Yudhvir Singh Charak', 'Tushar Deshpande', 'Kumar Kartikeya', 'Akash Madhwal', 'Kwena Maphaka', 'Maheesh Theekshana', 'Fazalhaq Farooqi', 'Ashok Sharma', 'Jofra Archer', 'Nandre Burger', 'Manish Pandey', 'Ajinkya Rahane', 'Rinku Singh', 'Angkrish Raghuvanshi', 'Anukul Roy', 'Ramandeep Singh', 'Venkatesh Iyer', 'Moeen Ali', 'Sunil Nari

In [31]:
d = {
  "player": [
    {
      "name": "BATTERS",
      "isHeader": True
    },
    {
      "id": "11813",
      "name": "Ruturaj Gaikwad",
      "role": "Batsman",
      "imageId": 781069,
      "battingStyle": "Right-hand bat",
      "bowlingStyle": "Right-arm offbreak"
    },
    {
      "id": "1427488",
      "name": "Andre Siddarth C",
      "role": "Batsman",
      "imageId": 174146,
      "battingStyle": "Right-hand bat"
    },
    {
      "id": "22571",
      "name": "Shaik Rasheed",
      "role": "Batsman",
      "imageId": 594287,
      "battingStyle": "Right-hand bat",
      "bowlingStyle": "Right-arm legbreak"
    },
    {
      "id": "9012",
      "name": "Rahul Tripathi",
      "role": "Batsman",
      "imageId": 788087,
      "battingStyle": "Right-hand bat",
      "bowlingStyle": "Right-arm medium"
    },
    {
      "id": "1431163",
      "name": "Ayush Mhatre",
      "role": "Batsman",
      "imageId": 174146,
      "battingStyle": "Right-hand bat",
      "bowlingStyle": "Right-arm offbreak"
    },
    {
      "id": "20538",
      "name": "Dewald Brevis",
      "role": "Batsman",
      "imageId": 594284,
      "battingStyle": "Right-hand bat",
      "bowlingStyle": "Right-arm legbreak"
    },
    {
      "name": "ALL ROUNDERS",
      "isHeader": True
    },
    {
      "id": "11195",
      "name": "Shivam Dube",
      "role": "Batting Allrounder",
      "imageId": 593805,
      "battingStyle": "Left-hand bat",
      "bowlingStyle": "Right-arm medium"
    },
    {
      "id": "11177",
      "name": "Rachin Ravindra",
      "role": "Batting Allrounder",
      "imageId": 655385,
      "battingStyle": "Left-hand bat",
      "bowlingStyle": "Left-arm orthodox"
    },
    {
      "id": "9427",
      "name": "Deepak Hooda",
      "role": "Batting Allrounder",
      "imageId": 244970,
      "battingStyle": "Right-hand bat",
      "bowlingStyle": "Right-arm offbreak"
    },
    {
      "id": "8204",
      "name": "Vijay Shankar",
      "role": "Batting Allrounder",
      "imageId": 170664,
      "battingStyle": "Right-hand bat",
      "bowlingStyle": "Right-arm medium"
    },
    {
      "id": "32835",
      "name": "Ramakrishna Ghosh",
      "role": "Batting Allrounder",
      "imageId": 781087,
      "battingStyle": "Right-hand bat",
      "bowlingStyle": "Right-arm fast-medium"
    },
    {
      "id": "587",
      "name": "Ravindra Jadeja",
      "role": "Bowling Allrounder",
      "imageId": 616520,
      "battingStyle": "Left-hand bat",
      "bowlingStyle": "Left-arm orthodox"
    },
    {
      "id": "14598",
      "name": "Anshul Kamboj",
      "role": "Bowling Allrounder",
      "imageId": 731465,
      "battingStyle": "Right-hand bat",
      "bowlingStyle": "Right-arm fast-medium"
    },
    {
      "id": "8512",
      "name": "Jamie Overton",
      "role": "Bowling Allrounder",
      "imageId": 717798,
      "battingStyle": "Right-hand bat",
      "bowlingStyle": "Right-arm fast"
    },
    {
      "id": "10420",
      "name": "Sam Curran",
      "role": "Bowling Allrounder",
      "imageId": 655387,
      "battingStyle": "Left-hand bat",
      "bowlingStyle": "Left-arm fast-medium"
    },
    {
      "id": "1593",
      "name": "Ravichandran Ashwin",
      "role": "Bowling Allrounder",
      "imageId": 591961,
      "battingStyle": "Right-hand bat",
      "bowlingStyle": "Right-arm offbreak"
    },
    {
      "name": "WICKET KEEPERS",
      "isHeader": True
    },
    {
      "id": "9838",
      "name": "Devon Conway",
      "role": "WK-Batsman",
      "imageId": 616434,
      "battingStyle": "Left-hand bat",
      "bowlingStyle": "Right-arm medium"
    },
    {
      "id": "265",
      "name": "MS Dhoni",
      "captain": True,
      "role": "WK-Batsman",
      "imageId": 170677,
      "battingStyle": "Right-hand bat",
      "bowlingStyle": "Right-arm medium"
    },
    {
      "id": "1429626",
      "name": "Vansh Bedi",
      "role": "WK-Batsman",
      "imageId": 174146,
      "battingStyle": "Right-hand bat"
    },
    {
      "id": "13476",
      "name": "Urvil Patel",
      "role": "WK-Batsman",
      "imageId": 174146,
      "battingStyle": "Right-hand bat"
    },
    {
      "name": "BOWLERS",
      "isHeader": True
    },
    {
      "id": "12092",
      "name": "Kamlesh Nagarkoti",
      "role": "Bowler",
      "imageId": 226313,
      "battingStyle": "Right-hand bat",
      "bowlingStyle": "Right-arm fast"
    },
    {
      "id": "9746",
      "name": "Shreyas Gopal",
      "role": "Bowler",
      "imageId": 226505,
      "battingStyle": "Right-hand bat",
      "bowlingStyle": "Right-arm legbreak"
    },
    {
      "id": "16458",
      "name": "Matheesha Pathirana",
      "role": "Bowler",
      "imageId": 351888,
      "battingStyle": "Right-hand bat",
      "bowlingStyle": "Right-arm fast"
    },
    {
      "id": "13184",
      "name": "Mukesh Choudhary",
      "role": "Bowler",
      "imageId": 781085,
      "battingStyle": "Left-hand bat",
      "bowlingStyle": "Left-arm fast-medium"
    },
    {
      "id": "15480",
      "name": "Nathan Ellis",
      "role": "Bowler",
      "imageId": 619885,
      "battingStyle": "Right-hand bat",
      "bowlingStyle": "Right-arm fast-medium"
    },
    {
      "id": "15452",
      "name": "Noor Ahmad",
      "role": "Bowler",
      "imageId": 616569,
      "battingStyle": "Right-hand bat",
      "bowlingStyle": "Left-arm wrist-spin"
    },
    {
      "id": "10952",
      "name": "Khaleel Ahmed",
      "role": "Bowler",
      "imageId": 655386,
      "battingStyle": "Right-hand bat",
      "bowlingStyle": "Left-arm fast-medium"
    }
  ]
}
for i in d["player"]:
  print(i[])

{'name': 'BATTERS', 'isHeader': True}
{'id': '11813', 'name': 'Ruturaj Gaikwad', 'role': 'Batsman', 'imageId': 781069, 'battingStyle': 'Right-hand bat', 'bowlingStyle': 'Right-arm offbreak'}
{'id': '1427488', 'name': 'Andre Siddarth C', 'role': 'Batsman', 'imageId': 174146, 'battingStyle': 'Right-hand bat'}
{'id': '22571', 'name': 'Shaik Rasheed', 'role': 'Batsman', 'imageId': 594287, 'battingStyle': 'Right-hand bat', 'bowlingStyle': 'Right-arm legbreak'}
{'id': '9012', 'name': 'Rahul Tripathi', 'role': 'Batsman', 'imageId': 788087, 'battingStyle': 'Right-hand bat', 'bowlingStyle': 'Right-arm medium'}
{'id': '1431163', 'name': 'Ayush Mhatre', 'role': 'Batsman', 'imageId': 174146, 'battingStyle': 'Right-hand bat', 'bowlingStyle': 'Right-arm offbreak'}
{'id': '20538', 'name': 'Dewald Brevis', 'role': 'Batsman', 'imageId': 594284, 'battingStyle': 'Right-hand bat', 'bowlingStyle': 'Right-arm legbreak'}
{'name': 'ALL ROUNDERS', 'isHeader': True}
{'id': '11195', 'name': 'Shivam Dube', 'role'

In [ ]:
for match in matches:
  print(match)
  print()

NameError: name 'matches' is not defined

### Jharkhand vs Saurashtra (two bowlers in an over)

In [32]:
commentary = {
  "inningsid": 0,
  "comwrapper": [
    {
      "commentary": {
        "commtxt": "B0$ Jharkhand won by 36 runs",
        "timestamp": 1765183907523,
        "overnum": 0,
        "inningsid": 2,
        "eventtype": "NONE",
        "commentaryformats": [
          {
            "type": "",
            "value": []
          },
          {
            "type": "bold",
            "value": [
              {
                "id": "B0$",
                "value": "Result:"
              }
            ]
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          }
        ],
        "videotype": "",
        "videourl": "",
        "videoid": 0,
        "ballnbr": 0,
        "planid": 0,
        "imageid": 0,
        "premiumvideourl": "",
        "ispremiumfree": False,
        "batteamscore": 0,
        "boundarytracker": False
      }
    },
    {
      "commentary": {
        "commtxt": "Sushant Mishra to Sahil Dhiwan, B0$ Caught by Kushagra!! B1$",
        "timestamp": 1765183866365,
        "overnum": 19.2,
        "inningsid": 2,
        "eventtype": "WICKET",
        "commentaryformats": [
          {
            "type": "",
            "value": []
          },
          {
            "type": "bold",
            "value": [
              {
                "id": "B0$",
                "value": "out"
              },
              {
                "id": "B1$",
                "value": "Sahil Dhiwan c Kushagra b Sushant Mishra 0(1)"
              }
            ]
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          }
        ],
        "videotype": "",
        "videourl": "",
        "videoid": 0,
        "ballnbr": 116,
        "planid": 0,
        "imageid": 0,
        "premiumvideourl": "",
        "ispremiumfree": False,
        "batteamscore": 0,
        "boundarytracker": False
      }
    },
    {
      "commentary": {
        "commtxt": "Sushant Mishra to Sahil Dhiwan, B0$ Caught!!",
        "timestamp": 1765183864113,
        "overnum": 0,
        "inningsid": 2,
        "eventtype": "NONE",
        "commentaryformats": [
          {
            "type": "",
            "value": []
          },
          {
            "type": "bold",
            "value": [
              {
                "id": "B0$",
                "value": "THATS OUT!!"
              }
            ]
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          }
        ],
        "videotype": "",
        "videourl": "",
        "videoid": 0,
        "ballnbr": 0,
        "planid": 0,
        "imageid": 0,
        "premiumvideourl": "",
        "ispremiumfree": False,
        "batteamscore": 0,
        "boundarytracker": False
      }
    },
    {
      "commentary": {
        "commtxt": "B0$",
        "timestamp": 1765183849133,
        "overnum": 0,
        "inningsid": 2,
        "eventtype": "NONE",
        "commentaryformats": [
          {
            "type": "",
            "value": []
          },
          {
            "type": "bold",
            "value": [
              {
                "id": "B0$",
                "value": "Sahil Dhiwan, right handed bat, comes to the crease"
              }
            ]
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          }
        ],
        "videotype": "",
        "videourl": "",
        "videoid": 0,
        "ballnbr": 0,
        "planid": 0,
        "imageid": 0,
        "premiumvideourl": "",
        "ispremiumfree": False,
        "batteamscore": 0,
        "boundarytracker": False
      }
    },
    {
      "commentary": {
        "commtxt": "Sushant Mishra to K Nagarkoti, B0$ Caught by Rajandeep Singh!! B1$",
        "timestamp": 1765183847054,
        "overnum": 19.1,
        "inningsid": 2,
        "eventtype": "WICKET",
        "commentaryformats": [
          {
            "type": "",
            "value": []
          },
          {
            "type": "bold",
            "value": [
              {
                "id": "B0$",
                "value": "out"
              },
              {
                "id": "B1$",
                "value": "K Nagarkoti c Rajandeep Singh b Sushant Mishra 3(6)"
              }
            ]
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          }
        ],
        "videotype": "",
        "videourl": "",
        "videoid": 0,
        "ballnbr": 115,
        "planid": 0,
        "imageid": 0,
        "premiumvideourl": "",
        "ispremiumfree": False,
        "batteamscore": 0,
        "boundarytracker": False
      }
    },
    {
      "commentary": {
        "commtxt": "Sushant Mishra to K Nagarkoti, B0$ Caught!!",
        "timestamp": 1765183845977,
        "overnum": 0,
        "inningsid": 2,
        "eventtype": "NONE",
        "commentaryformats": [
          {
            "type": "",
            "value": []
          },
          {
            "type": "bold",
            "value": [
              {
                "id": "B0$",
                "value": "THATS OUT!!"
              }
            ]
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          }
        ],
        "videotype": "",
        "videourl": "",
        "videoid": 0,
        "ballnbr": 0,
        "planid": 0,
        "imageid": 0,
        "premiumvideourl": "",
        "ispremiumfree": False,
        "batteamscore": 0,
        "boundarytracker": False
      }
    },
    {
      "commentary": {
        "commtxt": "Bal Krishna to K Nagarkoti, 1 run",
        "timestamp": 1765183820795,
        "overnum": 18.6,
        "inningsid": 2,
        "eventtype": "over-break",
        "oversep": {
          "score": 179,
          "wickets": 8,
          "inningsid": 2,
          "oversummary": "2 0 Wd 0 1 0 1 ",
          "runs": 5,
          "batstrikerdetails": "7(9)",
          "batnonstrikerdetails": "3(5)",
          "bowldetails": "3-0-27-0",
          "timestamp": 1765183820795,
          "overnum": 18.6,
          "ovrbatnames": [],
          "ovrbowlnames": [],
          "batstrikername": "Manav Suthar",
          "batnonstrikername": "Kamlesh Nagarkoti",
          "bowlname": "Bal Krishna",
          "battingteamname": "RAJ"
        },
        "commentaryformats": [
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          }
        ],
        "videotype": "",
        "videourl": "",
        "videoid": 0,
        "ballnbr": 114,
        "planid": 0,
        "imageid": 0,
        "premiumvideourl": "",
        "ispremiumfree": False,
        "batteamscore": 0,
        "boundarytracker": False
      }
    },
    {
      "commentary": {
        "commtxt": "Bal Krishna to K Nagarkoti, no run",
        "timestamp": 1765183708303,
        "overnum": 18.5,
        "inningsid": 2,
        "eventtype": "NONE",
        "commentaryformats": [
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          }
        ],
        "videotype": "",
        "videourl": "",
        "videoid": 0,
        "ballnbr": 113,
        "planid": 0,
        "imageid": 0,
        "premiumvideourl": "",
        "ispremiumfree": False,
        "batteamscore": 0,
        "boundarytracker": False
      }
    },
    {
      "commentary": {
        "commtxt": "Bal Krishna to Manav Suthar, 1 run",
        "timestamp": 1765183672024,
        "overnum": 18.4,
        "inningsid": 2,
        "eventtype": "NONE",
        "commentaryformats": [
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          }
        ],
        "videotype": "",
        "videourl": "",
        "videoid": 0,
        "ballnbr": 112,
        "planid": 0,
        "imageid": 0,
        "premiumvideourl": "",
        "ispremiumfree": False,
        "batteamscore": 0,
        "boundarytracker": False
      }
    },
    {
      "commentary": {
        "commtxt": "Bal Krishna to Manav Suthar, no run",
        "timestamp": 1765183640536,
        "overnum": 18.3,
        "inningsid": 2,
        "eventtype": "NONE",
        "commentaryformats": [
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          }
        ],
        "videotype": "",
        "videourl": "",
        "videoid": 0,
        "ballnbr": 111,
        "planid": 0,
        "imageid": 0,
        "premiumvideourl": "",
        "ispremiumfree": False,
        "batteamscore": 0,
        "boundarytracker": False
      }
    },
    {
      "commentary": {
        "commtxt": "Bal Krishna to Manav Suthar, B0$",
        "timestamp": 1765183610106,
        "overnum": 18.3,
        "inningsid": 2,
        "eventtype": "NONE",
        "commentaryformats": [
          {
            "type": "",
            "value": []
          },
          {
            "type": "bold",
            "value": [
              {
                "id": "B0$",
                "value": "wide"
              }
            ]
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          }
        ],
        "videotype": "",
        "videourl": "",
        "videoid": 0,
        "ballnbr": 111,
        "planid": 0,
        "imageid": 0,
        "premiumvideourl": "",
        "ispremiumfree": False,
        "batteamscore": 0,
        "boundarytracker": False
      }
    },
    {
      "commentary": {
        "commtxt": "Bal Krishna to Manav Suthar, no run",
        "timestamp": 1765183581392,
        "overnum": 18.2,
        "inningsid": 2,
        "eventtype": "NONE",
        "commentaryformats": [
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          }
        ],
        "videotype": "",
        "videourl": "",
        "videoid": 0,
        "ballnbr": 110,
        "planid": 0,
        "imageid": 0,
        "premiumvideourl": "",
        "ispremiumfree": False,
        "batteamscore": 0,
        "boundarytracker": False
      }
    },
    {
      "commentary": {
        "commtxt": "Bal Krishna to Manav Suthar, 2 runs",
        "timestamp": 1765183580448,
        "overnum": 18.1,
        "inningsid": 2,
        "eventtype": "NONE",
        "commentaryformats": [
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          }
        ],
        "videotype": "",
        "videourl": "",
        "videoid": 0,
        "ballnbr": 109,
        "planid": 0,
        "imageid": 0,
        "premiumvideourl": "",
        "ispremiumfree": False,
        "batteamscore": 0,
        "boundarytracker": False
      }
    },
    {
      "commentary": {
        "commtxt": "B0$",
        "timestamp": 1765183575668,
        "overnum": 0,
        "inningsid": 2,
        "eventtype": "NONE",
        "commentaryformats": [
          {
            "type": "",
            "value": []
          },
          {
            "type": "bold",
            "value": [
              {
                "id": "B0$",
                "value": "Bal Krishna [2.0-0-22-0] is back into the attack"
              }
            ]
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          }
        ],
        "videotype": "",
        "videourl": "",
        "videoid": 0,
        "ballnbr": 0,
        "planid": 0,
        "imageid": 0,
        "premiumvideourl": "",
        "ispremiumfree": False,
        "batteamscore": 0,
        "boundarytracker": False
      }
    },
    {
      "commentary": {
        "commtxt": "Sushant Mishra to Manav Suthar, 1 run",
        "timestamp": 1765183517927,
        "overnum": 17.6,
        "inningsid": 2,
        "eventtype": "over-break",
        "oversep": {
          "score": 174,
          "wickets": 8,
          "inningsid": 2,
          "oversummary": "1 0 1 1 1 1 ",
          "runs": 5,
          "batstrikerdetails": "2(3)",
          "batnonstrikerdetails": "4(5)",
          "bowldetails": "3-0-21-1",
          "timestamp": 1765183517927,
          "overnum": 17.6,
          "ovrbatnames": [],
          "ovrbowlnames": [],
          "batstrikername": "Kamlesh Nagarkoti",
          "batnonstrikername": "Manav Suthar",
          "bowlname": "Sushant Mishra",
          "battingteamname": "RAJ"
        },
        "commentaryformats": [
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          }
        ],
        "videotype": "",
        "videourl": "",
        "videoid": 0,
        "ballnbr": 108,
        "planid": 0,
        "imageid": 0,
        "premiumvideourl": "",
        "ispremiumfree": False,
        "batteamscore": 0,
        "boundarytracker": False
      }
    },
    {
      "commentary": {
        "commtxt": "Sushant Mishra to K Nagarkoti, 1 run",
        "timestamp": 1765183453644,
        "overnum": 17.5,
        "inningsid": 2,
        "eventtype": "NONE",
        "commentaryformats": [
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          }
        ],
        "videotype": "",
        "videourl": "",
        "videoid": 0,
        "ballnbr": 107,
        "planid": 0,
        "imageid": 0,
        "premiumvideourl": "",
        "ispremiumfree": False,
        "batteamscore": 0,
        "boundarytracker": False
      }
    },
    {
      "commentary": {
        "commtxt": "Sushant Mishra to Manav Suthar, 1 run",
        "timestamp": 1765183410849,
        "overnum": 17.4,
        "inningsid": 2,
        "eventtype": "NONE",
        "commentaryformats": [
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          }
        ],
        "videotype": "",
        "videourl": "",
        "videoid": 0,
        "ballnbr": 106,
        "planid": 0,
        "imageid": 0,
        "premiumvideourl": "",
        "ispremiumfree": False,
        "batteamscore": 0,
        "boundarytracker": False
      }
    },
    {
      "commentary": {
        "commtxt": "Sushant Mishra to K Nagarkoti, 1 run",
        "timestamp": 1765183377946,
        "overnum": 17.3,
        "inningsid": 2,
        "eventtype": "NONE",
        "commentaryformats": [
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          }
        ],
        "videotype": "",
        "videourl": "",
        "videoid": 0,
        "ballnbr": 105,
        "planid": 0,
        "imageid": 0,
        "premiumvideourl": "",
        "ispremiumfree": False,
        "batteamscore": 0,
        "boundarytracker": False
      }
    },
    {
      "commentary": {
        "commtxt": "Sushant Mishra to K Nagarkoti, no run",
        "timestamp": 1765183346374,
        "overnum": 17.2,
        "inningsid": 2,
        "eventtype": "NONE",
        "commentaryformats": [
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          }
        ],
        "videotype": "",
        "videourl": "",
        "videoid": 0,
        "ballnbr": 104,
        "planid": 0,
        "imageid": 0,
        "premiumvideourl": "",
        "ispremiumfree": False,
        "batteamscore": 0,
        "boundarytracker": False
      }
    },
    {
      "commentary": {
        "commtxt": "Sushant Mishra to Manav Suthar, 1 run",
        "timestamp": 1765183316966,
        "overnum": 17.1,
        "inningsid": 2,
        "eventtype": "NONE",
        "commentaryformats": [
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          },
          {
            "type": "",
            "value": []
          }
        ],
        "videotype": "",
        "videourl": "",
        "videoid": 0,
        "ballnbr": 103,
        "planid": 0,
        "imageid": 0,
        "premiumvideourl": "",
        "ispremiumfree": False,
        "batteamscore": 0,
        "boundarytracker": False
      }
    }
  ],
  "miniscore": {
    "batsmanstriker": {
      "id": 14596,
      "balls": 9,
      "runs": 7,
      "fours": 0,
      "sixes": 0,
      "strkrate": "77.77777777777779",
      "name": "Manav Suthar",
      "nickname": "Manav Suthar",
      "iscaptain": False,
      "iskeeper": False,
      "outdec": "",
      "videotype": "",
      "videourl": "",
      "videoid": 0,
      "planid": 0,
      "imageid": 0,
      "premiumvideourl": "",
      "iscbplusfree": False,
      "ispremiumfree": False,
      "inmatchchange": "",
      "isoverseas": False,
      "playingxichange": ""
    },
    "batsmannonstriker": {
      "id": 0,
      "balls": 0,
      "runs": 0,
      "fours": 0,
      "sixes": 0,
      "strkrate": "0",
      "name": "",
      "nickname": "",
      "iscaptain": False,
      "iskeeper": False,
      "outdec": "",
      "videotype": "",
      "videourl": "",
      "videoid": 0,
      "planid": 0,
      "imageid": 0,
      "premiumvideourl": "",
      "iscbplusfree": False,
      "ispremiumfree": False,
      "inmatchchange": "",
      "isoverseas": False,
      "playingxichange": ""
    },
    "bowlerstriker": {
      "id": 14706,
      "overs": "3.2",
      "maidens": 0,
      "wickets": 3,
      "runs": 21,
      "economy": "6.3",
      "name": "Sushant Mishra",
      "nickname": "",
      "iscaptain": False,
      "iskeeper": False,
      "videotype": "",
      "videourl": "",
      "videoid": 0,
      "dots": 0,
      "balls": 0,
      "rpb": 0,
      "planid": 0,
      "imageid": 0,
      "premiumvideourl": "",
      "ispremiumfree": False,
      "inmatchchange": "",
      "isoverseas": False,
      "playingxichange": ""
    },
    "bowlernonstriker": {
      "id": 18973,
      "overs": "3",
      "maidens": 0,
      "wickets": 0,
      "runs": 27,
      "economy": "9",
      "name": "Bal Krishna",
      "nickname": "",
      "iscaptain": False,
      "iskeeper": False,
      "videotype": "",
      "videourl": "",
      "videoid": 0,
      "dots": 0,
      "balls": 0,
      "rpb": 0,
      "planid": 0,
      "imageid": 0,
      "premiumvideourl": "",
      "ispremiumfree": False,
      "inmatchchange": "",
      "isoverseas": False,
      "playingxichange": ""
    },
    "crr": 9.26,
    "rrr": 0,
    "inningsnbr": "1st inn",
    "lastwkt": "Sahil Dhiwan  c Kushagra b Sushant Mishra 0(1)  - 179/10 in 19.2 ov.",
    "curovsstats": "... 1 1 1  | 2 0 Wd 0 1 0 1  | W W",
    "inningsscores": {
      "inningsscore": [
        {
          "inningsid": 2,
          "batteamid": 171,
          "batteamshortname": "RAJ",
          "runs": 179,
          "wickets": 10,
          "overs": 19.2,
          "isdeclared": False,
          "isfollowon": False,
          "target": 216,
          "balls": 116
        },
        {
          "inningsid": 1,
          "batteamid": 243,
          "batteamshortname": "JHKD",
          "runs": 215,
          "wickets": 5,
          "overs": 19.6,
          "isdeclared": False,
          "isfollowon": False,
          "target": 216,
          "balls": 120
        }
      ]
    },
    "inningsid": 2,
    "performance": [
      {
        "runs": 42,
        "wickets": 6,
        "label": "Last 5 overs"
      },
      {
        "runs": 12,
        "wickets": 3,
        "label": "Last 3 overs"
      }
    ],
    "partnership": "0(1)",
    "oversrem": "",
    "pp": {
      "powerplay": [
        {
          "id": 22,
          "ovrfrom": 0.1,
          "ovrto": 6,
          "pptype": "Mandatory",
          "run": 49,
          "wickets": 0
        }
      ]
    },
    "target": 0,
    "custstatus": "",
    "ballsrem": 0,
    "rpb": 0,
    "rrpb": 0,
    "responselastupdated": 0,
    "event": ""
  },
  "matchheaders": {
    "state": "Complete",
    "status": "Jharkhand won by 36 runs",
    "matchformat": "T20",
    "matchstarttimestamp": 1765171800000,
    "teamdetails": {
      "batteamid": 171,
      "batteamname": "RAJ",
      "bowlteamid": 243,
      "bowlteamname": "JHKD"
    },
    "momplayers": {
      "player": [
        {
          "id": "10213",
          "name": "Virat Singh",
          "captain": False,
          "role": "",
          "keeper": False,
          "teamname": "India U19",
          "isheader": False,
          "imageId": 0,
          "battingStyle": "",
          "bowlingStyle": "",
          "faceimageid": 182026,
          "countryimageid": 0,
          "playingxichange": "",
          "inmatchchange": "",
          "isoverseas": False
        }
      ],
      "category": ""
    },
    "mosplayers": {
      "player": [],
      "category": ""
    },
    "winningteamid": 243,
    "revisedtarget": 0,
    "matchendtimestamp": 1765183906419,
    "seriesid": 10493,
    "matchdesc": "Elite Group D",
    "seriesname": "Syed Mushtaq Ali Trophy Elite 2025",
    "alerttype": "",
    "tossresults": {
      "tosswinnerid": 243,
      "tosswinnername": "Jharkhand",
      "decision": "Batting"
    },
    "livestreamenabled": False,
    "team1": {
      "teamid": 243,
      "teamname": "Jharkhand",
      "teamsname": "JHKD",
      "isfullmember": False,
      "isassociated": False,
      "isleagueteam": False,
      "iswomenteam": False,
      "isheader": False,
      "isactive": False,
      "teampriority": "",
      "isvideopresent": False,
      "imageid": 0,
      "countryname": "",
      "belongsto": ""
    },
    "team2": {
      "teamid": 171,
      "teamname": "Rajasthan",
      "teamsname": "RAJ",
      "isfullmember": False,
      "isassociated": False,
      "isleagueteam": False,
      "iswomenteam": False,
      "isheader": False,
      "isactive": False,
      "teampriority": "",
      "isvideopresent": False,
      "imageid": 0,
      "countryname": "",
      "belongsto": ""
    }
  },
  "appindex": {
    "seotitle": "JHKD vs RAJ Elite Group D,Syed Mushtaq Ali Trophy Elite 2025 | Cricbuzz.com",
    "weburl": "http://www.cricbuzz.com/live-cricket-scores/128622/jhkd-vs-raj-elite-group-d-syed-mushtaq-ali-trophy-elite-2025"
  },
  "matchvideos": [],
  "responselastupdated": 1765386650,
  "buzz": {
    "imageids": [
      796722,
      796714,
      796704
    ],
    "position": 2,
    "applink": "cricbuzz://buzz",
    "isbuzzenabled": True
  },
  "matchVideos": [],
  "commentarySnippetList": []
}


url = "https://Cricbuzz-Official-Cricket-API.proxy-production.allthingsdev.co/match/"+match_id+"/commentary"

payload = {}
headers = {
   'x-apihub-key': 'NFrLWAL6waj1iYNyxjW594CFWp0KHzlZVJIMOAaEX7t3OOdbfk',
   'x-apihub-host': 'Cricbuzz-Official-Cricket-API.allthingsdev.co',
   'x-apihub-endpoint': '8cb69a0f-bcaa-45b5-a016-229a2e7594f6'
}

response = requests.request("GET", url, headers=headers, data=payload)

commentary = response.json()

for ball in commentary["comwrapper"]:
  ball_num = ball["commentary"]["overnum"]
  if ball_num!=0:
    ball_commentary = ball["commentary"]["commtxt"]
    action = ball_commentary.split(',')[0].strip()
    bowler,batsman = action.split(' to ')
    print(ball_num,bowler,batsman)

19.2 Sushant Mishra Sahil Dhiwan
19.1 Sushant Mishra K Nagarkoti
18.6 Bal Krishna K Nagarkoti
18.5 Bal Krishna K Nagarkoti
18.4 Bal Krishna Manav Suthar
18.3 Bal Krishna Manav Suthar
18.3 Bal Krishna Manav Suthar
18.2 Bal Krishna Manav Suthar
18.1 Bal Krishna Manav Suthar
17.6 Sushant Mishra Manav Suthar
17.5 Sushant Mishra K Nagarkoti
17.4 Sushant Mishra Manav Suthar
17.3 Sushant Mishra K Nagarkoti
17.2 Sushant Mishra K Nagarkoti
17.1 Sushant Mishra Manav Suthar


### Score of Big Bash Womens Game with many wickets

In [3]:
overs_score = {
  "miniscore": {
    "batsmanstriker": {
      "id": 11100,
      "balls": 0,
      "runs": 0,
      "fours": 0,
      "sixes": 0,
      "strkrate": "0",
      "name": "Lauren Cheatle",
      "nickname": "Lauren Cheatle",
      "iscaptain": False,
      "iskeeper": False,
      "outdec": "",
      "videotype": "",
      "videourl": "",
      "videoid": 0,
      "planid": 0,
      "imageid": 0,
      "premiumvideourl": "",
      "iscbplusfree": False,
      "ispremiumfree": False,
      "inmatchchange": "",
      "isoverseas": False,
      "playingxichange": ""
    },
    "batsmannonstriker": {
      "id": 0,
      "balls": 0,
      "runs": 0,
      "fours": 0,
      "sixes": 0,
      "strkrate": "0",
      "name": "",
      "nickname": "",
      "iscaptain": False,
      "iskeeper": False,
      "outdec": "",
      "videotype": "",
      "videourl": "",
      "videoid": 0,
      "planid": 0,
      "imageid": 0,
      "premiumvideourl": "",
      "iscbplusfree": False,
      "ispremiumfree": False,
      "inmatchchange": "",
      "isoverseas": False,
      "playingxichange": ""
    },
    "bowlerstriker": {
      "id": 12039,
      "overs": "0.4",
      "maidens": 0,
      "wickets": 1,
      "runs": 4,
      "economy": "6",
      "name": "Maisy Gibson",
      "nickname": "",
      "iscaptain": False,
      "iskeeper": False,
      "videotype": "",
      "videourl": "",
      "videoid": 0,
      "dots": 0,
      "balls": 0,
      "rpb": 0,
      "planid": 0,
      "imageid": 0,
      "premiumvideourl": "",
      "ispremiumfree": False,
      "inmatchchange": "",
      "isoverseas": False,
      "playingxichange": ""
    },
    "bowlernonstriker": {
      "id": 20631,
      "overs": "1",
      "maidens": 0,
      "wickets": 0,
      "runs": 3,
      "economy": "3",
      "name": "Rhys McKenna",
      "nickname": "",
      "iscaptain": False,
      "iskeeper": False,
      "videotype": "",
      "videourl": "",
      "videoid": 0,
      "dots": 0,
      "balls": 0,
      "rpb": 0,
      "planid": 0,
      "imageid": 0,
      "premiumvideourl": "",
      "ispremiumfree": False,
      "inmatchchange": "",
      "isoverseas": False,
      "playingxichange": ""
    },
    "crr": 5.48,
    "rrr": 0,
    "inningsnbr": "1st inn",
    "lastwkt": "Caoimhe Bray   b Maisy Gibson 8(8)  - 42/10 in 7.4 ov.",
    "curovsstats": "... 1 W  | 0 0 1 1 1 L1  | 4 0 W W",
    "inningsscores": {
      "inningsscore": [
        {
          "inningsid": 2,
          "batteamid": 361,
          "batteamshortname": "SYSW",
          "runs": 42,
          "wickets": 10,
          "overs": 7.4,
          "isdeclared": False,
          "isfollowon": False,
          "target": 154,
          "balls": 46
        },
        {
          "inningsid": 1,
          "batteamid": 359,
          "batteamshortname": "MLSW",
          "runs": 219,
          "wickets": 4,
          "overs": 19.6,
          "isdeclared": False,
          "isfollowon": False,
          "target": 154,
          "balls": 120
        }
      ]
    },
    "inningsid": 2,
    "performance": [
      {
        "runs": 32,
        "wickets": 8,
        "label": "Last 5 overs"
      },
      {
        "runs": 16,
        "wickets": 5,
        "label": "Last 3 overs"
      }
    ],
    "partnership": "0(1)",
    "oversrem": "",
    "pp": {
      "powerplay": [
        {
          "id": 10,
          "ovrfrom": 0.1,
          "ovrto": 4,
          "pptype": "Mandatory",
          "run": 21,
          "wickets": 0
        }
      ]
    },
    "target": 154,
    "batteamscore": {
      "teamid": 361,
      "teamscore": 42,
      "teamwkts": 10
    },
    "custstatus": "Melbourne Stars Women won by 111 runs (DLS method)",
    "ballsrem": 0,
    "rpb": 0,
    "rrpb": 0,
    "responselastupdated": 1763644974,
    "event": "WICKET"
  },
  "overseplist": {
    "oversep": [
      {
        "score": 42,
        "wickets": 10,
        "inningsid": 2,
        "oversummary": "4 0 W W ",
        "runs": 4,
        "batstrikerdetails": "",
        "batnonstrikerdetails": "",
        "bowldetails": "",
        "timestamp": 1763636809866,
        "overnum": 7.4,
        "ovrbatnames": [
          "Lauren Kua",
          "Caoimhe Bray",
          "Lauren Cheatle"
        ],
        "ovrbowlnames": [
          "Maisy Gibson"
        ],
        "batstrikername": "",
        "batnonstrikername": "",
        "bowlname": "",
        "battingteamname": "SYSW"
      },
      {
        "score": 38,
        "wickets": 8,
        "inningsid": 2,
        "oversummary": "0 0 1 1 1 L1 ",
        "runs": 4,
        "batstrikerdetails": "",
        "batnonstrikerdetails": "",
        "bowldetails": "",
        "timestamp": 1763636570476,
        "overnum": 6.6,
        "ovrbatnames": [
          "Lauren Kua",
          "Caoimhe Bray"
        ],
        "ovrbowlnames": [
          "Rhys McKenna"
        ],
        "batstrikername": "",
        "batnonstrikername": "",
        "bowlname": "",
        "battingteamname": "SYSW"
      },
      {
        "score": 34,
        "wickets": 8,
        "inningsid": 2,
        "oversummary": "W 1 1 4 1 W ",
        "runs": 7,
        "batstrikerdetails": "",
        "batnonstrikerdetails": "",
        "bowldetails": "",
        "timestamp": 1763636371515,
        "overnum": 5.6,
        "ovrbatnames": [
          "Mady Villiers",
          "Caoimhe Bray",
          "Erin Burns"
        ],
        "ovrbowlnames": [
          "Annabel Sutherland"
        ],
        "batstrikername": "",
        "batnonstrikername": "",
        "bowlname": "",
        "battingteamname": "SYSW"
      },
      {
        "score": 27,
        "wickets": 6,
        "inningsid": 2,
        "oversummary": "0 4 1 W 1 W ",
        "runs": 6,
        "batstrikerdetails": "",
        "batnonstrikerdetails": "",
        "bowldetails": "",
        "timestamp": 1763636049869,
        "overnum": 4.6,
        "ovrbatnames": [
          "Mady Villiers",
          "Gardner",
          "Maitlan Brown"
        ],
        "ovrbowlnames": [
          "Sophie Day"
        ],
        "batstrikername": "",
        "batnonstrikername": "",
        "bowlname": "",
        "battingteamname": "SYSW"
      },
      {
        "score": 21,
        "wickets": 4,
        "inningsid": 2,
        "oversummary": "W Wd 0 Wd B4 W 0 1 ",
        "runs": 7,
        "batstrikerdetails": "",
        "batnonstrikerdetails": "",
        "bowldetails": "",
        "timestamp": 1763633608028,
        "overnum": 3.6,
        "ovrbatnames": [
          "Perry",
          "Elsa Hunter",
          "Maitlan Brown",
          "Gardner"
        ],
        "ovrbowlnames": [
          "Kim Garth"
        ],
        "batstrikername": "",
        "batnonstrikername": "",
        "bowlname": "",
        "battingteamname": "SYSW"
      },
      {
        "score": 14,
        "wickets": 2,
        "inningsid": 2,
        "oversummary": "0 4 0 1 0 0 ",
        "runs": 5,
        "batstrikerdetails": "",
        "batnonstrikerdetails": "",
        "bowldetails": "",
        "timestamp": 1763633228076,
        "overnum": 2.6,
        "ovrbatnames": [
          "Perry",
          "Gardner"
        ],
        "ovrbowlnames": [
          "Marizanne Kapp"
        ],
        "batstrikername": "",
        "batnonstrikername": "",
        "bowlname": "",
        "battingteamname": "SYSW"
      },
      {
        "score": 9,
        "wickets": 2,
        "inningsid": 2,
        "oversummary": "W W 0 0 0 0 ",
        "runs": 0,
        "batstrikerdetails": "",
        "batnonstrikerdetails": "",
        "bowldetails": "",
        "timestamp": 1763632832433,
        "overnum": 1.6,
        "ovrbatnames": [
          "Healy",
          "Sophia Dunkley",
          "Gardner",
          "Perry"
        ],
        "ovrbowlnames": [
          "Kim Garth"
        ],
        "batstrikername": "",
        "batnonstrikername": "",
        "bowlname": "",
        "battingteamname": "SYSW"
      },
      {
        "score": 9,
        "wickets": 0,
        "inningsid": 2,
        "oversummary": "4 0 0 0 1 4 ",
        "runs": 9,
        "batstrikerdetails": "",
        "batnonstrikerdetails": "",
        "bowldetails": "",
        "timestamp": 1763632607996,
        "overnum": 0.6,
        "ovrbatnames": [
          "Perry",
          "Sophia Dunkley"
        ],
        "ovrbowlnames": [
          "Marizanne Kapp"
        ],
        "batstrikername": "",
        "batnonstrikername": "",
        "bowlname": "",
        "battingteamname": "SYSW"
      }
    ]
  },
  "matchheaders": {
    "state": "Complete",
    "status": "Melbourne Stars Women won by 111 runs (DLS method)",
    "matchformat": "T20",
    "matchstarttimestamp": 1763626200000,
    "teamdetails": {
      "batteamid": 361,
      "batteamname": "SYSW",
      "bowlteamid": 359,
      "bowlteamname": "MLSW"
    },
    "momplayers": {
      "player": [
        {
          "id": "9114",
          "name": "Meg Lanning",
          "captain": False,
          "role": "",
          "keeper": False,
          "teamname": "Australia Women",
          "isheader": False,
          "imageId": 0,
          "battingStyle": "",
          "bowlingStyle": "",
          "faceimageid": 255006,
          "countryimageid": 0,
          "playingxichange": "",
          "inmatchchange": "",
          "isoverseas": False
        }
      ],
      "category": ""
    },
    "mosplayers": {
      "player": [],
      "category": ""
    },
    "winningteamid": 359,
    "revisedtarget": 0,
    "matchendtimestamp": 1763637056244,
    "seriesid": 10405,
    "matchdesc": "17th Match",
    "seriesname": "Womens Big Bash League 2025",
    "alerttype": "auto",
    "tossresults": {
      "tosswinnerid": 361,
      "tosswinnername": "Sydney Sixers Women",
      "decision": "Bowling"
    },
    "livestreamenabled": False,
    "team1": {
      "teamid": 361,
      "teamname": "Sydney Sixers Women",
      "teamsname": "SYSW",
      "isfullmember": False,
      "isassociated": False,
      "isleagueteam": False,
      "iswomenteam": False,
      "isheader": False,
      "isactive": False,
      "teampriority": "",
      "isvideopresent": False,
      "imageid": 0,
      "countryname": "",
      "belongsto": ""
    },
    "team2": {
      "teamid": 359,
      "teamname": "Melbourne Stars Women",
      "teamsname": "MLSW",
      "isfullmember": False,
      "isassociated": False,
      "isleagueteam": False,
      "iswomenteam": False,
      "isheader": False,
      "isactive": False,
      "teampriority": "",
      "isvideopresent": False,
      "imageid": 0,
      "countryname": "",
      "belongsto": ""
    }
  },
  "responselastupdated": 1763644974,
  "overSepList": {},
  "matchScoreDetails": {
    "customStatus": ""
  }
}
def match_dots(match_id):
  def calculate_dots(over_summary):
    over = over_summary.split(' ')
    dots = 0
    for ball in over:
      if ball == '0' or ball == 'W' or 'L' in ball or 'B' in ball:
        dots += 1
    return dots

  payload = {}
  headers = {
    'x-apihub-key': 'NFrLWAL6waj1iYNyxjW594CFWp0KHzlZVJIMOAaEX7t3OOdbfk',
    'x-apihub-host': 'Cricbuzz-Official-Cricket-API.allthingsdev.co',
    'x-apihub-endpoint': '5db6b2f0-86b9-44b5-bfc4-8c2888acd4de'
  }

  match_number_string = str(match_id)
  url = "https://Cricbuzz-Official-Cricket-API.proxy-production.allthingsdev.co/match/"+match_number_string+"/overs"
  response = requests.request("GET", url, headers=headers, data=payload)
  overs_score = response.json()  

  bowlers_info = {}
  for over in overs_score['overseplist']['oversep']:
    bowler_names = over['ovrbowlnames']
    over_summary = over['oversummary']
    if len(bowler_names) == 1:
      bowler = bowler_names[0]
      if bowler not in bowlers_info.keys():
        bowlers_info[bowler] = {'dots': 0}
        dots = calculate_dots(over_summary)
        bowlers_info[bowler]['dots'] += dots
      else:
        dots = calculate_dots(over_summary)
        bowlers_info[bowler]['dots'] += dots
    else:
      for bowler in bowler_names:
        x=2 #fill this later

  return bowlers_info

print(match_dots(126796))

{'Maisy Gibson': {'dots': 3}, 'Rhys McKenna': {'dots': 3}, 'Annabel Sutherland': {'dots': 2}, 'Sophie Day': {'dots': 3}, 'Kim Garth': {'dots': 11}, 'Marizanne Kapp': {'dots': 7}}


### India vs SA t20i match

In [ ]:
over_score = {
  "miniscore": {
    "batsmanstriker": {
      "id": 8733,
      "balls": 56,
      "runs": 51,
      "fours": 2,
      "sixes": 4,
      "strkrate": "91.07",
      "name": "KL Rahul",
      "nickname": "KL Rahul",
      "iscaptain": False,
      "iskeeper": False,
      "outdec": "",
      "videotype": "",
      "videourl": "",
      "videoid": 0,
      "planid": 0,
      "imageid": 0,
      "premiumvideourl": "",
      "iscbplusfree": False,
      "ispremiumfree": False,
      "inmatchchange": "",
      "isoverseas": False,
      "playingxichange": ""
    },
    "batsmannonstriker": {
      "id": 7915,
      "balls": 33,
      "runs": 50,
      "fours": 5,
      "sixes": 3,
      "strkrate": "151.52",
      "name": "Suryakumar Yadav",
      "nickname": "Suryakumar Yadav",
      "iscaptain": False,
      "iskeeper": False,
      "outdec": "",
      "videotype": "",
      "videourl": "",
      "videoid": 0,
      "planid": 0,
      "imageid": 0,
      "premiumvideourl": "",
      "iscbplusfree": False,
      "ispremiumfree": False,
      "inmatchchange": "",
      "isoverseas": False,
      "playingxichange": ""
    },
    "bowlerstriker": {
      "id": 10173,
      "overs": "2.4",
      "maidens": 0,
      "wickets": 0,
      "runs": 27,
      "economy": "10.13",
      "name": "Tabraiz Shamsi",
      "nickname": "",
      "iscaptain": False,
      "iskeeper": False,
      "videotype": "",
      "videourl": "",
      "videoid": 0,
      "dots": 0,
      "balls": 0,
      "rpb": 0,
      "planid": 0,
      "imageid": 0,
      "premiumvideourl": "",
      "ispremiumfree": False,
      "inmatchchange": "",
      "isoverseas": False,
      "playingxichange": ""
    },
    "bowlernonstriker": {
      "id": 9585,
      "overs": "4",
      "maidens": 1,
      "wickets": 1,
      "runs": 16,
      "economy": "4",
      "name": "Kagiso Rabada",
      "nickname": "",
      "iscaptain": False,
      "iskeeper": False,
      "videotype": "",
      "videourl": "",
      "videoid": 0,
      "dots": 0,
      "balls": 0,
      "rpb": 0,
      "planid": 0,
      "imageid": 0,
      "premiumvideourl": "",
      "ispremiumfree": False,
      "inmatchchange": "",
      "isoverseas": False,
      "playingxichange": ""
    },
    "crr": 6.6,
    "rrr": 0,
    "inningsnbr": "1st inn",
    "lastwkt": "Virat Kohli  c de Kock b Nortje 3(9)  - 17/2 in 6.1 ov.",
    "curovsstats": "... 4 0  | 0 1 4 4 0 1  | 1 1 1 6",
    "inningsscores": {
      "inningsscore": [
        {
          "inningsid": 2,
          "batteamid": 2,
          "batteamshortname": "IND",
          "runs": 110,
          "wickets": 2,
          "overs": 16.4,
          "isdeclared": False,
          "isfollowon": False,
          "target": 107,
          "balls": 100
        },
        {
          "inningsid": 1,
          "batteamid": 11,
          "batteamshortname": "RSA",
          "runs": 106,
          "wickets": 8,
          "overs": 19.6,
          "isdeclared": False,
          "isfollowon": False,
          "target": 107,
          "balls": 120
        }
      ]
    },
    "inningsid": 2,
    "performance": [
      {
        "runs": 48,
        "wickets": 0,
        "label": "Last 5 overs"
      },
      {
        "runs": 31,
        "wickets": 0,
        "label": "Last 3 overs"
      }
    ],
    "udrs": {
      "team1id": 2,
      "team1remaining": 2,
      "team1successful": 0,
      "team1unsuccessful": 0,
      "team2id": 11,
      "team2remaining": 0,
      "team2successful": 1,
      "team2unsuccessful": 2
    },
    "partnership": "93(63)",
    "oversrem": "",
    "pp": {
      "powerplay": [
        {
          "id": 68,
          "ovrfrom": 0.1,
          "ovrto": 6,
          "pptype": "Mandatory",
          "run": 17,
          "wickets": 0
        }
      ]
    },
    "target": 107,
    "batteamscore": {
      "teamid": 2,
      "teamscore": 110,
      "teamwkts": 2
    },
    "custstatus": "India won by 8 wkts",
    "ballsrem": 0,
    "rpb": 0,
    "rrpb": 0,
    "responselastupdated": 1763740066,
    "event": "FIFTY,SIX"
  },
  "overseplist": {
    "oversep": [
      {
        "score": 110,
        "wickets": 2,
        "inningsid": 2,
        "oversummary": "1 1 1 6 ",
        "runs": 9,
        "batstrikerdetails": "",
        "batnonstrikerdetails": "",
        "bowldetails": "",
        "timestamp": 1664383471886,
        "overnum": 16.4,
        "ovrbatnames": [
          "Rahul",
          "Suryakumar Yadav"
        ],
        "ovrbowlnames": [
          "Tabraiz Shamsi"
        ],
        "batstrikername": "",
        "batnonstrikername": "",
        "bowlname": "",
        "battingteamname": "IND"
      },
      {
        "score": 101,
        "wickets": 2,
        "inningsid": 2,
        "oversummary": "0 1 4 4 0 1 ",
        "runs": 10,
        "batstrikerdetails": "",
        "batnonstrikerdetails": "",
        "bowldetails": "",
        "timestamp": 1664383250008,
        "overnum": 15.6,
        "ovrbatnames": [
          "Rahul",
          "Suryakumar Yadav"
        ],
        "ovrbowlnames": [
          "Kagiso Rabada"
        ],
        "batstrikername": "",
        "batnonstrikername": "",
        "bowlname": "",
        "battingteamname": "IND"
      },
      {
        "score": 91,
        "wickets": 2,
        "inningsid": 2,
        "oversummary": "0 0 6 1 4 0 ",
        "runs": 11,
        "batstrikerdetails": "",
        "batnonstrikerdetails": "",
        "bowldetails": "",
        "timestamp": 1664382994583,
        "overnum": 14.6,
        "ovrbatnames": [
          "Rahul",
          "Suryakumar Yadav"
        ],
        "ovrbowlnames": [
          "Anrich Nortje"
        ],
        "batstrikername": "",
        "batnonstrikername": "",
        "bowlname": "",
        "battingteamname": "IND"
      },
      {
        "score": 80,
        "wickets": 2,
        "inningsid": 2,
        "oversummary": "1 0 1 0 0 1 ",
        "runs": 3,
        "batstrikerdetails": "",
        "batnonstrikerdetails": "",
        "bowldetails": "",
        "timestamp": 1664382701931,
        "overnum": 13.6,
        "ovrbatnames": [
          "Rahul",
          "Suryakumar Yadav"
        ],
        "ovrbowlnames": [
          "Wayne Parnell"
        ],
        "batstrikername": "",
        "batnonstrikername": "",
        "bowlname": "",
        "battingteamname": "IND"
      },
      {
        "score": 77,
        "wickets": 2,
        "inningsid": 2,
        "oversummary": "1 1 1 6 1 1 ",
        "runs": 11,
        "batstrikerdetails": "",
        "batnonstrikerdetails": "",
        "bowldetails": "",
        "timestamp": 1664382427887,
        "overnum": 12.6,
        "ovrbatnames": [
          "Rahul",
          "Suryakumar Yadav"
        ],
        "ovrbowlnames": [
          "Keshav Maharaj"
        ],
        "batstrikername": "",
        "batnonstrikername": "",
        "bowlname": "",
        "battingteamname": "IND"
      },
      {
        "score": 66,
        "wickets": 2,
        "inningsid": 2,
        "oversummary": "6 2 1 0 0 4 ",
        "runs": 13,
        "batstrikerdetails": "",
        "batnonstrikerdetails": "",
        "bowldetails": "",
        "timestamp": 1664382177093,
        "overnum": 11.6,
        "ovrbatnames": [
          "Rahul",
          "Suryakumar Yadav"
        ],
        "ovrbowlnames": [
          "Tabraiz Shamsi"
        ],
        "batstrikername": "",
        "batnonstrikername": "",
        "bowlname": "",
        "battingteamname": "IND"
      },
      {
        "score": 53,
        "wickets": 2,
        "inningsid": 2,
        "oversummary": "4 0 0 1 1 0 ",
        "runs": 6,
        "batstrikerdetails": "",
        "batnonstrikerdetails": "",
        "bowldetails": "",
        "timestamp": 1664381953138,
        "overnum": 10.6,
        "ovrbatnames": [
          "Rahul",
          "Suryakumar Yadav"
        ],
        "ovrbowlnames": [
          "Keshav Maharaj"
        ],
        "batstrikername": "",
        "batnonstrikername": "",
        "bowlname": "",
        "battingteamname": "IND"
      },
      {
        "score": 47,
        "wickets": 2,
        "inningsid": 2,
        "oversummary": "1 Wd2 0 0 0 0 6 ",
        "runs": 9,
        "batstrikerdetails": "",
        "batnonstrikerdetails": "",
        "bowldetails": "",
        "timestamp": 1664381748430,
        "overnum": 9.6,
        "ovrbatnames": [
          "Rahul",
          "Suryakumar Yadav"
        ],
        "ovrbowlnames": [
          "Anrich Nortje"
        ],
        "batstrikername": "",
        "batnonstrikername": "",
        "bowlname": "",
        "battingteamname": "IND"
      },
      {
        "score": 38,
        "wickets": 2,
        "inningsid": 2,
        "oversummary": "1 0 1 2 0 0 ",
        "runs": 4,
        "batstrikerdetails": "",
        "batnonstrikerdetails": "",
        "bowldetails": "",
        "timestamp": 1664381125797,
        "overnum": 8.6,
        "ovrbatnames": [
          "Rahul",
          "Suryakumar Yadav"
        ],
        "ovrbowlnames": [
          "Keshav Maharaj"
        ],
        "batstrikername": "",
        "batnonstrikername": "",
        "bowlname": "",
        "battingteamname": "IND"
      },
      {
        "score": 34,
        "wickets": 2,
        "inningsid": 2,
        "oversummary": "1 0 Wd 1 1 1 0 ",
        "runs": 5,
        "batstrikerdetails": "",
        "batnonstrikerdetails": "",
        "bowldetails": "",
        "timestamp": 1664380913126,
        "overnum": 7.6,
        "ovrbatnames": [
          "Rahul",
          "Suryakumar Yadav"
        ],
        "ovrbowlnames": [
          "Tabraiz Shamsi"
        ],
        "batstrikername": "",
        "batnonstrikername": "",
        "bowlname": "",
        "battingteamname": "IND"
      },
      {
        "score": 29,
        "wickets": 2,
        "inningsid": 2,
        "oversummary": "W 0 6 6 0 0 ",
        "runs": 12,
        "batstrikerdetails": "",
        "batnonstrikerdetails": "",
        "bowldetails": "",
        "timestamp": 1664380633235,
        "overnum": 6.6,
        "ovrbatnames": [
          "Suryakumar Yadav",
          "Rahul"
        ],
        "ovrbowlnames": [
          "Anrich Nortje"
        ],
        "batstrikername": "",
        "batnonstrikername": "",
        "bowlname": "",
        "battingteamname": "IND"
      }
    ]
  },
  "matchheaders": {
    "state": "Complete",
    "status": "India won by 8 wkts",
    "matchformat": "T20",
    "matchstarttimestamp": 1664371800000,
    "teamdetails": {
      "batteamid": 2,
      "batteamname": "IND",
      "bowlteamid": 11,
      "bowlteamname": "RSA"
    },
    "momplayers": {
      "player": [
        {
          "id": "13217",
          "name": "Arshdeep Singh",
          "captain": False,
          "role": "",
          "keeper": False,
          "teamname": "India U19",
          "isheader": False,
          "imageId": 0,
          "battingStyle": "",
          "bowlingStyle": "",
          "faceimageid": 616527,
          "countryimageid": 0,
          "playingxichange": "",
          "inmatchchange": "",
          "isoverseas": False
        }
      ],
      "category": ""
    },
    "mosplayers": {
      "player": [],
      "category": ""
    },
    "winningteamid": 2,
    "revisedtarget": 0,
    "matchendtimestamp": 1664383503337,
    "seriesid": 4471,
    "matchdesc": "1st T20I",
    "seriesname": "South Africa tour of India, 2022",
    "alerttype": "",
    "tossresults": {
      "tosswinnerid": 2,
      "tosswinnername": "India",
      "decision": "Bowling"
    },
    "livestreamenabled": False,
    "team1": {
      "teamid": 2,
      "teamname": "India",
      "teamsname": "IND",
      "isfullmember": False,
      "isassociated": False,
      "isleagueteam": False,
      "iswomenteam": False,
      "isheader": False,
      "isactive": False,
      "teampriority": "",
      "isvideopresent": False,
      "imageid": 0,
      "countryname": "",
      "belongsto": ""
    },
    "team2": {
      "teamid": 11,
      "teamname": "South Africa",
      "teamsname": "RSA",
      "isfullmember": False,
      "isassociated": False,
      "isleagueteam": False,
      "iswomenteam": False,
      "isheader": False,
      "isactive": False,
      "teampriority": "",
      "isvideopresent": False,
      "imageid": 0,
      "countryname": "",
      "belongsto": ""
    }
  },
  "responselastupdated": 1763740066,
  "overSepList": {},
  "matchScoreDetails": {
    "customStatus": ""
  }
}

### IPL Series Data

In [ ]:
series_info = {
  "matchDetails": [
    {
      "matchDetailsMap": {
        "key": "Sat, 22 Mar 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 114960,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "1st Match",
              "matchFormat": "T20",
              "startDate": "1742652000000",
              "endDate": "1742652000000",
              "state": "complete",
              "status": "Royal Challengers Bengaluru won by 7 wkts",
              "team1": {
                "teamId": 63,
                "teamName": "KOLKATA KNIGHT RIDERS",
                "teamSName": "KKR",
                "imageId": 225646
              },
              "team2": {
                "teamId": 59,
                "teamName": "ROYAL CHALLENGERS BENGALURU",
                "teamSName": "RCB",
                "imageId": 225643
              },
              "venueInfo": {
                "ground": "Eden Gardens",
                "city": "Kolkata",
                "timezone": "+05:30"
              },
              "currBatTeamId": 59,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 174,
                  "wickets": 8,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 177,
                  "wickets": 3,
                  "overs": 16.2
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Sun, 23 Mar 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 114967,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "2nd Match",
              "matchFormat": "T20",
              "startDate": "1742724000000",
              "endDate": "1742724000000",
              "state": "complete",
              "status": "Sunrisers Hyderabad won by 44 runs",
              "team1": {
                "teamId": 255,
                "teamName": "SUNRISERS HYDERABAD",
                "teamSName": "SRH",
                "imageId": 225649
              },
              "team2": {
                "teamId": 64,
                "teamName": "RAJASTHAN ROYALS",
                "teamSName": "RR",
                "imageId": 225647
              },
              "venueInfo": {
                "ground": "Rajiv Gandhi International Stadium",
                "city": "Hyderabad",
                "timezone": "+05:30"
              },
              "currBatTeamId": 255,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 286,
                  "wickets": 6,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 242,
                  "wickets": 6,
                  "overs": 20
                }
              }
            }
          },
          {
            "matchInfo": {
              "matchId": 114976,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "3rd Match",
              "matchFormat": "T20",
              "startDate": "1742738400000",
              "endDate": "1742738400000",
              "state": "complete",
              "status": "Chennai Super Kings won by 4 wkts",
              "team1": {
                "teamId": 58,
                "teamName": "CHENNAI SUPER KINGS",
                "teamSName": "CSK",
                "imageId": 225641
              },
              "team2": {
                "teamId": 62,
                "teamName": "MUMBAI INDIANS",
                "teamSName": "MI",
                "imageId": 225645
              },
              "venueInfo": {
                "ground": "MA Chidambaram Stadium",
                "city": "Chennai",
                "timezone": "+05:30"
              },
              "currBatTeamId": 58,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 158,
                  "wickets": 6,
                  "overs": 19.1
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 155,
                  "wickets": 9,
                  "overs": 20
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Mon, 24 Mar 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 114985,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "4th Match",
              "matchFormat": "T20",
              "startDate": "1742824800000",
              "endDate": "1742824800000",
              "state": "complete",
              "status": "Delhi Capitals won by 1 wkt",
              "team1": {
                "teamId": 61,
                "teamName": "DELHI CAPITALS",
                "teamSName": "DC",
                "imageId": 225644
              },
              "team2": {
                "teamId": 966,
                "teamName": "LUCKNOW SUPER GIANTS",
                "teamSName": "LSG",
                "imageId": 389444
              },
              "venueInfo": {
                "ground": "ACA-VDCA Cricket Stadium",
                "city": "Visakhapatnam",
                "timezone": "+05:30"
              },
              "currBatTeamId": 61,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 211,
                  "wickets": 9,
                  "overs": 19.3
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 209,
                  "wickets": 8,
                  "overs": 20
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Tue, 25 Mar 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 114987,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "5th Match",
              "matchFormat": "T20",
              "startDate": "1742911200000",
              "endDate": "1742911200000",
              "state": "complete",
              "status": "Punjab Kings won by 11 runs",
              "team1": {
                "teamId": 971,
                "teamName": "GUJARAT TITANS",
                "teamSName": "GT",
                "imageId": 235085
              },
              "team2": {
                "teamId": 65,
                "teamName": "PUNJAB KINGS",
                "teamSName": "PBKS",
                "imageId": 225648
              },
              "venueInfo": {
                "ground": "Narendra Modi Stadium",
                "city": "Ahmedabad",
                "timezone": "+05:30"
              },
              "currBatTeamId": 65,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 232,
                  "wickets": 5,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 243,
                  "wickets": 5,
                  "overs": 20
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Wed, 26 Mar 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 114996,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "6th Match",
              "matchFormat": "T20",
              "startDate": "1742997600000",
              "endDate": "1742997600000",
              "state": "complete",
              "status": "Kolkata Knight Riders won by 8 wkts",
              "team1": {
                "teamId": 64,
                "teamName": "RAJASTHAN ROYALS",
                "teamSName": "RR",
                "imageId": 225647
              },
              "team2": {
                "teamId": 63,
                "teamName": "KOLKATA KNIGHT RIDERS",
                "teamSName": "KKR",
                "imageId": 225646
              },
              "venueInfo": {
                "ground": "Barsapara Cricket Stadium",
                "city": "Guwahati",
                "timezone": "+05:30"
              },
              "currBatTeamId": 63,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 151,
                  "wickets": 9,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 153,
                  "wickets": 2,
                  "overs": 17.3
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Thu, 27 Mar 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115005,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "7th Match",
              "matchFormat": "T20",
              "startDate": "1743084000000",
              "endDate": "1743084000000",
              "state": "complete",
              "status": "Lucknow Super Giants won by 5 wkts",
              "team1": {
                "teamId": 255,
                "teamName": "SUNRISERS HYDERABAD",
                "teamSName": "SRH",
                "imageId": 225649
              },
              "team2": {
                "teamId": 966,
                "teamName": "LUCKNOW SUPER GIANTS",
                "teamSName": "LSG",
                "imageId": 389444
              },
              "venueInfo": {
                "ground": "Rajiv Gandhi International Stadium",
                "city": "Hyderabad",
                "timezone": "+05:30"
              },
              "currBatTeamId": 966,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 190,
                  "wickets": 9,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 193,
                  "wickets": 5,
                  "overs": 16.1
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Fri, 28 Mar 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115012,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "8th Match",
              "matchFormat": "T20",
              "startDate": "1743170400000",
              "endDate": "1743170400000",
              "state": "complete",
              "status": "Royal Challengers Bengaluru won by 50 runs",
              "team1": {
                "teamId": 58,
                "teamName": "CHENNAI SUPER KINGS",
                "teamSName": "CSK",
                "imageId": 225641
              },
              "team2": {
                "teamId": 59,
                "teamName": "ROYAL CHALLENGERS BENGALURU",
                "teamSName": "RCB",
                "imageId": 225643
              },
              "venueInfo": {
                "ground": "MA Chidambaram Stadium",
                "city": "Chennai",
                "timezone": "+05:30"
              },
              "currBatTeamId": 59,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 146,
                  "wickets": 8,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 196,
                  "wickets": 7,
                  "overs": 20
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Sat, 29 Mar 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115014,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "9th Match",
              "matchFormat": "T20",
              "startDate": "1743256800000",
              "endDate": "1743256800000",
              "state": "complete",
              "status": "Gujarat Titans won by 36 runs",
              "team1": {
                "teamId": 971,
                "teamName": "GUJARAT TITANS",
                "teamSName": "GT",
                "imageId": 235085
              },
              "team2": {
                "teamId": 62,
                "teamName": "MUMBAI INDIANS",
                "teamSName": "MI",
                "imageId": 225645
              },
              "venueInfo": {
                "ground": "Narendra Modi Stadium",
                "city": "Ahmedabad",
                "timezone": "+05:30"
              },
              "currBatTeamId": 971,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 196,
                  "wickets": 8,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 160,
                  "wickets": 6,
                  "overs": 20
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Sun, 30 Mar 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115021,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "10th Match",
              "matchFormat": "T20",
              "startDate": "1743328800000",
              "endDate": "1743328800000",
              "state": "complete",
              "status": "Delhi Capitals won by 7 wkts",
              "team1": {
                "teamId": 61,
                "teamName": "DELHI CAPITALS",
                "teamSName": "DC",
                "imageId": 225644
              },
              "team2": {
                "teamId": 255,
                "teamName": "SUNRISERS HYDERABAD",
                "teamSName": "SRH",
                "imageId": 225649
              },
              "venueInfo": {
                "ground": "ACA-VDCA Cricket Stadium",
                "city": "Visakhapatnam",
                "timezone": "+05:30"
              },
              "currBatTeamId": 61,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 166,
                  "wickets": 3,
                  "overs": 16
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 163,
                  "wickets": 10,
                  "overs": 18.4
                }
              }
            }
          },
          {
            "matchInfo": {
              "matchId": 115030,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "11th Match",
              "matchFormat": "T20",
              "startDate": "1743343200000",
              "endDate": "1743343200000",
              "state": "complete",
              "status": "Rajasthan Royals won by 6 runs",
              "team1": {
                "teamId": 64,
                "teamName": "RAJASTHAN ROYALS",
                "teamSName": "RR",
                "imageId": 225647
              },
              "team2": {
                "teamId": 58,
                "teamName": "CHENNAI SUPER KINGS",
                "teamSName": "CSK",
                "imageId": 225641
              },
              "venueInfo": {
                "ground": "Barsapara Cricket Stadium",
                "city": "Guwahati",
                "timezone": "+05:30"
              },
              "currBatTeamId": 64,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 182,
                  "wickets": 9,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 176,
                  "wickets": 6,
                  "overs": 20
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Mon, 31 Mar 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115032,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "12th Match",
              "matchFormat": "T20",
              "startDate": "1743429600000",
              "endDate": "1743429600000",
              "state": "complete",
              "status": "Mumbai Indians won by 8 wkts",
              "team1": {
                "teamId": 62,
                "teamName": "MUMBAI INDIANS",
                "teamSName": "MI",
                "imageId": 225645
              },
              "team2": {
                "teamId": 63,
                "teamName": "KOLKATA KNIGHT RIDERS",
                "teamSName": "KKR",
                "imageId": 225646
              },
              "venueInfo": {
                "ground": "Wankhede Stadium",
                "city": "Mumbai",
                "timezone": "+05:30"
              },
              "currBatTeamId": 62,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 121,
                  "wickets": 2,
                  "overs": 12.5
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 116,
                  "wickets": 10,
                  "overs": 16.2
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Tue, 01 Apr 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115039,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "13th Match",
              "matchFormat": "T20",
              "startDate": "1743516000000",
              "endDate": "1743516000000",
              "state": "complete",
              "status": "Punjab Kings won by 8 wkts",
              "team1": {
                "teamId": 966,
                "teamName": "LUCKNOW SUPER GIANTS",
                "teamSName": "LSG",
                "imageId": 389444
              },
              "team2": {
                "teamId": 65,
                "teamName": "PUNJAB KINGS",
                "teamSName": "PBKS",
                "imageId": 225648
              },
              "venueInfo": {
                "ground": "Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium",
                "city": "Lucknow",
                "timezone": "+05:30"
              },
              "currBatTeamId": 65,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 171,
                  "wickets": 7,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 177,
                  "wickets": 2,
                  "overs": 16.2
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Wed, 02 Apr 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115048,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "14th Match",
              "matchFormat": "T20",
              "startDate": "1743602400000",
              "endDate": "1743602400000",
              "state": "complete",
              "status": "Gujarat Titans won by 8 wkts",
              "team1": {
                "teamId": 59,
                "teamName": "ROYAL CHALLENGERS BENGALURU",
                "teamSName": "RCB",
                "imageId": 225643
              },
              "team2": {
                "teamId": 971,
                "teamName": "GUJARAT TITANS",
                "teamSName": "GT",
                "imageId": 235085
              },
              "venueInfo": {
                "ground": "M.Chinnaswamy Stadium",
                "city": "Bengaluru",
                "timezone": "+05:30"
              },
              "currBatTeamId": 971,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 169,
                  "wickets": 8,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 170,
                  "wickets": 2,
                  "overs": 17.5
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Thu, 03 Apr 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115050,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "15th Match",
              "matchFormat": "T20",
              "startDate": "1743688800000",
              "endDate": "1743688800000",
              "state": "complete",
              "status": "Kolkata Knight Riders won by 80 runs",
              "team1": {
                "teamId": 63,
                "teamName": "KOLKATA KNIGHT RIDERS",
                "teamSName": "KKR",
                "imageId": 225646
              },
              "team2": {
                "teamId": 255,
                "teamName": "SUNRISERS HYDERABAD",
                "teamSName": "SRH",
                "imageId": 225649
              },
              "venueInfo": {
                "ground": "Eden Gardens",
                "city": "Kolkata",
                "timezone": "+05:30"
              },
              "currBatTeamId": 63,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 200,
                  "wickets": 6,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 120,
                  "wickets": 10,
                  "overs": 16.4
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Fri, 04 Apr 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115059,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "16th Match",
              "matchFormat": "T20",
              "startDate": "1743775200000",
              "endDate": "1743775200000",
              "state": "complete",
              "status": "Lucknow Super Giants won by 12 runs",
              "team1": {
                "teamId": 966,
                "teamName": "LUCKNOW SUPER GIANTS",
                "teamSName": "LSG",
                "imageId": 389444
              },
              "team2": {
                "teamId": 62,
                "teamName": "MUMBAI INDIANS",
                "teamSName": "MI",
                "imageId": 225645
              },
              "venueInfo": {
                "ground": "Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium",
                "city": "Lucknow",
                "timezone": "+05:30"
              },
              "currBatTeamId": 966,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 203,
                  "wickets": 8,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 191,
                  "wickets": 5,
                  "overs": 20
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Sat, 05 Apr 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115068,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "17th Match",
              "matchFormat": "T20",
              "startDate": "1743847200000",
              "endDate": "1743847200000",
              "state": "complete",
              "status": "Delhi Capitals won by 25 runs",
              "team1": {
                "teamId": 58,
                "teamName": "CHENNAI SUPER KINGS",
                "teamSName": "CSK",
                "imageId": 225641
              },
              "team2": {
                "teamId": 61,
                "teamName": "DELHI CAPITALS",
                "teamSName": "DC",
                "imageId": 225644
              },
              "venueInfo": {
                "ground": "MA Chidambaram Stadium",
                "city": "Chennai",
                "timezone": "+05:30"
              },
              "currBatTeamId": 61,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 158,
                  "wickets": 5,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 183,
                  "wickets": 6,
                  "overs": 20
                }
              }
            }
          },
          {
            "matchInfo": {
              "matchId": 115075,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "18th Match",
              "matchFormat": "T20",
              "startDate": "1743861600000",
              "endDate": "1743861600000",
              "state": "complete",
              "status": "Rajasthan Royals won by 50 runs",
              "team1": {
                "teamId": 65,
                "teamName": "PUNJAB KINGS",
                "teamSName": "PBKS",
                "imageId": 225648
              },
              "team2": {
                "teamId": 64,
                "teamName": "RAJASTHAN ROYALS",
                "teamSName": "RR",
                "imageId": 225647
              },
              "venueInfo": {
                "ground": "Maharaja Yadavindra Singh International Cricket Stadium, Mullanpur",
                "city": "New Chandigarh",
                "timezone": "+05:30"
              },
              "currBatTeamId": 64,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 155,
                  "wickets": 9,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 205,
                  "wickets": 4,
                  "overs": 20
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Sun, 06 Apr 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115093,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "19th Match",
              "matchFormat": "T20",
              "startDate": "1743948000000",
              "endDate": "1743948000000",
              "state": "complete",
              "status": "Gujarat Titans won by 7 wkts",
              "team1": {
                "teamId": 255,
                "teamName": "SUNRISERS HYDERABAD",
                "teamSName": "SRH",
                "imageId": 225649
              },
              "team2": {
                "teamId": 971,
                "teamName": "GUJARAT TITANS",
                "teamSName": "GT",
                "imageId": 235085
              },
              "venueInfo": {
                "ground": "Rajiv Gandhi International Stadium",
                "city": "Hyderabad",
                "timezone": "+05:30"
              },
              "currBatTeamId": 971,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 152,
                  "wickets": 8,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 153,
                  "wickets": 3,
                  "overs": 16.4
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Mon, 07 Apr 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115095,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "20th Match",
              "matchFormat": "T20",
              "startDate": "1744034400000",
              "endDate": "1744034400000",
              "state": "complete",
              "status": "Royal Challengers Bengaluru won by 12 runs",
              "team1": {
                "teamId": 62,
                "teamName": "MUMBAI INDIANS",
                "teamSName": "MI",
                "imageId": 225645
              },
              "team2": {
                "teamId": 59,
                "teamName": "ROYAL CHALLENGERS BENGALURU",
                "teamSName": "RCB",
                "imageId": 225643
              },
              "venueInfo": {
                "ground": "Wankhede Stadium",
                "city": "Mumbai",
                "timezone": "+05:30"
              },
              "currBatTeamId": 59,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 209,
                  "wickets": 9,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 221,
                  "wickets": 5,
                  "overs": 20
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Tue, 08 Apr 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115084,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "21st Match",
              "matchFormat": "T20",
              "startDate": "1744106400000",
              "endDate": "1744106400000",
              "state": "complete",
              "status": "Lucknow Super Giants won by 4 runs",
              "team1": {
                "teamId": 63,
                "teamName": "KOLKATA KNIGHT RIDERS",
                "teamSName": "KKR",
                "imageId": 225646
              },
              "team2": {
                "teamId": 966,
                "teamName": "LUCKNOW SUPER GIANTS",
                "teamSName": "LSG",
                "imageId": 389444
              },
              "venueInfo": {
                "ground": "Eden Gardens",
                "city": "Kolkata",
                "timezone": "+05:30"
              },
              "currBatTeamId": 966,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 234,
                  "wickets": 7,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 238,
                  "wickets": 3,
                  "overs": 20
                }
              }
            }
          },
          {
            "matchInfo": {
              "matchId": 115102,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "22nd Match",
              "matchFormat": "T20",
              "startDate": "1744120800000",
              "endDate": "1744120800000",
              "state": "complete",
              "status": "Punjab Kings won by 18 runs",
              "team1": {
                "teamId": 65,
                "teamName": "PUNJAB KINGS",
                "teamSName": "PBKS",
                "imageId": 225648
              },
              "team2": {
                "teamId": 58,
                "teamName": "CHENNAI SUPER KINGS",
                "teamSName": "CSK",
                "imageId": 225641
              },
              "venueInfo": {
                "ground": "Maharaja Yadavindra Singh International Cricket Stadium, Mullanpur",
                "city": "New Chandigarh",
                "timezone": "+05:30"
              },
              "currBatTeamId": 65,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 219,
                  "wickets": 6,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 201,
                  "wickets": 5,
                  "overs": 20
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Wed, 09 Apr 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115104,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "23rd Match",
              "matchFormat": "T20",
              "startDate": "1744207200000",
              "endDate": "1744207200000",
              "state": "complete",
              "status": "Gujarat Titans won by 58 runs",
              "team1": {
                "teamId": 971,
                "teamName": "GUJARAT TITANS",
                "teamSName": "GT",
                "imageId": 235085
              },
              "team2": {
                "teamId": 64,
                "teamName": "RAJASTHAN ROYALS",
                "teamSName": "RR",
                "imageId": 225647
              },
              "venueInfo": {
                "ground": "Narendra Modi Stadium",
                "city": "Ahmedabad",
                "timezone": "+05:30"
              },
              "currBatTeamId": 971,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 217,
                  "wickets": 6,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 159,
                  "wickets": 10,
                  "overs": 19.2
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Thu, 10 Apr 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115111,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "24th Match",
              "matchFormat": "T20",
              "startDate": "1744293600000",
              "endDate": "1744293600000",
              "state": "complete",
              "status": "Delhi Capitals won by 6 wkts",
              "team1": {
                "teamId": 59,
                "teamName": "ROYAL CHALLENGERS BENGALURU",
                "teamSName": "RCB",
                "imageId": 225643
              },
              "team2": {
                "teamId": 61,
                "teamName": "DELHI CAPITALS",
                "teamSName": "DC",
                "imageId": 225644
              },
              "venueInfo": {
                "ground": "M.Chinnaswamy Stadium",
                "city": "Bengaluru",
                "timezone": "+05:30"
              },
              "currBatTeamId": 61,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 163,
                  "wickets": 7,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 169,
                  "wickets": 4,
                  "overs": 17.5
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Fri, 11 Apr 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115113,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "25th Match",
              "matchFormat": "T20",
              "startDate": "1744380000000",
              "endDate": "1744380000000",
              "state": "complete",
              "status": "Kolkata Knight Riders won by 8 wkts",
              "team1": {
                "teamId": 58,
                "teamName": "CHENNAI SUPER KINGS",
                "teamSName": "CSK",
                "imageId": 225641
              },
              "team2": {
                "teamId": 63,
                "teamName": "KOLKATA KNIGHT RIDERS",
                "teamSName": "KKR",
                "imageId": 225646
              },
              "venueInfo": {
                "ground": "MA Chidambaram Stadium",
                "city": "Chennai",
                "timezone": "+05:30"
              },
              "currBatTeamId": 63,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 103,
                  "wickets": 9,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 107,
                  "wickets": 2,
                  "overs": 10.1
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Sat, 12 Apr 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115122,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "26th Match",
              "matchFormat": "T20",
              "startDate": "1744452000000",
              "endDate": "1744452000000",
              "state": "complete",
              "status": "Lucknow Super Giants won by 6 wkts",
              "team1": {
                "teamId": 966,
                "teamName": "LUCKNOW SUPER GIANTS",
                "teamSName": "LSG",
                "imageId": 389444
              },
              "team2": {
                "teamId": 971,
                "teamName": "GUJARAT TITANS",
                "teamSName": "GT",
                "imageId": 235085
              },
              "venueInfo": {
                "ground": "Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium",
                "city": "Lucknow",
                "timezone": "+05:30"
              },
              "currBatTeamId": 966,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 186,
                  "wickets": 4,
                  "overs": 19.3
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 180,
                  "wickets": 6,
                  "overs": 20
                }
              }
            }
          },
          {
            "matchInfo": {
              "matchId": 115129,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "27th Match",
              "matchFormat": "T20",
              "startDate": "1744466400000",
              "endDate": "1744466400000",
              "state": "complete",
              "status": "Sunrisers Hyderabad won by 8 wkts",
              "team1": {
                "teamId": 255,
                "teamName": "SUNRISERS HYDERABAD",
                "teamSName": "SRH",
                "imageId": 225649
              },
              "team2": {
                "teamId": 65,
                "teamName": "PUNJAB KINGS",
                "teamSName": "PBKS",
                "imageId": 225648
              },
              "venueInfo": {
                "ground": "Rajiv Gandhi International Stadium",
                "city": "Hyderabad",
                "timezone": "+05:30"
              },
              "currBatTeamId": 255,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 247,
                  "wickets": 2,
                  "overs": 18.3
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 245,
                  "wickets": 6,
                  "overs": 20
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Sun, 13 Apr 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115138,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "28th Match",
              "matchFormat": "T20",
              "startDate": "1744538400000",
              "endDate": "1744538400000",
              "state": "complete",
              "status": "Royal Challengers Bengaluru won by 9 wkts",
              "team1": {
                "teamId": 64,
                "teamName": "RAJASTHAN ROYALS",
                "teamSName": "RR",
                "imageId": 225647
              },
              "team2": {
                "teamId": 59,
                "teamName": "ROYAL CHALLENGERS BENGALURU",
                "teamSName": "RCB",
                "imageId": 225643
              },
              "venueInfo": {
                "ground": "Sawai Mansingh Stadium",
                "city": "Jaipur",
                "timezone": "+05:30"
              },
              "currBatTeamId": 59,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 173,
                  "wickets": 4,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 175,
                  "wickets": 1,
                  "overs": 17.3
                }
              }
            }
          },
          {
            "matchInfo": {
              "matchId": 115140,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "29th Match",
              "matchFormat": "T20",
              "startDate": "1744552800000",
              "endDate": "1744552800000",
              "state": "complete",
              "status": "Mumbai Indians won by 12 runs",
              "team1": {
                "teamId": 61,
                "teamName": "DELHI CAPITALS",
                "teamSName": "DC",
                "imageId": 225644
              },
              "team2": {
                "teamId": 62,
                "teamName": "MUMBAI INDIANS",
                "teamSName": "MI",
                "imageId": 225645
              },
              "venueInfo": {
                "ground": "Arun Jaitley Stadium",
                "city": "Delhi",
                "timezone": "+05:30"
              },
              "currBatTeamId": 62,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 193,
                  "wickets": 10,
                  "overs": 19
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 205,
                  "wickets": 5,
                  "overs": 20
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Mon, 14 Apr 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115149,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "30th Match",
              "matchFormat": "T20",
              "startDate": "1744639200000",
              "endDate": "1744639200000",
              "state": "complete",
              "status": "Chennai Super Kings won by 5 wkts",
              "team1": {
                "teamId": 966,
                "teamName": "LUCKNOW SUPER GIANTS",
                "teamSName": "LSG",
                "imageId": 389444
              },
              "team2": {
                "teamId": 58,
                "teamName": "CHENNAI SUPER KINGS",
                "teamSName": "CSK",
                "imageId": 225641
              },
              "venueInfo": {
                "ground": "Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium",
                "city": "Lucknow",
                "timezone": "+05:30"
              },
              "currBatTeamId": 58,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 166,
                  "wickets": 7,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 168,
                  "wickets": 5,
                  "overs": 19.3
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Tue, 15 Apr 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115156,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "31st Match",
              "matchFormat": "T20",
              "startDate": "1744725600000",
              "endDate": "1744725600000",
              "state": "complete",
              "status": "Punjab Kings won by 16 runs",
              "team1": {
                "teamId": 65,
                "teamName": "PUNJAB KINGS",
                "teamSName": "PBKS",
                "imageId": 225648
              },
              "team2": {
                "teamId": 63,
                "teamName": "KOLKATA KNIGHT RIDERS",
                "teamSName": "KKR",
                "imageId": 225646
              },
              "venueInfo": {
                "ground": "Maharaja Yadavindra Singh International Cricket Stadium, Mullanpur",
                "city": "New Chandigarh",
                "timezone": "+05:30"
              },
              "currBatTeamId": 65,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 111,
                  "wickets": 10,
                  "overs": 15.3
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 95,
                  "wickets": 10,
                  "overs": 15.1
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Wed, 16 Apr 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115165,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "32nd Match",
              "matchFormat": "T20",
              "startDate": "1744812000000",
              "endDate": "1744812000000",
              "state": "complete",
              "status": "Match tied (Delhi Capitals won the Super Over)",
              "team1": {
                "teamId": 61,
                "teamName": "DELHI CAPITALS",
                "teamSName": "DC",
                "imageId": 225644
              },
              "team2": {
                "teamId": 64,
                "teamName": "RAJASTHAN ROYALS",
                "teamSName": "RR",
                "imageId": 225647
              },
              "venueInfo": {
                "ground": "Arun Jaitley Stadium",
                "city": "Delhi",
                "timezone": "+05:30"
              },
              "currBatTeamId": 64,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 188,
                  "wickets": 5,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 188,
                  "wickets": 4,
                  "overs": 20
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Thu, 17 Apr 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115167,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "33rd Match",
              "matchFormat": "T20",
              "startDate": "1744898400000",
              "endDate": "1744898400000",
              "state": "complete",
              "status": "Mumbai Indians won by 4 wkts",
              "team1": {
                "teamId": 62,
                "teamName": "MUMBAI INDIANS",
                "teamSName": "MI",
                "imageId": 225645
              },
              "team2": {
                "teamId": 255,
                "teamName": "SUNRISERS HYDERABAD",
                "teamSName": "SRH",
                "imageId": 225649
              },
              "venueInfo": {
                "ground": "Wankhede Stadium",
                "city": "Mumbai",
                "timezone": "+05:30"
              },
              "currBatTeamId": 62,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 166,
                  "wickets": 6,
                  "overs": 18.1
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 162,
                  "wickets": 5,
                  "overs": 20
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Fri, 18 Apr 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115174,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "34th Match",
              "matchFormat": "T20",
              "startDate": "1744984800000",
              "endDate": "1744984800000",
              "state": "complete",
              "status": "Punjab Kings won by 5 wkts - 14 overs game - due to rain",
              "team1": {
                "teamId": 59,
                "teamName": "ROYAL CHALLENGERS BENGALURU",
                "teamSName": "RCB",
                "imageId": 225643
              },
              "team2": {
                "teamId": 65,
                "teamName": "PUNJAB KINGS",
                "teamSName": "PBKS",
                "imageId": 225648
              },
              "venueInfo": {
                "ground": "M.Chinnaswamy Stadium",
                "city": "Bengaluru",
                "timezone": "+05:30"
              },
              "currBatTeamId": 65,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 95,
                  "wickets": 9,
                  "overs": 14
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 98,
                  "wickets": 5,
                  "overs": 12.1
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Sat, 19 Apr 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115176,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "35th Match",
              "matchFormat": "T20",
              "startDate": "1745056800000",
              "endDate": "1745056800000",
              "state": "complete",
              "status": "Gujarat Titans won by 7 wkts",
              "team1": {
                "teamId": 971,
                "teamName": "GUJARAT TITANS",
                "teamSName": "GT",
                "imageId": 235085
              },
              "team2": {
                "teamId": 61,
                "teamName": "DELHI CAPITALS",
                "teamSName": "DC",
                "imageId": 225644
              },
              "venueInfo": {
                "ground": "Narendra Modi Stadium",
                "city": "Ahmedabad",
                "timezone": "+05:30"
              },
              "currBatTeamId": 971,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 204,
                  "wickets": 3,
                  "overs": 19.2
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 203,
                  "wickets": 8,
                  "overs": 20
                }
              }
            }
          },
          {
            "matchInfo": {
              "matchId": 115183,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "36th Match",
              "matchFormat": "T20",
              "startDate": "1745071200000",
              "endDate": "1745071200000",
              "state": "complete",
              "status": "Lucknow Super Giants won by 2 runs",
              "team1": {
                "teamId": 64,
                "teamName": "RAJASTHAN ROYALS",
                "teamSName": "RR",
                "imageId": 225647
              },
              "team2": {
                "teamId": 966,
                "teamName": "LUCKNOW SUPER GIANTS",
                "teamSName": "LSG",
                "imageId": 389444
              },
              "venueInfo": {
                "ground": "Sawai Mansingh Stadium",
                "city": "Jaipur",
                "timezone": "+05:30"
              },
              "currBatTeamId": 966,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 178,
                  "wickets": 5,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 180,
                  "wickets": 5,
                  "overs": 20
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Sun, 20 Apr 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115192,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "37th Match",
              "matchFormat": "T20",
              "startDate": "1745143200000",
              "endDate": "1745143200000",
              "state": "complete",
              "status": "Royal Challengers Bengaluru won by 7 wkts",
              "team1": {
                "teamId": 65,
                "teamName": "PUNJAB KINGS",
                "teamSName": "PBKS",
                "imageId": 225648
              },
              "team2": {
                "teamId": 59,
                "teamName": "ROYAL CHALLENGERS BENGALURU",
                "teamSName": "RCB",
                "imageId": 225643
              },
              "venueInfo": {
                "ground": "Maharaja Yadavindra Singh International Cricket Stadium, Mullanpur",
                "city": "New Chandigarh",
                "timezone": "+05:30"
              },
              "currBatTeamId": 59,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 157,
                  "wickets": 6,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 159,
                  "wickets": 3,
                  "overs": 18.5
                }
              }
            }
          },
          {
            "matchInfo": {
              "matchId": 115201,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "38th Match",
              "matchFormat": "T20",
              "startDate": "1745157600000",
              "endDate": "1745157600000",
              "state": "complete",
              "status": "Mumbai Indians won by 9 wkts",
              "team1": {
                "teamId": 62,
                "teamName": "MUMBAI INDIANS",
                "teamSName": "MI",
                "imageId": 225645
              },
              "team2": {
                "teamId": 58,
                "teamName": "CHENNAI SUPER KINGS",
                "teamSName": "CSK",
                "imageId": 225641
              },
              "venueInfo": {
                "ground": "Wankhede Stadium",
                "city": "Mumbai",
                "timezone": "+05:30"
              },
              "currBatTeamId": 62,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 177,
                  "wickets": 1,
                  "overs": 15.4
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 176,
                  "wickets": 5,
                  "overs": 20
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Mon, 21 Apr 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115210,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "39th Match",
              "matchFormat": "T20",
              "startDate": "1745244000000",
              "endDate": "1745244000000",
              "state": "complete",
              "status": "Gujarat Titans won by 39 runs",
              "team1": {
                "teamId": 63,
                "teamName": "KOLKATA KNIGHT RIDERS",
                "teamSName": "KKR",
                "imageId": 225646
              },
              "team2": {
                "teamId": 971,
                "teamName": "GUJARAT TITANS",
                "teamSName": "GT",
                "imageId": 235085
              },
              "venueInfo": {
                "ground": "Eden Gardens",
                "city": "Kolkata",
                "timezone": "+05:30"
              },
              "currBatTeamId": 971,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 159,
                  "wickets": 8,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 198,
                  "wickets": 3,
                  "overs": 20
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Tue, 22 Apr 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115212,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "40th Match",
              "matchFormat": "T20",
              "startDate": "1745330400000",
              "endDate": "1745330400000",
              "state": "complete",
              "status": "Delhi Capitals won by 8 wkts",
              "team1": {
                "teamId": 966,
                "teamName": "LUCKNOW SUPER GIANTS",
                "teamSName": "LSG",
                "imageId": 389444
              },
              "team2": {
                "teamId": 61,
                "teamName": "DELHI CAPITALS",
                "teamSName": "DC",
                "imageId": 225644
              },
              "venueInfo": {
                "ground": "Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium",
                "city": "Lucknow",
                "timezone": "+05:30"
              },
              "currBatTeamId": 61,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 159,
                  "wickets": 6,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 161,
                  "wickets": 2,
                  "overs": 17.5
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Wed, 23 Apr 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115221,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "41st Match",
              "matchFormat": "T20",
              "startDate": "1745416800000",
              "endDate": "1745416800000",
              "state": "complete",
              "status": "Mumbai Indians won by 7 wkts",
              "team1": {
                "teamId": 255,
                "teamName": "SUNRISERS HYDERABAD",
                "teamSName": "SRH",
                "imageId": 225649
              },
              "team2": {
                "teamId": 62,
                "teamName": "MUMBAI INDIANS",
                "teamSName": "MI",
                "imageId": 225645
              },
              "venueInfo": {
                "ground": "Rajiv Gandhi International Stadium",
                "city": "Hyderabad",
                "timezone": "+05:30"
              },
              "currBatTeamId": 62,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 143,
                  "wickets": 8,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 146,
                  "wickets": 3,
                  "overs": 15.4
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Thu, 24 Apr 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115230,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "42nd Match",
              "matchFormat": "T20",
              "startDate": "1745503200000",
              "endDate": "1745503200000",
              "state": "complete",
              "status": "Royal Challengers Bengaluru won by 11 runs",
              "team1": {
                "teamId": 59,
                "teamName": "ROYAL CHALLENGERS BENGALURU",
                "teamSName": "RCB",
                "imageId": 225643
              },
              "team2": {
                "teamId": 64,
                "teamName": "RAJASTHAN ROYALS",
                "teamSName": "RR",
                "imageId": 225647
              },
              "venueInfo": {
                "ground": "M.Chinnaswamy Stadium",
                "city": "Bengaluru",
                "timezone": "+05:30"
              },
              "currBatTeamId": 59,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 205,
                  "wickets": 5,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 194,
                  "wickets": 9,
                  "overs": 20
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Fri, 25 Apr 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115239,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "43rd Match",
              "matchFormat": "T20",
              "startDate": "1745589600000",
              "endDate": "1745589600000",
              "state": "complete",
              "status": "Sunrisers Hyderabad won by 5 wkts",
              "team1": {
                "teamId": 58,
                "teamName": "CHENNAI SUPER KINGS",
                "teamSName": "CSK",
                "imageId": 225641
              },
              "team2": {
                "teamId": 255,
                "teamName": "SUNRISERS HYDERABAD",
                "teamSName": "SRH",
                "imageId": 225649
              },
              "venueInfo": {
                "ground": "MA Chidambaram Stadium",
                "city": "Chennai",
                "timezone": "+05:30"
              },
              "currBatTeamId": 255,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 154,
                  "wickets": 10,
                  "overs": 19.5
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 155,
                  "wickets": 5,
                  "overs": 18.4
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Sat, 26 Apr 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115248,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "44th Match",
              "matchFormat": "T20",
              "startDate": "1745676000000",
              "endDate": "1745676000000",
              "state": "complete",
              "status": "No result due to rain",
              "team1": {
                "teamId": 63,
                "teamName": "KOLKATA KNIGHT RIDERS",
                "teamSName": "KKR",
                "imageId": 225646
              },
              "team2": {
                "teamId": 65,
                "teamName": "PUNJAB KINGS",
                "teamSName": "PBKS",
                "imageId": 225648
              },
              "venueInfo": {
                "ground": "Eden Gardens",
                "city": "Kolkata",
                "timezone": "+05:30"
              },
              "currBatTeamId": 63,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 7,
                  "overs": 1
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 201,
                  "wickets": 4,
                  "overs": 20
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Sun, 27 Apr 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115255,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "45th Match",
              "matchFormat": "T20",
              "startDate": "1745748000000",
              "endDate": "1745748000000",
              "state": "complete",
              "status": "Mumbai Indians won by 54 runs",
              "team1": {
                "teamId": 62,
                "teamName": "MUMBAI INDIANS",
                "teamSName": "MI",
                "imageId": 225645
              },
              "team2": {
                "teamId": 966,
                "teamName": "LUCKNOW SUPER GIANTS",
                "teamSName": "LSG",
                "imageId": 389444
              },
              "venueInfo": {
                "ground": "Wankhede Stadium",
                "city": "Mumbai",
                "timezone": "+05:30"
              },
              "currBatTeamId": 62,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 215,
                  "wickets": 7,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 161,
                  "wickets": 10,
                  "overs": 20
                }
              }
            }
          },
          {
            "matchInfo": {
              "matchId": 115257,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "46th Match",
              "matchFormat": "T20",
              "startDate": "1745762400000",
              "endDate": "1745762400000",
              "state": "complete",
              "status": "Royal Challengers Bengaluru won by 6 wkts",
              "team1": {
                "teamId": 61,
                "teamName": "DELHI CAPITALS",
                "teamSName": "DC",
                "imageId": 225644
              },
              "team2": {
                "teamId": 59,
                "teamName": "ROYAL CHALLENGERS BENGALURU",
                "teamSName": "RCB",
                "imageId": 225643
              },
              "venueInfo": {
                "ground": "Arun Jaitley Stadium",
                "city": "Delhi",
                "timezone": "+05:30"
              },
              "currBatTeamId": 59,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 162,
                  "wickets": 8,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 165,
                  "wickets": 4,
                  "overs": 18.3
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Mon, 28 Apr 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115266,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "47th Match",
              "matchFormat": "T20",
              "startDate": "1745848800000",
              "endDate": "1745848800000",
              "state": "complete",
              "status": "Rajasthan Royals won by 8 wkts",
              "team1": {
                "teamId": 64,
                "teamName": "RAJASTHAN ROYALS",
                "teamSName": "RR",
                "imageId": 225647
              },
              "team2": {
                "teamId": 971,
                "teamName": "GUJARAT TITANS",
                "teamSName": "GT",
                "imageId": 235085
              },
              "venueInfo": {
                "ground": "Sawai Mansingh Stadium",
                "city": "Jaipur",
                "timezone": "+05:30"
              },
              "currBatTeamId": 64,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 212,
                  "wickets": 2,
                  "overs": 15.5
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 209,
                  "wickets": 4,
                  "overs": 20
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Tue, 29 Apr 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115275,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "48th Match",
              "matchFormat": "T20",
              "startDate": "1745935200000",
              "endDate": "1745935200000",
              "state": "complete",
              "status": "Kolkata Knight Riders won by 14 runs",
              "team1": {
                "teamId": 61,
                "teamName": "DELHI CAPITALS",
                "teamSName": "DC",
                "imageId": 225644
              },
              "team2": {
                "teamId": 63,
                "teamName": "KOLKATA KNIGHT RIDERS",
                "teamSName": "KKR",
                "imageId": 225646
              },
              "venueInfo": {
                "ground": "Arun Jaitley Stadium",
                "city": "Delhi",
                "timezone": "+05:30"
              },
              "currBatTeamId": 63,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 190,
                  "wickets": 9,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 204,
                  "wickets": 9,
                  "overs": 20
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Wed, 30 Apr 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115282,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "49th Match",
              "matchFormat": "T20",
              "startDate": "1746021600000",
              "endDate": "1746021600000",
              "state": "complete",
              "status": "Punjab Kings won by 4 wkts",
              "team1": {
                "teamId": 58,
                "teamName": "CHENNAI SUPER KINGS",
                "teamSName": "CSK",
                "imageId": 225641
              },
              "team2": {
                "teamId": 65,
                "teamName": "PUNJAB KINGS",
                "teamSName": "PBKS",
                "imageId": 225648
              },
              "venueInfo": {
                "ground": "MA Chidambaram Stadium",
                "city": "Chennai",
                "timezone": "+05:30"
              },
              "currBatTeamId": 65,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 190,
                  "wickets": 10,
                  "overs": 19.2
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 194,
                  "wickets": 6,
                  "overs": 19.4
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Thu, 01 May 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115291,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "50th Match",
              "matchFormat": "T20",
              "startDate": "1746108000000",
              "endDate": "1746108000000",
              "state": "complete",
              "status": "Mumbai Indians won by 100 runs",
              "team1": {
                "teamId": 64,
                "teamName": "RAJASTHAN ROYALS",
                "teamSName": "RR",
                "imageId": 225647
              },
              "team2": {
                "teamId": 62,
                "teamName": "MUMBAI INDIANS",
                "teamSName": "MI",
                "imageId": 225645
              },
              "venueInfo": {
                "ground": "Sawai Mansingh Stadium",
                "city": "Jaipur",
                "timezone": "+05:30"
              },
              "currBatTeamId": 62,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 117,
                  "wickets": 10,
                  "overs": 16.1
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 217,
                  "wickets": 2,
                  "overs": 20
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Fri, 02 May 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115300,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "51st Match",
              "matchFormat": "T20",
              "startDate": "1746194400000",
              "endDate": "1746194400000",
              "state": "complete",
              "status": "Gujarat Titans won by 38 runs",
              "team1": {
                "teamId": 971,
                "teamName": "GUJARAT TITANS",
                "teamSName": "GT",
                "imageId": 235085
              },
              "team2": {
                "teamId": 255,
                "teamName": "SUNRISERS HYDERABAD",
                "teamSName": "SRH",
                "imageId": 225649
              },
              "venueInfo": {
                "ground": "Narendra Modi Stadium",
                "city": "Ahmedabad",
                "timezone": "+05:30"
              },
              "currBatTeamId": 971,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 224,
                  "wickets": 6,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 186,
                  "wickets": 6,
                  "overs": 20
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Sat, 03 May 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115302,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "52nd Match",
              "matchFormat": "T20",
              "startDate": "1746280800000",
              "endDate": "1746280800000",
              "state": "complete",
              "status": "Royal Challengers Bengaluru won by 2 runs",
              "team1": {
                "teamId": 59,
                "teamName": "ROYAL CHALLENGERS BENGALURU",
                "teamSName": "RCB",
                "imageId": 225643
              },
              "team2": {
                "teamId": 58,
                "teamName": "CHENNAI SUPER KINGS",
                "teamSName": "CSK",
                "imageId": 225641
              },
              "venueInfo": {
                "ground": "M.Chinnaswamy Stadium",
                "city": "Bengaluru",
                "timezone": "+05:30"
              },
              "currBatTeamId": 59,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 213,
                  "wickets": 5,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 211,
                  "wickets": 5,
                  "overs": 20
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Sun, 04 May 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115309,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "53rd Match",
              "matchFormat": "T20",
              "startDate": "1746352800000",
              "endDate": "1746352800000",
              "state": "complete",
              "status": "Kolkata Knight Riders won by 1 run",
              "team1": {
                "teamId": 63,
                "teamName": "KOLKATA KNIGHT RIDERS",
                "teamSName": "KKR",
                "imageId": 225646
              },
              "team2": {
                "teamId": 64,
                "teamName": "RAJASTHAN ROYALS",
                "teamSName": "RR",
                "imageId": 225647
              },
              "venueInfo": {
                "ground": "Eden Gardens",
                "city": "Kolkata",
                "timezone": "+05:30"
              },
              "currBatTeamId": 63,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 206,
                  "wickets": 4,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 205,
                  "wickets": 8,
                  "overs": 20
                }
              }
            }
          },
          {
            "matchInfo": {
              "matchId": 115318,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "54th Match",
              "matchFormat": "T20",
              "startDate": "1746367200000",
              "endDate": "1746367200000",
              "state": "complete",
              "status": "Punjab Kings won by 37 runs",
              "team1": {
                "teamId": 65,
                "teamName": "PUNJAB KINGS",
                "teamSName": "PBKS",
                "imageId": 225648
              },
              "team2": {
                "teamId": 966,
                "teamName": "LUCKNOW SUPER GIANTS",
                "teamSName": "LSG",
                "imageId": 389444
              },
              "venueInfo": {
                "ground": "Himachal Pradesh Cricket Association Stadium",
                "city": "Dharamsala",
                "timezone": "+05:30"
              },
              "currBatTeamId": 65,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 236,
                  "wickets": 5,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 199,
                  "wickets": 7,
                  "overs": 20
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Mon, 05 May 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115327,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "55th Match",
              "matchFormat": "T20",
              "startDate": "1746453600000",
              "endDate": "1746453600000",
              "state": "complete",
              "status": "No Result - due to rain",
              "team1": {
                "teamId": 255,
                "teamName": "SUNRISERS HYDERABAD",
                "teamSName": "SRH",
                "imageId": 225649
              },
              "team2": {
                "teamId": 61,
                "teamName": "DELHI CAPITALS",
                "teamSName": "DC",
                "imageId": 225644
              },
              "venueInfo": {
                "ground": "Rajiv Gandhi International Stadium",
                "city": "Hyderabad",
                "timezone": "+05:30"
              },
              "currBatTeamId": 61,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team2Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 133,
                  "wickets": 7,
                  "overs": 20
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Tue, 06 May 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115336,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "56th Match",
              "matchFormat": "T20",
              "startDate": "1746540000000",
              "endDate": "1746540000000",
              "state": "complete",
              "status": "Gujarat Titans won by 3 wkts by DLS Method (Target 147)",
              "team1": {
                "teamId": 62,
                "teamName": "MUMBAI INDIANS",
                "teamSName": "MI",
                "imageId": 225645
              },
              "team2": {
                "teamId": 971,
                "teamName": "GUJARAT TITANS",
                "teamSName": "GT",
                "imageId": 235085
              },
              "venueInfo": {
                "ground": "Wankhede Stadium",
                "city": "Mumbai",
                "timezone": "+05:30"
              },
              "currBatTeamId": 971,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 155,
                  "wickets": 8,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 147,
                  "wickets": 7,
                  "overs": 19
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Wed, 07 May 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 115345,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "57th Match",
              "matchFormat": "T20",
              "startDate": "1746626400000",
              "endDate": "1746626400000",
              "state": "complete",
              "status": "Chennai Super Kings won by 2 wkts",
              "team1": {
                "teamId": 63,
                "teamName": "KOLKATA KNIGHT RIDERS",
                "teamSName": "KKR",
                "imageId": 225646
              },
              "team2": {
                "teamId": 58,
                "teamName": "CHENNAI SUPER KINGS",
                "teamSName": "CSK",
                "imageId": 225641
              },
              "venueInfo": {
                "ground": "Eden Gardens",
                "city": "Kolkata",
                "timezone": "+05:30"
              },
              "currBatTeamId": 58,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 179,
                  "wickets": 6,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 183,
                  "wickets": 8,
                  "overs": 19.4
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Sat, 17 May 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 118811,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "58th Match",
              "matchFormat": "T20",
              "startDate": "1747490400000",
              "endDate": "1747490400000",
              "state": "complete",
              "status": "Match abandoned due to rain (no toss)",
              "team1": {
                "teamId": 59,
                "teamName": "ROYAL CHALLENGERS BENGALURU",
                "teamSName": "RCB",
                "imageId": 225643
              },
              "team2": {
                "teamId": 63,
                "teamName": "KOLKATA KNIGHT RIDERS",
                "teamSName": "KKR",
                "imageId": 225646
              },
              "venueInfo": {
                "ground": "M.Chinnaswamy Stadium",
                "city": "Bengaluru",
                "timezone": "+05:30"
              },
              "isTimeAnnounced": True
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Sun, 18 May 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 118820,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "59th Match",
              "matchFormat": "T20",
              "startDate": "1747562400000",
              "endDate": "1747562400000",
              "state": "complete",
              "status": "Punjab Kings won by 10 runs",
              "team1": {
                "teamId": 64,
                "teamName": "RAJASTHAN ROYALS",
                "teamSName": "RR",
                "imageId": 225647
              },
              "team2": {
                "teamId": 65,
                "teamName": "PUNJAB KINGS",
                "teamSName": "PBKS",
                "imageId": 225648
              },
              "venueInfo": {
                "ground": "Sawai Mansingh Stadium",
                "city": "Jaipur",
                "timezone": "+05:30"
              },
              "currBatTeamId": 65,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 209,
                  "wickets": 7,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 219,
                  "wickets": 5,
                  "overs": 20
                }
              }
            }
          },
          {
            "matchInfo": {
              "matchId": 118829,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "60th Match",
              "matchFormat": "T20",
              "startDate": "1747576800000",
              "endDate": "1747576800000",
              "state": "complete",
              "status": "Gujarat Titans won by 10 wkts",
              "team1": {
                "teamId": 61,
                "teamName": "DELHI CAPITALS",
                "teamSName": "DC",
                "imageId": 225644
              },
              "team2": {
                "teamId": 971,
                "teamName": "GUJARAT TITANS",
                "teamSName": "GT",
                "imageId": 235085
              },
              "venueInfo": {
                "ground": "Arun Jaitley Stadium",
                "city": "Delhi",
                "timezone": "+05:30"
              },
              "currBatTeamId": 971,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 199,
                  "wickets": 3,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 205,
                  "overs": 19
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Mon, 19 May 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 118838,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "61st Match",
              "matchFormat": "T20",
              "startDate": "1747663200000",
              "endDate": "1747663200000",
              "state": "complete",
              "status": "Sunrisers Hyderabad won by 6 wkts",
              "team1": {
                "teamId": 966,
                "teamName": "LUCKNOW SUPER GIANTS",
                "teamSName": "LSG",
                "imageId": 389444
              },
              "team2": {
                "teamId": 255,
                "teamName": "SUNRISERS HYDERABAD",
                "teamSName": "SRH",
                "imageId": 225649
              },
              "venueInfo": {
                "ground": "Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium",
                "city": "Lucknow",
                "timezone": "+05:30"
              },
              "currBatTeamId": 255,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 205,
                  "wickets": 7,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 206,
                  "wickets": 4,
                  "overs": 18.2
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Tue, 20 May 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 118847,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "62nd Match",
              "matchFormat": "T20",
              "startDate": "1747749600000",
              "endDate": "1747749600000",
              "state": "complete",
              "status": "Rajasthan Royals won by 6 wkts",
              "team1": {
                "teamId": 58,
                "teamName": "CHENNAI SUPER KINGS",
                "teamSName": "CSK",
                "imageId": 225641
              },
              "team2": {
                "teamId": 64,
                "teamName": "RAJASTHAN ROYALS",
                "teamSName": "RR",
                "imageId": 225647
              },
              "venueInfo": {
                "ground": "Arun Jaitley Stadium",
                "city": "Delhi",
                "timezone": "+05:30"
              },
              "currBatTeamId": 64,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 187,
                  "wickets": 8,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 188,
                  "wickets": 4,
                  "overs": 17.1
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Wed, 21 May 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 118853,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "63rd Match",
              "matchFormat": "T20",
              "startDate": "1747836000000",
              "endDate": "1747836000000",
              "state": "complete",
              "status": "Mumbai Indians won by 59 runs",
              "team1": {
                "teamId": 62,
                "teamName": "MUMBAI INDIANS",
                "teamSName": "MI",
                "imageId": 225645
              },
              "team2": {
                "teamId": 61,
                "teamName": "DELHI CAPITALS",
                "teamSName": "DC",
                "imageId": 225644
              },
              "venueInfo": {
                "ground": "Wankhede Stadium",
                "city": "Mumbai",
                "timezone": "+05:30"
              },
              "currBatTeamId": 62,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 180,
                  "wickets": 5,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 121,
                  "wickets": 10,
                  "overs": 18.2
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Thu, 22 May 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 118862,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "64th Match",
              "matchFormat": "T20",
              "startDate": "1747922400000",
              "endDate": "1747922400000",
              "state": "complete",
              "status": "Lucknow Super Giants won by 33 runs",
              "team1": {
                "teamId": 971,
                "teamName": "GUJARAT TITANS",
                "teamSName": "GT",
                "imageId": 235085
              },
              "team2": {
                "teamId": 966,
                "teamName": "LUCKNOW SUPER GIANTS",
                "teamSName": "LSG",
                "imageId": 389444
              },
              "venueInfo": {
                "ground": "Narendra Modi Stadium",
                "city": "Ahmedabad",
                "timezone": "+05:30"
              },
              "currBatTeamId": 966,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 202,
                  "wickets": 9,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 235,
                  "wickets": 2,
                  "overs": 20
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Fri, 23 May 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 118865,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "65th Match",
              "matchFormat": "T20",
              "startDate": "1748008800000",
              "endDate": "1748008800000",
              "state": "complete",
              "status": "Sunrisers Hyderabad won by 42 runs",
              "team1": {
                "teamId": 59,
                "teamName": "ROYAL CHALLENGERS BENGALURU",
                "teamSName": "RCB",
                "imageId": 225643
              },
              "team2": {
                "teamId": 255,
                "teamName": "SUNRISERS HYDERABAD",
                "teamSName": "SRH",
                "imageId": 225649
              },
              "venueInfo": {
                "ground": "Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium",
                "city": "Lucknow",
                "timezone": "+05:30"
              },
              "currBatTeamId": 255,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 189,
                  "wickets": 10,
                  "overs": 19.5
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 231,
                  "wickets": 6,
                  "overs": 20
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Sat, 24 May 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 118874,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "66th Match",
              "matchFormat": "T20",
              "startDate": "1748095200000",
              "endDate": "1748095200000",
              "state": "complete",
              "status": "Delhi Capitals won by 6 wkts",
              "team1": {
                "teamId": 65,
                "teamName": "PUNJAB KINGS",
                "teamSName": "PBKS",
                "imageId": 225648
              },
              "team2": {
                "teamId": 61,
                "teamName": "DELHI CAPITALS",
                "teamSName": "DC",
                "imageId": 225644
              },
              "venueInfo": {
                "ground": "Sawai Mansingh Stadium",
                "city": "Jaipur",
                "timezone": "+05:30"
              },
              "currBatTeamId": 61,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 206,
                  "wickets": 8,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 208,
                  "wickets": 4,
                  "overs": 19.3
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Sun, 25 May 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 118880,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "67th Match",
              "matchFormat": "T20",
              "startDate": "1748167200000",
              "endDate": "1748167200000",
              "state": "complete",
              "status": "Chennai Super Kings won by 83 runs",
              "team1": {
                "teamId": 971,
                "teamName": "GUJARAT TITANS",
                "teamSName": "GT",
                "imageId": 235085
              },
              "team2": {
                "teamId": 58,
                "teamName": "CHENNAI SUPER KINGS",
                "teamSName": "CSK",
                "imageId": 225641
              },
              "venueInfo": {
                "ground": "Narendra Modi Stadium",
                "city": "Ahmedabad",
                "timezone": "+05:30"
              },
              "currBatTeamId": 58,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 147,
                  "wickets": 10,
                  "overs": 18.3
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 230,
                  "wickets": 5,
                  "overs": 20
                }
              }
            }
          },
          {
            "matchInfo": {
              "matchId": 118883,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "68th Match",
              "matchFormat": "T20",
              "startDate": "1748181600000",
              "endDate": "1748181600000",
              "state": "complete",
              "status": "Sunrisers Hyderabad won by 110 runs",
              "team1": {
                "teamId": 255,
                "teamName": "SUNRISERS HYDERABAD",
                "teamSName": "SRH",
                "imageId": 225649
              },
              "team2": {
                "teamId": 63,
                "teamName": "KOLKATA KNIGHT RIDERS",
                "teamSName": "KKR",
                "imageId": 225646
              },
              "venueInfo": {
                "ground": "Arun Jaitley Stadium",
                "city": "Delhi",
                "timezone": "+05:30"
              },
              "currBatTeamId": 255,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 278,
                  "wickets": 3,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 168,
                  "wickets": 10,
                  "overs": 18.4
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Mon, 26 May 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 118892,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "69th Match",
              "matchFormat": "T20",
              "startDate": "1748268000000",
              "endDate": "1748268000000",
              "state": "complete",
              "status": "Punjab Kings won by 7 wkts",
              "team1": {
                "teamId": 65,
                "teamName": "PUNJAB KINGS",
                "teamSName": "PBKS",
                "imageId": 225648
              },
              "team2": {
                "teamId": 62,
                "teamName": "MUMBAI INDIANS",
                "teamSName": "MI",
                "imageId": 225645
              },
              "venueInfo": {
                "ground": "Sawai Mansingh Stadium",
                "city": "Jaipur",
                "timezone": "+05:30"
              },
              "currBatTeamId": 65,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 187,
                  "wickets": 3,
                  "overs": 18.3
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 184,
                  "wickets": 7,
                  "overs": 20
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Tue, 27 May 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 118898,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "70th Match",
              "matchFormat": "T20",
              "startDate": "1748354400000",
              "endDate": "1748354400000",
              "state": "complete",
              "status": "Royal Challengers Bengaluru won by 6 wkts",
              "team1": {
                "teamId": 966,
                "teamName": "LUCKNOW SUPER GIANTS",
                "teamSName": "LSG",
                "imageId": 389444
              },
              "team2": {
                "teamId": 59,
                "teamName": "ROYAL CHALLENGERS BENGALURU",
                "teamSName": "RCB",
                "imageId": 225643
              },
              "venueInfo": {
                "ground": "Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium",
                "city": "Lucknow",
                "timezone": "+05:30"
              },
              "currBatTeamId": 59,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 227,
                  "wickets": 3,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 230,
                  "wickets": 4,
                  "overs": 18.4
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Thu, 29 May 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 118907,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "Qualifier 1",
              "matchFormat": "T20",
              "startDate": "1748527200000",
              "endDate": "1748527200000",
              "state": "complete",
              "status": "Royal Challengers Bengaluru won by 8 wkts",
              "team1": {
                "teamId": 65,
                "teamName": "PUNJAB KINGS",
                "teamSName": "PBKS",
                "imageId": 225648
              },
              "team2": {
                "teamId": 59,
                "teamName": "ROYAL CHALLENGERS BENGALURU",
                "teamSName": "RCB",
                "imageId": 225643
              },
              "venueInfo": {
                "ground": "Maharaja Yadavindra Singh International Cricket Stadium, Mullanpur",
                "city": "New Chandigarh",
                "timezone": "+05:30"
              },
              "currBatTeamId": 59,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 101,
                  "wickets": 10,
                  "overs": 14.1
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 106,
                  "wickets": 2,
                  "overs": 10
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Fri, 30 May 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 118916,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "Eliminator",
              "matchFormat": "T20",
              "startDate": "1748613600000",
              "endDate": "1748613600000",
              "state": "complete",
              "status": "Mumbai Indians won by 20 runs",
              "team1": {
                "teamId": 971,
                "teamName": "GUJARAT TITANS",
                "teamSName": "GT",
                "imageId": 235085
              },
              "team2": {
                "teamId": 62,
                "teamName": "MUMBAI INDIANS",
                "teamSName": "MI",
                "imageId": 225645
              },
              "venueInfo": {
                "ground": "Maharaja Yadavindra Singh International Cricket Stadium, Mullanpur",
                "city": "New Chandigarh",
                "timezone": "+05:30"
              },
              "currBatTeamId": 62,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 208,
                  "wickets": 6,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 228,
                  "wickets": 5,
                  "overs": 20
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Sun, 01 Jun 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 118919,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "Qualifier 2",
              "matchFormat": "T20",
              "startDate": "1748786400000",
              "endDate": "1748786400000",
              "state": "complete",
              "status": "Punjab Kings won by 5 wkts",
              "team1": {
                "teamId": 65,
                "teamName": "PUNJAB KINGS",
                "teamSName": "PBKS",
                "imageId": 225648
              },
              "team2": {
                "teamId": 62,
                "teamName": "MUMBAI INDIANS",
                "teamSName": "MI",
                "imageId": 225645
              },
              "venueInfo": {
                "ground": "Narendra Modi Stadium",
                "city": "Ahmedabad",
                "timezone": "+05:30"
              },
              "currBatTeamId": 65,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 207,
                  "wickets": 5,
                  "overs": 19
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 203,
                  "wickets": 6,
                  "overs": 20
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    },
    {
      "matchDetailsMap": {
        "key": "Tue, 03 Jun 2025",
        "match": [
          {
            "matchInfo": {
              "matchId": 118928,
              "seriesId": 9237,
              "seriesName": "Indian Premier League 2025",
              "matchDesc": "Final",
              "matchFormat": "T20",
              "startDate": "1748959200000",
              "endDate": "1748959200000",
              "state": "complete",
              "status": "Royal Challengers Bengaluru won by 6 runs",
              "team1": {
                "teamId": 59,
                "teamName": "ROYAL CHALLENGERS BENGALURU",
                "teamSName": "RCB",
                "imageId": 225643
              },
              "team2": {
                "teamId": 65,
                "teamName": "PUNJAB KINGS",
                "teamSName": "PBKS",
                "imageId": 225648
              },
              "venueInfo": {
                "ground": "Narendra Modi Stadium",
                "city": "Ahmedabad",
                "timezone": "+05:30"
              },
              "currBatTeamId": 59,
              "isTimeAnnounced": True
            },
            "matchScore": {
              "team1Score": {
                "inngs1": {
                  "inningsId": 1,
                  "runs": 190,
                  "wickets": 9,
                  "overs": 20
                }
              },
              "team2Score": {
                "inngs1": {
                  "inningsId": 2,
                  "runs": 184,
                  "wickets": 7,
                  "overs": 20
                }
              }
            }
          }
        ],
        "seriesId": 9237
      }
    }
  ]
}


### India vs SA Test Game Data

In [ ]:
data = {
  "scorecard": [
    {
      "inningsid": 1,
      "batsman": [
        {
          "id": 9582,
          "balls": 48,
          "runs": 31,
          "fours": 5,
          "sixes": 1,
          "strkrate": "64.58",
          "name": "Markram",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "c Pant b Bumrah",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 13070,
          "balls": 22,
          "runs": 23,
          "fours": 4,
          "sixes": 0,
          "strkrate": "104.55",
          "name": "Rickelton",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "b Bumrah",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 11200,
          "balls": 51,
          "runs": 24,
          "fours": 3,
          "sixes": 0,
          "strkrate": "47.06",
          "name": "Mulder",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "lbw b Kuldeep Yadav",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 8583,
          "balls": 11,
          "runs": 3,
          "fours": 0,
          "sixes": 0,
          "strkrate": "27.27",
          "name": "Bavuma",
          "nickname": "",
          "iscaptain": True,
          "iskeeper": False,
          "outdec": "c Dhruv Jurel b Kuldeep Yadav",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 11196,
          "balls": 55,
          "runs": 24,
          "fours": 1,
          "sixes": 1,
          "strkrate": "43.64",
          "name": "Tony de Zorzi",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "lbw b Bumrah",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 19243,
          "balls": 74,
          "runs": 15,
          "fours": 1,
          "sixes": 0,
          "strkrate": "20.27",
          "name": "Stubbs",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "not out",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 11209,
          "balls": 36,
          "runs": 16,
          "fours": 2,
          "sixes": 0,
          "strkrate": "44.44",
          "name": "Verreynne",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": True,
          "outdec": "lbw b Siraj",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 14565,
          "balls": 3,
          "runs": 0,
          "fours": 0,
          "sixes": 0,
          "strkrate": "0",
          "name": "Marco Jansen",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "b Siraj",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 9576,
          "balls": 23,
          "runs": 3,
          "fours": 0,
          "sixes": 0,
          "strkrate": "13.04",
          "name": "Corbin Bosch",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "lbw b Axar",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 9540,
          "balls": 5,
          "runs": 5,
          "fours": 1,
          "sixes": 0,
          "strkrate": "100",
          "name": "Harmer",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "b Bumrah",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 9720,
          "balls": 3,
          "runs": 0,
          "fours": 0,
          "sixes": 0,
          "strkrate": "0",
          "name": "Maharaj",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "lbw b Bumrah",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        }
      ],
      "bowler": [
        {
          "id": 9311,
          "overs": "14",
          "maidens": 5,
          "wickets": 5,
          "runs": 27,
          "economy": "1.9",
          "name": "Bumrah",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "dots": 5,
          "balls": 140,
          "rpb": 0.19,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 10808,
          "overs": "12",
          "maidens": 0,
          "wickets": 2,
          "runs": 47,
          "economy": "3.9",
          "name": "Siraj",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "dots": 0,
          "balls": 120,
          "rpb": 0.39,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 8808,
          "overs": "6",
          "maidens": 2,
          "wickets": 1,
          "runs": 21,
          "economy": "3.5",
          "name": "Axar",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "dots": 2,
          "balls": 60,
          "rpb": 0.35,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 8292,
          "overs": "14",
          "maidens": 1,
          "wickets": 2,
          "runs": 36,
          "economy": "2.6",
          "name": "Kuldeep Yadav",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "dots": 1,
          "balls": 140,
          "rpb": 0.26,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 587,
          "overs": "8",
          "maidens": 2,
          "wickets": 0,
          "runs": 13,
          "economy": "1.6",
          "name": "Ravindra Jadeja",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "dots": 2,
          "balls": 80,
          "rpb": 0.16,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 10945,
          "overs": "1",
          "maidens": 0,
          "wickets": 0,
          "runs": 3,
          "economy": "3",
          "name": "Washington Sundar",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "dots": 0,
          "balls": 10,
          "rpb": 0.3,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        }
      ],
      "fow": {
        "fow": [
          {
            "batsmanid": 13070,
            "batsmanname": "Rickelton",
            "overnbr": 10.3,
            "runs": 57,
            "ballnbr": 63
          },
          {
            "batsmanid": 9582,
            "batsmanname": "Markram",
            "overnbr": 12.1,
            "runs": 62,
            "ballnbr": 73
          },
          {
            "batsmanid": 8583,
            "batsmanname": "Bavuma",
            "overnbr": 15.6,
            "runs": 71,
            "ballnbr": 96
          },
          {
            "batsmanid": 11200,
            "batsmanname": "Mulder",
            "overnbr": 29.5,
            "runs": 114,
            "ballnbr": 179
          },
          {
            "batsmanid": 11196,
            "batsmanname": "Tony de Zorzi",
            "overnbr": 32.3,
            "runs": 120,
            "ballnbr": 195
          },
          {
            "batsmanid": 11209,
            "batsmanname": "Verreynne",
            "overnbr": 44.1,
            "runs": 146,
            "ballnbr": 265
          },
          {
            "batsmanid": 14565,
            "batsmanname": "Marco Jansen",
            "overnbr": 44.4,
            "runs": 147,
            "ballnbr": 268
          },
          {
            "batsmanid": 9576,
            "batsmanname": "Corbin Bosch",
            "overnbr": 51.6,
            "runs": 154,
            "ballnbr": 312
          },
          {
            "batsmanid": 9540,
            "batsmanname": "Harmer",
            "overnbr": 54.3,
            "runs": 159,
            "ballnbr": 327
          },
          {
            "batsmanid": 9720,
            "batsmanname": "Maharaj",
            "overnbr": 54.6,
            "runs": 159,
            "ballnbr": 330
          }
        ]
      },
      "extras": {
        "legbyes": 4,
        "byes": 8,
        "wides": 2,
        "noballs": 1,
        "penalty": 0,
        "total": 15
      },
      "score": 159,
      "wickets": 10,
      "overs": 55,
      "runrate": 2.89,
      "batteamname": "South Africa",
      "batteamsname": "RSA",
      "isdeclared": False,
      "isfollowon": False,
      "ballnbr": 330,
      "rpb": 0.48,
      "partnership": {
        "partnership": [
          {
            "id": 0,
            "bat1id": 9582,
            "bat1name": "Markram",
            "bat1runs": 26,
            "bat1fours": 4,
            "bat1sixes": 1,
            "bat2id": 13070,
            "bat2name": "Rickelton",
            "bat2runs": 23,
            "bat2fours": 4,
            "bat2sixes": 0,
            "totalruns": 57,
            "totalballs": 63,
            "bat1balls": 0,
            "bat2balls": 0,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 0,
            "bat1twos": 0,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 0,
            "bat1sixers": 0,
            "bat2ones": 0,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          },
          {
            "id": 0,
            "bat1id": 9582,
            "bat1name": "Markram",
            "bat1runs": 5,
            "bat1fours": 1,
            "bat1sixes": 0,
            "bat2id": 11200,
            "bat2name": "Mulder",
            "bat2runs": 0,
            "bat2fours": 0,
            "bat2sixes": 0,
            "totalruns": 5,
            "totalballs": 10,
            "bat1balls": 0,
            "bat2balls": 0,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 0,
            "bat1twos": 0,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 0,
            "bat1sixers": 0,
            "bat2ones": 0,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          },
          {
            "id": 0,
            "bat1id": 8583,
            "bat1name": "Bavuma",
            "bat1runs": 3,
            "bat1fours": 0,
            "bat1sixes": 0,
            "bat2id": 11200,
            "bat2name": "Mulder",
            "bat2runs": 6,
            "bat2fours": 1,
            "bat2sixes": 0,
            "totalruns": 9,
            "totalballs": 23,
            "bat1balls": 0,
            "bat2balls": 0,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 0,
            "bat1twos": 0,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 0,
            "bat1sixers": 0,
            "bat2ones": 0,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          },
          {
            "id": 0,
            "bat1id": 11196,
            "bat1name": "de Zorzi",
            "bat1runs": 22,
            "bat1fours": 1,
            "bat1sixes": 1,
            "bat2id": 11200,
            "bat2name": "Mulder",
            "bat2runs": 18,
            "bat2fours": 2,
            "bat2sixes": 0,
            "totalruns": 43,
            "totalballs": 83,
            "bat1balls": 0,
            "bat2balls": 0,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 0,
            "bat1twos": 0,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 0,
            "bat1sixers": 0,
            "bat2ones": 0,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          },
          {
            "id": 0,
            "bat1id": 11196,
            "bat1name": "de Zorzi",
            "bat1runs": 2,
            "bat1fours": 0,
            "bat1sixes": 0,
            "bat2id": 19243,
            "bat2name": "Stubbs",
            "bat2runs": 2,
            "bat2fours": 0,
            "bat2sixes": 0,
            "totalruns": 6,
            "totalballs": 16,
            "bat1balls": 0,
            "bat2balls": 0,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 0,
            "bat1twos": 0,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 0,
            "bat1sixers": 0,
            "bat2ones": 0,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          },
          {
            "id": 0,
            "bat1id": 11209,
            "bat1name": "Verreynne",
            "bat1runs": 16,
            "bat1fours": 2,
            "bat1sixes": 0,
            "bat2id": 19243,
            "bat2name": "Stubbs",
            "bat2runs": 9,
            "bat2fours": 1,
            "bat2sixes": 0,
            "totalruns": 26,
            "totalballs": 70,
            "bat1balls": 0,
            "bat2balls": 0,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 0,
            "bat1twos": 0,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 0,
            "bat1sixers": 0,
            "bat2ones": 0,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          },
          {
            "id": 0,
            "bat1id": 14565,
            "bat1name": "Marco Jansen",
            "bat1runs": 0,
            "bat1fours": 0,
            "bat1sixes": 0,
            "bat2id": 19243,
            "bat2name": "Stubbs",
            "bat2runs": 0,
            "bat2fours": 0,
            "bat2sixes": 0,
            "totalruns": 1,
            "totalballs": 3,
            "bat1balls": 0,
            "bat2balls": 0,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 0,
            "bat1twos": 0,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 0,
            "bat1sixers": 0,
            "bat2ones": 0,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          },
          {
            "id": 0,
            "bat1id": 9576,
            "bat1name": "Corbin Bosch",
            "bat1runs": 3,
            "bat1fours": 0,
            "bat1sixes": 0,
            "bat2id": 19243,
            "bat2name": "Stubbs",
            "bat2runs": 4,
            "bat2fours": 0,
            "bat2sixes": 0,
            "totalruns": 7,
            "totalballs": 44,
            "bat1balls": 0,
            "bat2balls": 0,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 0,
            "bat1twos": 0,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 0,
            "bat1sixers": 0,
            "bat2ones": 0,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          },
          {
            "id": 0,
            "bat1id": 9540,
            "bat1name": "Harmer",
            "bat1runs": 5,
            "bat1fours": 1,
            "bat1sixes": 0,
            "bat2id": 19243,
            "bat2name": "Stubbs",
            "bat2runs": 0,
            "bat2fours": 0,
            "bat2sixes": 0,
            "totalruns": 5,
            "totalballs": 15,
            "bat1balls": 0,
            "bat2balls": 0,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 0,
            "bat1twos": 0,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 0,
            "bat1sixers": 0,
            "bat2ones": 0,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          },
          {
            "id": 0,
            "bat1id": 9720,
            "bat1name": "Maharaj",
            "bat1runs": 0,
            "bat1fours": 0,
            "bat1sixes": 0,
            "bat2id": 19243,
            "bat2name": "Stubbs",
            "bat2runs": 0,
            "bat2fours": 0,
            "bat2sixes": 0,
            "totalruns": 0,
            "totalballs": 3,
            "bat1balls": 0,
            "bat2balls": 0,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 0,
            "bat1twos": 0,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 0,
            "bat1sixers": 0,
            "bat2ones": 0,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          }
        ]
      }
    },
    {
      "inningsid": 2,
      "batsman": [
        {
          "id": 13940,
          "balls": 27,
          "runs": 12,
          "fours": 3,
          "sixes": 0,
          "strkrate": "44.44",
          "name": "Yashasvi Jaiswal",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "b Marco Jansen",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 8733,
          "balls": 119,
          "runs": 39,
          "fours": 4,
          "sixes": 1,
          "strkrate": "32.77",
          "name": "Rahul",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "c Markram b Maharaj",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 10945,
          "balls": 82,
          "runs": 29,
          "fours": 2,
          "sixes": 1,
          "strkrate": "35.37",
          "name": "Washington Sundar",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "c Markram b Harmer",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 11808,
          "balls": 3,
          "runs": 4,
          "fours": 1,
          "sixes": 0,
          "strkrate": "133.33",
          "name": "Shubman Gill",
          "nickname": "",
          "iscaptain": True,
          "iskeeper": False,
          "outdec": "retd out",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 10744,
          "balls": 24,
          "runs": 27,
          "fours": 2,
          "sixes": 2,
          "strkrate": "112.5",
          "name": "Pant",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": True,
          "outdec": "c Verreynne b Corbin Bosch",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 587,
          "balls": 45,
          "runs": 27,
          "fours": 3,
          "sixes": 0,
          "strkrate": "60",
          "name": "Ravindra Jadeja",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "lbw b Harmer",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 14691,
          "balls": 14,
          "runs": 14,
          "fours": 3,
          "sixes": 0,
          "strkrate": "100",
          "name": "Dhruv Jurel",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "c and b Harmer",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 8808,
          "balls": 45,
          "runs": 16,
          "fours": 2,
          "sixes": 0,
          "strkrate": "35.56",
          "name": "Axar",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "c Marco Jansen b Harmer",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 8292,
          "balls": 8,
          "runs": 1,
          "fours": 0,
          "sixes": 0,
          "strkrate": "12.5",
          "name": "Kuldeep Yadav",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "c Verreynne b Marco Jansen",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 10808,
          "balls": 9,
          "runs": 1,
          "fours": 0,
          "sixes": 0,
          "strkrate": "11.11",
          "name": "Siraj",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "b Marco Jansen",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 9311,
          "balls": 3,
          "runs": 1,
          "fours": 0,
          "sixes": 0,
          "strkrate": "33.33",
          "name": "Bumrah",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "not out",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        }
      ],
      "bowler": [
        {
          "id": 14565,
          "overs": "15",
          "maidens": 4,
          "wickets": 3,
          "runs": 35,
          "economy": "2.3",
          "name": "Marco Jansen",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "dots": 4,
          "balls": 150,
          "rpb": 0.23,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 11200,
          "overs": "5",
          "maidens": 1,
          "wickets": 0,
          "runs": 15,
          "economy": "3",
          "name": "Mulder",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "dots": 1,
          "balls": 50,
          "rpb": 0.3,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 9720,
          "overs": "16",
          "maidens": 1,
          "wickets": 1,
          "runs": 66,
          "economy": "4.1",
          "name": "Maharaj",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "dots": 1,
          "balls": 160,
          "rpb": 0.41,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 9576,
          "overs": "11",
          "maidens": 4,
          "wickets": 1,
          "runs": 32,
          "economy": "2.9",
          "name": "Corbin Bosch",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "dots": 4,
          "balls": 110,
          "rpb": 0.29,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 9540,
          "overs": "15.2",
          "maidens": 4,
          "wickets": 4,
          "runs": 30,
          "economy": "2",
          "name": "Harmer",
          "nickname": "",
          "iscaptain": False,
          "iskeeper": False,
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "dots": 4,
          "balls": 152,
          "rpb": 0.2,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        }
      ],
      "fow": {
        "fow": [
          {
            "batsmanid": 13940,
            "batsmanname": "Yashasvi Jaiswal",
            "overnbr": 6.6,
            "runs": 18,
            "ballnbr": 42
          },
          {
            "batsmanid": 10945,
            "batsmanname": "Washington Sundar",
            "overnbr": 34.2,
            "runs": 75,
            "ballnbr": 206
          },
          {
            "batsmanid": 8733,
            "batsmanname": "Rahul",
            "overnbr": 39.5,
            "runs": 109,
            "ballnbr": 239
          },
          {
            "batsmanid": 10744,
            "batsmanname": "Pant",
            "overnbr": 43.4,
            "runs": 132,
            "ballnbr": 262
          },
          {
            "batsmanid": 14691,
            "batsmanname": "Dhruv Jurel",
            "overnbr": 48.3,
            "runs": 153,
            "ballnbr": 291
          },
          {
            "batsmanid": 587,
            "batsmanname": "Ravindra Jadeja",
            "overnbr": 54.3,
            "runs": 171,
            "ballnbr": 327
          },
          {
            "batsmanid": 8292,
            "batsmanname": "Kuldeep Yadav",
            "overnbr": 55.5,
            "runs": 172,
            "ballnbr": 335
          },
          {
            "batsmanid": 10808,
            "batsmanname": "Siraj",
            "overnbr": 59.6,
            "runs": 187,
            "ballnbr": 360
          },
          {
            "batsmanid": 8808,
            "batsmanname": "Axar",
            "overnbr": 62.2,
            "runs": 189,
            "ballnbr": 374
          }
        ]
      },
      "extras": {
        "legbyes": 2,
        "byes": 9,
        "wides": 2,
        "noballs": 5,
        "penalty": 0,
        "total": 18
      },
      "score": 189,
      "wickets": 10,
      "overs": 62.2,
      "runrate": 3.03,
      "batteamname": "India",
      "batteamsname": "IND",
      "isdeclared": False,
      "isfollowon": False,
      "ballnbr": 374,
      "rpb": 0.51,
      "partnership": {
        "partnership": [
          {
            "id": 0,
            "bat1id": 13940,
            "bat1name": "Jaiswal",
            "bat1runs": 12,
            "bat1fours": 3,
            "bat1sixes": 0,
            "bat2id": 8733,
            "bat2name": "Rahul",
            "bat2runs": 2,
            "bat2fours": 0,
            "bat2sixes": 0,
            "totalruns": 18,
            "totalballs": 42,
            "bat1balls": 0,
            "bat2balls": 0,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 0,
            "bat1twos": 0,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 0,
            "bat1sixers": 0,
            "bat2ones": 0,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          },
          {
            "id": 0,
            "bat1id": 10945,
            "bat1name": "Washington Sundar",
            "bat1runs": 29,
            "bat1fours": 2,
            "bat1sixes": 1,
            "bat2id": 8733,
            "bat2name": "Rahul",
            "bat2runs": 25,
            "bat2fours": 4,
            "bat2sixes": 0,
            "totalruns": 57,
            "totalballs": 164,
            "bat1balls": 0,
            "bat2balls": 0,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 0,
            "bat1twos": 0,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 0,
            "bat1sixers": 0,
            "bat2ones": 0,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          },
          {
            "id": 0,
            "bat1id": 11808,
            "bat1name": "Gill",
            "bat1runs": 4,
            "bat1fours": 1,
            "bat1sixes": 0,
            "bat2id": 8733,
            "bat2name": "Rahul",
            "bat2runs": 0,
            "bat2fours": 0,
            "bat2sixes": 0,
            "totalruns": 4,
            "totalballs": 3,
            "bat1balls": 0,
            "bat2balls": 0,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 0,
            "bat1twos": 0,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 0,
            "bat1sixers": 0,
            "bat2ones": 0,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          },
          {
            "id": 0,
            "bat1id": 10744,
            "bat1name": "Pant",
            "bat1runs": 14,
            "bat1fours": 1,
            "bat1sixes": 1,
            "bat2id": 8733,
            "bat2name": "Rahul",
            "bat2runs": 12,
            "bat2fours": 0,
            "bat2sixes": 1,
            "totalruns": 30,
            "totalballs": 30,
            "bat1balls": 0,
            "bat2balls": 0,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 0,
            "bat1twos": 0,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 0,
            "bat1sixers": 0,
            "bat2ones": 0,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          },
          {
            "id": 0,
            "bat1id": 10744,
            "bat1name": "Pant",
            "bat1runs": 13,
            "bat1fours": 1,
            "bat1sixes": 1,
            "bat2id": 587,
            "bat2name": "Jadeja",
            "bat2runs": 10,
            "bat2fours": 1,
            "bat2sixes": 0,
            "totalruns": 23,
            "totalballs": 23,
            "bat1balls": 0,
            "bat2balls": 0,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 0,
            "bat1twos": 0,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 0,
            "bat1sixers": 0,
            "bat2ones": 0,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          },
          {
            "id": 0,
            "bat1id": 14691,
            "bat1name": "Dhruv Jurel",
            "bat1runs": 14,
            "bat1fours": 3,
            "bat1sixes": 0,
            "bat2id": 587,
            "bat2name": "Jadeja",
            "bat2runs": 7,
            "bat2fours": 1,
            "bat2sixes": 0,
            "totalruns": 21,
            "totalballs": 29,
            "bat1balls": 0,
            "bat2balls": 0,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 0,
            "bat1twos": 0,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 0,
            "bat1sixers": 0,
            "bat2ones": 0,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          },
          {
            "id": 0,
            "bat1id": 8808,
            "bat1name": "Axar",
            "bat1runs": 2,
            "bat1fours": 0,
            "bat1sixes": 0,
            "bat2id": 587,
            "bat2name": "Jadeja",
            "bat2runs": 10,
            "bat2fours": 1,
            "bat2sixes": 0,
            "totalruns": 18,
            "totalballs": 36,
            "bat1balls": 0,
            "bat2balls": 0,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 0,
            "bat1twos": 0,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 0,
            "bat1sixers": 0,
            "bat2ones": 0,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          },
          {
            "id": 0,
            "bat1id": 8808,
            "bat1name": "Axar",
            "bat1runs": 0,
            "bat1fours": 0,
            "bat1sixes": 0,
            "bat2id": 8292,
            "bat2name": "Kuldeep Yadav",
            "bat2runs": 1,
            "bat2fours": 0,
            "bat2sixes": 0,
            "totalruns": 1,
            "totalballs": 8,
            "bat1balls": 0,
            "bat2balls": 0,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 0,
            "bat1twos": 0,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 0,
            "bat1sixers": 0,
            "bat2ones": 0,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          },
          {
            "id": 0,
            "bat1id": 8808,
            "bat1name": "Axar",
            "bat1runs": 13,
            "bat1fours": 2,
            "bat1sixes": 0,
            "bat2id": 10808,
            "bat2name": "Siraj",
            "bat2runs": 1,
            "bat2fours": 0,
            "bat2sixes": 0,
            "totalruns": 15,
            "totalballs": 25,
            "bat1balls": 0,
            "bat2balls": 0,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 0,
            "bat1twos": 0,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 0,
            "bat1sixers": 0,
            "bat2ones": 0,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          },
          {
            "id": 0,
            "bat1id": 8808,
            "bat1name": "Axar",
            "bat1runs": 1,
            "bat1fours": 0,
            "bat1sixes": 0,
            "bat2id": 9311,
            "bat2name": "Bumrah",
            "bat2runs": 1,
            "bat2fours": 0,
            "bat2sixes": 0,
            "totalruns": 2,
            "totalballs": 14,
            "bat1balls": 0,
            "bat2balls": 0,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 0,
            "bat1twos": 0,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 0,
            "bat1sixers": 0,
            "bat2ones": 0,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          }
        ]
      }
    },
    {
      "inningsid": 3,
      "batsman": [
        {
          "id": 13070,
          "balls": 23,
          "runs": 11,
          "fours": 1,
          "sixes": 0,
          "strkrate": "47.83",
          "name": "Ryan Rickelton",
          "nickname": "Rickelton",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "lbw b Kuldeep Yadav",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 9582,
          "balls": 23,
          "runs": 4,
          "fours": 0,
          "sixes": 0,
          "strkrate": "17.39",
          "name": "Aiden Markram",
          "nickname": "Markram",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "c Dhruv Jurel b Ravindra Jadeja",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 11200,
          "balls": 30,
          "runs": 11,
          "fours": 1,
          "sixes": 0,
          "strkrate": "36.67",
          "name": "Wiaan Mulder",
          "nickname": "Mulder",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "c Pant b Ravindra Jadeja",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 8583,
          "balls": 78,
          "runs": 29,
          "fours": 3,
          "sixes": 0,
          "strkrate": "37.18",
          "name": "Temba Bavuma",
          "nickname": "Bavuma",
          "iscaptain": True,
          "iskeeper": False,
          "outdec": "batting",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 11196,
          "balls": 2,
          "runs": 2,
          "fours": 0,
          "sixes": 0,
          "strkrate": "100",
          "name": "Tony de Zorzi",
          "nickname": "Tony de Zorzi",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "c Dhruv Jurel b Ravindra Jadeja",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 19243,
          "balls": 18,
          "runs": 5,
          "fours": 0,
          "sixes": 0,
          "strkrate": "27.78",
          "name": "Tristan Stubbs",
          "nickname": "Stubbs",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "b Ravindra Jadeja",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 11209,
          "balls": 16,
          "runs": 9,
          "fours": 1,
          "sixes": 0,
          "strkrate": "56.25",
          "name": "Kyle Verreynne",
          "nickname": "Verreynne",
          "iscaptain": False,
          "iskeeper": True,
          "outdec": "b Axar",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 14565,
          "balls": 16,
          "runs": 13,
          "fours": 0,
          "sixes": 1,
          "strkrate": "81.25",
          "name": "Marco Jansen",
          "nickname": "Marco Jansen",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "c Rahul b Kuldeep Yadav",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 9576,
          "balls": 4,
          "runs": 1,
          "fours": 0,
          "sixes": 0,
          "strkrate": "25",
          "name": "Corbin Bosch",
          "nickname": "Corbin Bosch",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "batting",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 9540,
          "balls": 0,
          "runs": 0,
          "fours": 0,
          "sixes": 0,
          "strkrate": "0",
          "name": "Simon Harmer",
          "nickname": "Harmer",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 9720,
          "balls": 0,
          "runs": 0,
          "fours": 0,
          "sixes": 0,
          "strkrate": "0",
          "name": "Keshav Maharaj",
          "nickname": "Maharaj",
          "iscaptain": False,
          "iskeeper": False,
          "outdec": "",
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "iscbplusfree": False,
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        }
      ],
      "bowler": [
        {
          "id": 9311,
          "overs": "6",
          "maidens": 1,
          "wickets": 0,
          "runs": 14,
          "economy": "2.3",
          "name": "Jasprit Bumrah",
          "nickname": "Bumrah",
          "iscaptain": False,
          "iskeeper": False,
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "dots": 1,
          "balls": 60,
          "rpb": 0.23,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 8808,
          "overs": "11",
          "maidens": 0,
          "wickets": 1,
          "runs": 30,
          "economy": "2.7",
          "name": "Axar Patel",
          "nickname": "Axar",
          "iscaptain": False,
          "iskeeper": False,
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "dots": 0,
          "balls": 110,
          "rpb": 0.27,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 8292,
          "overs": "5",
          "maidens": 1,
          "wickets": 2,
          "runs": 12,
          "economy": "2.4",
          "name": "Kuldeep Yadav",
          "nickname": "Kuldeep Yadav",
          "iscaptain": False,
          "iskeeper": False,
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "dots": 1,
          "balls": 50,
          "rpb": 0.24,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        },
        {
          "id": 587,
          "overs": "13",
          "maidens": 3,
          "wickets": 4,
          "runs": 29,
          "economy": "2.2",
          "name": "Ravindra Jadeja",
          "nickname": "Ravindra Jadeja",
          "iscaptain": False,
          "iskeeper": False,
          "videotype": "",
          "videourl": "",
          "videoid": 0,
          "dots": 3,
          "balls": 130,
          "rpb": 0.22,
          "planid": 0,
          "imageid": 0,
          "premiumvideourl": "",
          "ispremiumfree": False,
          "inmatchchange": "",
          "isoverseas": False,
          "playingxichange": ""
        }
      ],
      "fow": {
        "fow": [
          {
            "batsmanid": 13070,
            "batsmanname": "Ryan Rickelton",
            "overnbr": 6.4,
            "runs": 18,
            "ballnbr": 40
          },
          {
            "batsmanid": 9582,
            "batsmanname": "Aiden Markram",
            "overnbr": 8.2,
            "runs": 25,
            "ballnbr": 50
          },
          {
            "batsmanid": 11200,
            "batsmanname": "Wiaan Mulder",
            "overnbr": 16.3,
            "runs": 38,
            "ballnbr": 99
          },
          {
            "batsmanid": 11196,
            "batsmanname": "Tony de Zorzi",
            "overnbr": 16.5,
            "runs": 40,
            "ballnbr": 101
          },
          {
            "batsmanid": 19243,
            "batsmanname": "Tristan Stubbs",
            "overnbr": 22.6,
            "runs": 60,
            "ballnbr": 138
          },
          {
            "batsmanid": 11209,
            "batsmanname": "Kyle Verreynne",
            "overnbr": 27.5,
            "runs": 75,
            "ballnbr": 167
          },
          {
            "batsmanid": 14565,
            "batsmanname": "Marco Jansen",
            "overnbr": 33.6,
            "runs": 91,
            "ballnbr": 204
          }
        ]
      },
      "extras": {
        "legbyes": 8,
        "byes": 0,
        "wides": 0,
        "noballs": 0,
        "penalty": 0,
        "total": 8
      },
      "score": 93,
      "wickets": 7,
      "overs": 35,
      "runrate": 2.66,
      "batteamname": "South Africa",
      "batteamsname": "RSA",
      "isdeclared": False,
      "isfollowon": False,
      "ballnbr": 210,
      "rpb": 0.44,
      "partnership": {
        "partnership": [
          {
            "id": 0,
            "bat1id": 13070,
            "bat1name": "Rickelton",
            "bat1runs": 11,
            "bat1fours": 1,
            "bat1sixes": 0,
            "bat2id": 9582,
            "bat2name": "Markram",
            "bat2runs": 3,
            "bat2fours": 0,
            "bat2sixes": 0,
            "totalruns": 18,
            "totalballs": 40,
            "bat1balls": 23,
            "bat2balls": 17,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 5,
            "bat1twos": 1,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 1,
            "bat1sixers": 0,
            "bat2ones": 3,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          },
          {
            "id": 0,
            "bat1id": 11200,
            "bat1name": "Mulder",
            "bat1runs": 2,
            "bat1fours": 0,
            "bat1sixes": 0,
            "bat2id": 9582,
            "bat2name": "Markram",
            "bat2runs": 1,
            "bat2fours": 0,
            "bat2sixes": 0,
            "totalruns": 7,
            "totalballs": 10,
            "bat1balls": 4,
            "bat2balls": 6,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 2,
            "bat1twos": 0,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 0,
            "bat1sixers": 0,
            "bat2ones": 1,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          },
          {
            "id": 0,
            "bat1id": 11200,
            "bat1name": "Mulder",
            "bat1runs": 9,
            "bat1fours": 1,
            "bat1sixes": 0,
            "bat2id": 8583,
            "bat2name": "Bavuma",
            "bat2runs": 4,
            "bat2fours": 0,
            "bat2sixes": 0,
            "totalruns": 13,
            "totalballs": 49,
            "bat1balls": 26,
            "bat2balls": 23,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 5,
            "bat1twos": 0,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 1,
            "bat1sixers": 0,
            "bat2ones": 4,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          },
          {
            "id": 0,
            "bat1id": 11196,
            "bat1name": "de Zorzi",
            "bat1runs": 2,
            "bat1fours": 0,
            "bat1sixes": 0,
            "bat2id": 8583,
            "bat2name": "Bavuma",
            "bat2runs": 0,
            "bat2fours": 0,
            "bat2sixes": 0,
            "totalruns": 2,
            "totalballs": 2,
            "bat1balls": 2,
            "bat2balls": 0,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 0,
            "bat1twos": 1,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 0,
            "bat1sixers": 0,
            "bat2ones": 0,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          },
          {
            "id": 0,
            "bat1id": 19243,
            "bat1name": "Stubbs",
            "bat1runs": 5,
            "bat1fours": 0,
            "bat1sixes": 0,
            "bat2id": 8583,
            "bat2name": "Bavuma",
            "bat2runs": 15,
            "bat2fours": 3,
            "bat2sixes": 0,
            "totalruns": 20,
            "totalballs": 37,
            "bat1balls": 18,
            "bat2balls": 19,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 5,
            "bat1twos": 0,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 0,
            "bat1sixers": 0,
            "bat2ones": 3,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 3,
            "bat2sixers": 0
          },
          {
            "id": 0,
            "bat1id": 11209,
            "bat1name": "Verreynne",
            "bat1runs": 9,
            "bat1fours": 1,
            "bat1sixes": 0,
            "bat2id": 8583,
            "bat2name": "Bavuma",
            "bat2runs": 6,
            "bat2fours": 0,
            "bat2sixes": 0,
            "totalruns": 15,
            "totalballs": 29,
            "bat1balls": 16,
            "bat2balls": 13,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 3,
            "bat1twos": 1,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 1,
            "bat1sixers": 0,
            "bat2ones": 6,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          },
          {
            "id": 0,
            "bat1id": 14565,
            "bat1name": "Marco Jansen",
            "bat1runs": 13,
            "bat1fours": 0,
            "bat1sixes": 1,
            "bat2id": 8583,
            "bat2name": "Bavuma",
            "bat2runs": 3,
            "bat2fours": 0,
            "bat2sixes": 0,
            "totalruns": 16,
            "totalballs": 37,
            "bat1balls": 16,
            "bat2balls": 21,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 5,
            "bat1twos": 1,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 0,
            "bat1sixers": 1,
            "bat2ones": 3,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          },
          {
            "id": 0,
            "bat1id": 9576,
            "bat1name": "Corbin Bosch",
            "bat1runs": 1,
            "bat1fours": 0,
            "bat1sixes": 0,
            "bat2id": 8583,
            "bat2name": "Bavuma",
            "bat2runs": 1,
            "bat2fours": 0,
            "bat2sixes": 0,
            "totalruns": 2,
            "totalballs": 6,
            "bat1balls": 4,
            "bat2balls": 2,
            "teamname": "",
            "teamid": 0,
            "bat1ones": 1,
            "bat1twos": 0,
            "bat1threes": 0,
            "bat1fives": 0,
            "bat1boundaries": 0,
            "bat1sixers": 0,
            "bat2ones": 1,
            "bat2twos": 0,
            "bat2threes": 0,
            "bat2fives": 0,
            "bat2boundaries": 0,
            "bat2sixers": 0
          }
        ]
      }
    }
  ],
  "ismatchcomplete": False,
  "appindex": {
    "seotitle": "Cricket scorecard - IND vs RSA 1st Test,South Africa tour of India, 2025 | Cricbuzz.com",
    "weburl": "http://www.cricbuzz.com/live-cricket-scorecard/117371/ind-vs-rsa-1st-test-south-africa-tour-of-india-2025"
  },
  "status": "Day 2: Stumps - South Africa lead by 63 runs",
  "responselastupdated": 1763208282
}

### Rest of the code

In [ ]:
for key in data.keys():

In [ ]:
match = response.json()
scorecard = match['scorecard']
completion = match['ismatchcomplete']
url = match['appindex']['weburl']
status = match['status']

for i in scorecard:
    for j in i.keys():
        print(j,":",i[j])
    print()

inningsid : 1
batsman : [{'id': 12139, 'balls': 146, 'runs': 57, 'fours': 4, 'sixes': 0, 'strkrate': '39.04', 'name': 'Shafique', 'nickname': '', 'iscaptain': False, 'iskeeper': False, 'outdec': 'c Verreynne b Harmer', 'videotype': '', 'videourl': '', 'videoid': 0, 'planid': 0, 'imageid': 0, 'premiumvideourl': '', 'iscbplusfree': False, 'ispremiumfree': False, 'inmatchchange': '', 'isoverseas': False, 'playingxichange': ''}, {'id': 8364, 'balls': 35, 'runs': 17, 'fours': 2, 'sixes': 0, 'strkrate': '48.57', 'name': 'Imam-ul-Haq', 'nickname': '', 'iscaptain': False, 'iskeeper': False, 'outdec': 'b Harmer', 'videotype': '', 'videourl': '', 'videoid': 0, 'planid': 0, 'imageid': 0, 'premiumvideourl': '', 'iscbplusfree': False, 'ispremiumfree': False, 'inmatchchange': '', 'isoverseas': False, 'playingxichange': ''}, {'id': 9545, 'balls': 176, 'runs': 87, 'fours': 2, 'sixes': 3, 'strkrate': '49.43', 'name': 'Masood', 'nickname': '', 'iscaptain': True, 'iskeeper': False, 'outdec': 'c Marco Jan

In [10]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
import os
import dill
import pickle
from pathlib import Path
import difflib

names = ['Ruturaj Gaikwad', 'Andre Siddarth C', 'Shaik Rasheed', 'Rahul Tripathi', 'Ayush Mhatre', 'Dewald Brevis', 'Shivam Dube', 'Rachin Ravindra', 'Deepak Hooda', 'Vijay Shankar', 'Ramakrishna Ghosh', 'Ravindra Jadeja', 'Anshul Kamboj', 'Jamie Overton', 'Sam Curran', 'Ravichandran Ashwin', 'Devon Conway', 'MS Dhoni', 'Vansh Bedi', 'Urvil Patel', 'Kamlesh Nagarkoti', 'Shreyas Gopal', 'Matheesha Pathirana', 'Mukesh Choudhary', 'Nathan Ellis', 'Noor Ahmad', 'Khaleel Ahmed', 'Vaibhav Suryavanshi', 'Shimron Hetmyer', 'Yashasvi Jaiswal', 'Shubham Dubey', 'Riyan Parag', 'Wanindu Hasaranga', 'Sanju Samson', 'Dhruv Jurel', 'Kunal Singh Rathore', 'Lhuan-dre Pretorius', 'Yudhvir Singh Charak', 'Tushar Deshpande', 'Kumar Kartikeya', 'Akash Madhwal', 'Kwena Maphaka', 'Maheesh Theekshana', 'Fazalhaq Farooqi', 'Ashok Sharma', 'Jofra Archer', 'Nandre Burger', 'Manish Pandey', 'Ajinkya Rahane', 'Rinku Singh', 'Angkrish Raghuvanshi', 'Anukul Roy', 'Ramandeep Singh', 'Venkatesh Iyer', 'Moeen Ali', 'Sunil Narine', 'Andre Russell', 'Quinton de Kock', 'Rahmanullah Gurbaz', 'Luvnith Sisodia', 'Varun Chakaravarthy', 'Mayank Markande', 'Vaibhav Arora', 'Harshit Rana', 'Anrich Nortje', 'Spencer Johnson', 'Chetan Sakariya', 'Shivam Shukla', 'Atharva Taide', 'Travis Head', 'Abhinav Manohar', 'Sachin Baby', 'Aniket Verma', 'Nitish Kumar Reddy', 'Abhishek Sharma', 'Kamindu Mendis', 'Wiaan Mulder', 'Harsh Dubey', 'Heinrich Klaasen', 'Ishan Kishan', 'Zeeshan Ansari', 'Pat Cummins', 'Mohammed Shami', 'Harshal Patel', 'Rahul Chahar', 'Simarjeet Singh', 'Eshan Malinga', 'Jaydev Unadkat', 'Virat Kohli', 'Rajat Patidar', 'Swastik Chikara', 'Tim David', 'Mayank Agarwal', 'Krunal Pandya', 'Liam Livingstone', 'Manoj Bhandage', 'Romario Shepherd', 'Swapnil Singh', 'Mohit Rathee', 'Philip Salt', 'Jitesh Sharma', 'Tim Seifert', 'Josh Hazlewood', 'Bhuvneshwar Kumar', 'Rasikh Dar Salam', 'Suyash Sharma', 'Yash Dayal', 'Nuwan Thushara', 'Abhinandan Singh', 'Blessing Muzarabani', 'Faf du Plessis', 'Karun Nair', 'Sameer Rizvi', 'Sediqullah Atal', 'Ashutosh Sharma', 'Tripurana Vijay', 'Axar Patel', 'Darshan Nalkande', 'Ajay Jadav Mandal', 'Manvanth Kumar L', 'Madhav Tiwari', 'Tristan Stubbs', 'Abishek Porel', 'Donovan Ferreira', 'KL Rahul', 'Vipraj Nigam', 'Kuldeep Yadav', 'Dushmantha Chameera', 'Mohit Sharma', 'T Natarajan', 'Mukesh Kumar', 'Mustafizur Rahman', 'Nehal Wadhera', 'Harnoor Singh', 'Shreyas Iyer', 'Pyla Avinash', 'Priyansh Arya', 'Musheer Khan', 'Marcus Stoinis', 'Aaron Hardie', 'Suryansh Shedge', 'Shashank Singh', 'Mitchell Owen', 'Praveen Dubey', 'Azmatullah Omarzai', 'Prabhsimran Singh', 'Josh Inglis', 'Vishnu Vinod', 'Harpreet Brar', 'Arshdeep Singh', 'Yuzvendra Chahal', 'Vijaykumar Vyshak', 'Kuldeep Sen', 'Yash Thakur', 'Xavier Bartlett', 'Kyle Jamieson', 'Rohit Sharma', 'Suryakumar Yadav', 'Tilak Varma', 'Naman Dhir', 'Bevon Jacobs', 'Hardik Pandya', 'Raj Bawa', 'Charith Asalanka', 'Mitchell Santner', 'Arjun Tendulkar', 'Krishnan Shrijith', 'Robin Minz', 'Jonny Bairstow', 'Jasprit Bumrah', 'Ashwani Kumar', 'Reece Topley', 'Karn Sharma', 'Trent Boult', 'Satyanarayana Raju', 'Deepak Chahar', 'Mujeeb Ur Rahman', 'Raghu Sharma', 'Richard Gleeson', 'Sai Sudharsan', 'Shubman Gill', 'Shahrukh Khan', 'Rahul Tewatia', 'Nishant Sindhu', 'Sherfane Rutherford', 'Mahipal Lomror', 'Dasun Shanaka', 'Rashid Khan', 'Ravisrinivasan Sai Kishore', 'Arshad Khan', 'Jayant Yadav', 'Karim Janat', 'Washington Sundar', 'Kumar Kushagra', 'Anuj Rawat', 'Kusal Mendis', 'Gerald Coetzee', 'Manav Suthar', 'Gurnoor Brar', 'Ishant Sharma', 'Kulwant Khejroliya', 'Prasidh Krishna', 'Mohammed Siraj', 'Himmat Singh', 'David Miller', 'Aiden Markram', 'Ayush Badoni', 'Mitchell Marsh', 'Abdul Samad', 'Arshin Kulkarni', 'Yuvraj Chaudhary', 'Shahbaz Ahmed', 'RS Hangargekar', 'Shardul Thakur', 'Matthew Breetzke', 'Nicholas Pooran', 'Aryan Juyal', 'Rishabh Pant', 'Ravi Bishnoi', 'Akash Deep', 'Manimaran Siddharth', 'Shamar Joseph', 'Avesh Khan', 'Prince Yadav', 'Akash Maharaj Singh', 'Digvesh Singh Rathi', 'William ORourke']
roles = ['BAT', 'BAT', 'BAT', 'BAT', 'BAT', 'BAT', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'WK', 'WK', 'WK', 'WK', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BAT', 'BAT', 'BAT', 'BAT', 'AR', 'AR', 'WK', 'WK', 'WK', 'WK', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BAT', 'BAT', 'BAT', 'BAT', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'WK', 'WK', 'WK', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BAT', 'BAT', 'BAT', 'BAT', 'BAT', 'AR', 'AR', 'AR', 'AR', 'AR', 'WK', 'WK', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BAT', 'BAT', 'BAT', 'BAT', 'BAT', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'WK', 'WK', 'WK', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BAT', 'BAT', 'BAT', 'BAT', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'WK', 'WK', 'WK', 'WK', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BAT', 'BAT', 'BAT', 'BAT', 'BAT', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'WK', 'WK', 'WK', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BAT', 'BAT', 'BAT', 'BAT', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'WK', 'WK', 'WK', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BAT', 'BAT', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'WK', 'WK', 'WK', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BAT', 'BAT', 'BAT', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'WK', 'WK', 'WK', 'WK', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL']
squads = {'Chennai Super Kings': ['Ruturaj Gaikwad', 'Andre Siddarth C', 'Shaik Rasheed', 'Rahul Tripathi', 'Ayush Mhatre', 'Dewald Brevis', 'Shivam Dube', 'Rachin Ravindra', 'Deepak Hooda', 'Vijay Shankar', 'Ramakrishna Ghosh', 'Ravindra Jadeja', 'Anshul Kamboj', 'Jamie Overton', 'Sam Curran', 'Ravichandran Ashwin', 'Devon Conway', 'MS Dhoni', 'Vansh Bedi', 'Urvil Patel', 'Kamlesh Nagarkoti', 'Shreyas Gopal', 'Matheesha Pathirana', 'Mukesh Choudhary', 'Nathan Ellis', 'Noor Ahmad', 'Khaleel Ahmed'], 'Rajasthan Royals': ['Vaibhav Suryavanshi', 'Shimron Hetmyer', 'Yashasvi Jaiswal', 'Shubham Dubey', 'Riyan Parag', 'Wanindu Hasaranga', 'Sanju Samson', 'Dhruv Jurel', 'Kunal Singh Rathore', 'Lhuan-dre Pretorius', 'Yudhvir Singh Charak', 'Tushar Deshpande', 'Kumar Kartikeya', 'Akash Madhwal', 'Kwena Maphaka', 'Maheesh Theekshana', 'Fazalhaq Farooqi', 'Ashok Sharma', 'Jofra Archer', 'Nandre Burger'], 'Kolkata Knight Riders': ['Manish Pandey', 'Ajinkya Rahane', 'Rinku Singh', 'Angkrish Raghuvanshi', 'Anukul Roy', 'Ramandeep Singh', 'Venkatesh Iyer', 'Moeen Ali', 'Sunil Narine', 'Andre Russell', 'Quinton de Kock', 'Rahmanullah Gurbaz', 'Luvnith Sisodia', 'Varun Chakaravarthy', 'Mayank Markande', 'Vaibhav Arora', 'Harshit Rana', 'Anrich Nortje', 'Spencer Johnson', 'Chetan Sakariya', 'Shivam Shukla'], 'Sunrisers Hyderabad': ['Atharva Taide', 'Travis Head', 'Abhinav Manohar', 'Sachin Baby', 'Aniket Verma', 'Nitish Kumar Reddy', 'Abhishek Sharma', 'Kamindu Mendis', 'Wiaan Mulder', 'Harsh Dubey', 'Heinrich Klaasen', 'Ishan Kishan', 'Zeeshan Ansari', 'Pat Cummins', 'Mohammed Shami', 'Harshal Patel', 'Rahul Chahar', 'Simarjeet Singh', 'Eshan Malinga', 'Jaydev Unadkat'], 'Royal Challengers Bengaluru': ['Virat Kohli', 'Rajat Patidar', 'Swastik Chikara', 'Tim David', 'Mayank Agarwal', 'Krunal Pandya', 'Liam Livingstone', 'Manoj Bhandage', 'Romario Shepherd', 'Swapnil Singh', 'Mohit Rathee', 'Philip Salt', 'Jitesh Sharma', 'Tim Seifert', 'Josh Hazlewood', 'Bhuvneshwar Kumar', 'Rasikh Dar Salam', 'Suyash Sharma', 'Yash Dayal', 'Nuwan Thushara', 'Abhinandan Singh', 'Blessing Muzarabani'], 'Delhi Capitals': ['Faf du Plessis', 'Karun Nair', 'Sameer Rizvi', 'Sediqullah Atal', 'Ashutosh Sharma', 'Tripurana Vijay', 'Axar Patel', 'Darshan Nalkande', 'Ajay Jadav Mandal', 'Manvanth Kumar L', 'Madhav Tiwari', 'Tristan Stubbs', 'Abishek Porel', 'Donovan Ferreira', 'KL Rahul', 'Vipraj Nigam', 'Kuldeep Yadav', 'Dushmantha Chameera', 'Mohit Sharma', 'T Natarajan', 'Mukesh Kumar', 'Mustafizur Rahman'], 'Punjab Kings': ['Nehal Wadhera', 'Harnoor Singh', 'Shreyas Iyer', 'Pyla Avinash', 'Priyansh Arya', 'Musheer Khan', 'Marcus Stoinis', 'Aaron Hardie', 'Suryansh Shedge', 'Shashank Singh', 'Mitchell Owen', 'Praveen Dubey', 'Azmatullah Omarzai', 'Prabhsimran Singh', 'Josh Inglis', 'Vishnu Vinod', 'Harpreet Brar', 'Arshdeep Singh', 'Yuzvendra Chahal', 'Vijaykumar Vyshak', 'Kuldeep Sen', 'Yash Thakur', 'Xavier Bartlett', 'Kyle Jamieson'], 'Mumbai Indians': ['Rohit Sharma', 'Suryakumar Yadav', 'Tilak Varma', 'Naman Dhir', 'Bevon Jacobs', 'Hardik Pandya', 'Raj Bawa', 'Charith Asalanka', 'Mitchell Santner', 'Arjun Tendulkar', 'Krishnan Shrijith', 'Robin Minz', 'Jonny Bairstow', 'Jasprit Bumrah', 'Ashwani Kumar', 'Reece Topley', 'Karn Sharma', 'Trent Boult', 'Satyanarayana Raju', 'Deepak Chahar', 'Mujeeb Ur Rahman', 'Raghu Sharma', 'Richard Gleeson'], 'Gujarat Titans': ['Sai Sudharsan', 'Shubman Gill', 'Shahrukh Khan', 'Rahul Tewatia', 'Nishant Sindhu', 'Sherfane Rutherford', 'Mahipal Lomror', 'Dasun Shanaka', 'Rashid Khan', 'Ravisrinivasan Sai Kishore', 'Arshad Khan', 'Jayant Yadav', 'Karim Janat', 'Washington Sundar', 'Kumar Kushagra', 'Anuj Rawat', 'Kusal Mendis', 'Gerald Coetzee', 'Manav Suthar', 'Gurnoor Brar', 'Ishant Sharma', 'Kulwant Khejroliya', 'Prasidh Krishna', 'Mohammed Siraj'], 'Lucknow Super Giants': ['Himmat Singh', 'David Miller', 'Aiden Markram', 'Ayush Badoni', 'Mitchell Marsh', 'Abdul Samad', 'Arshin Kulkarni', 'Yuvraj Chaudhary', 'Shahbaz Ahmed', 'RS Hangargekar', 'Shardul Thakur', 'Matthew Breetzke', 'Nicholas Pooran', 'Aryan Juyal', 'Rishabh Pant', 'Ravi Bishnoi', 'Akash Deep', 'Manimaran Siddharth', 'Shamar Joseph', 'Avesh Khan', 'Prince Yadav', 'Akash Maharaj Singh', 'Digvesh Singh Rathi', 'William ORourke']}
team_names_sf = ["KKR","GT","MI","CSK","RR","RCB","PBKS","DC","SRH","LSG"]
team_names_ff = ["Kolkata Knight Riders", "Gujarat Titans", "Mumbai Indians", "Chennai Super Kings","Rajasthan Royals","Royal Challengers Bengaluru", "Punjab Kings","Delhi Capitals","Sunrisers Hyderabad","Lucknow Super Giants"]

names = ['Ruturaj Gaikwad', 'Andre Siddarth C', 'Shaik Rasheed', 'Rahul Tripathi', 'Ayush Mhatre', 'Dewald Brevis', 'Shivam Dube', 'Rachin Ravindra', 'Deepak Hooda', 'Vijay Shankar', 'Ramakrishna Ghosh', 'Ravindra Jadeja', 'Anshul Kamboj', 'Jamie Overton', 'Sam Curran', 'Ravichandran Ashwin', 'Devon Conway', 'MS Dhoni', 'Vansh Bedi', 'Urvil Patel', 'Kamlesh Nagarkoti', 'Shreyas Gopal', 'Matheesha Pathirana', 'Mukesh Choudhary', 'Nathan Ellis', 'Noor Ahmad', 'Khaleel Ahmed', 'Vaibhav Suryavanshi', 'Shimron Hetmyer', 'Yashasvi Jaiswal', 'Shubham Dubey', 'Riyan Parag', 'Wanindu Hasaranga', 'Sanju Samson', 'Dhruv Jurel', 'Kunal Singh Rathore', 'Lhuan-dre Pretorius', 'Yudhvir Singh Charak', 'Tushar Deshpande', 'Kumar Kartikeya', 'Akash Madhwal', 'Kwena Maphaka', 'Maheesh Theekshana', 'Fazalhaq Farooqi', 'Ashok Sharma', 'Jofra Archer', 'Nandre Burger', 'Manish Pandey', 'Ajinkya Rahane', 'Rinku Singh', 'Angkrish Raghuvanshi', 'Anukul Roy', 'Ramandeep Singh', 'Venkatesh Iyer', 'Moeen Ali', 'Sunil Narine', 'Andre Russell', 'Quinton de Kock', 'Rahmanullah Gurbaz', 'Luvnith Sisodia', 'Varun Chakaravarthy', 'Mayank Markande', 'Vaibhav Arora', 'Harshit Rana', 'Anrich Nortje', 'Spencer Johnson', 'Chetan Sakariya', 'Shivam Shukla', 'Atharva Taide', 'Travis Head', 'Abhinav Manohar', 'Sachin Baby', 'Aniket Verma', 'Nitish Kumar Reddy', 'Abhishek Sharma', 'Kamindu Mendis', 'Wiaan Mulder', 'Harsh Dubey', 'Heinrich Klaasen', 'Ishan Kishan', 'Zeeshan Ansari', 'Pat Cummins', 'Mohammed Shami', 'Harshal Patel', 'Rahul Chahar', 'Simarjeet Singh', 'Eshan Malinga', 'Jaydev Unadkat', 'Virat Kohli', 'Rajat Patidar', 'Swastik Chikara', 'Tim David', 'Mayank Agarwal', 'Krunal Pandya', 'Liam Livingstone', 'Manoj Bhandage', 'Romario Shepherd', 'Swapnil Singh', 'Mohit Rathee', 'Philip Salt', 'Jitesh Sharma', 'Tim Seifert', 'Josh Hazlewood', 'Bhuvneshwar Kumar', 'Rasikh Dar Salam', 'Suyash Sharma', 'Yash Dayal', 'Nuwan Thushara', 'Abhinandan Singh', 'Blessing Muzarabani', 'Faf du Plessis', 'Karun Nair', 'Sameer Rizvi', 'Sediqullah Atal', 'Ashutosh Sharma', 'Tripurana Vijay', 'Axar Patel', 'Darshan Nalkande', 'Ajay Jadav Mandal', 'Manvanth Kumar L', 'Madhav Tiwari', 'Tristan Stubbs', 'Abishek Porel', 'Donovan Ferreira', 'KL Rahul', 'Vipraj Nigam', 'Kuldeep Yadav', 'Dushmantha Chameera', 'Mohit Sharma', 'T Natarajan', 'Mukesh Kumar', 'Mustafizur Rahman', 'Nehal Wadhera', 'Harnoor Singh', 'Shreyas Iyer', 'Pyla Avinash', 'Priyansh Arya', 'Musheer Khan', 'Marcus Stoinis', 'Aaron Hardie', 'Suryansh Shedge', 'Shashank Singh', 'Mitchell Owen', 'Praveen Dubey', 'Azmatullah Omarzai', 'Prabhsimran Singh', 'Josh Inglis', 'Vishnu Vinod', 'Harpreet Brar', 'Arshdeep Singh', 'Yuzvendra Chahal', 'Vijaykumar Vyshak', 'Kuldeep Sen', 'Yash Thakur', 'Xavier Bartlett', 'Kyle Jamieson', 'Rohit Sharma', 'Suryakumar Yadav', 'Tilak Varma', 'Naman Dhir', 'Bevon Jacobs', 'Hardik Pandya', 'Raj Bawa', 'Charith Asalanka', 'Mitchell Santner', 'Arjun Tendulkar', 'Krishnan Shrijith', 'Robin Minz', 'Jonny Bairstow', 'Jasprit Bumrah', 'Ashwani Kumar', 'Reece Topley', 'Karn Sharma', 'Trent Boult', 'Satyanarayana Raju', 'Deepak Chahar', 'Mujeeb Ur Rahman', 'Raghu Sharma', 'Richard Gleeson', 'Sai Sudharsan', 'Shubman Gill', 'Shahrukh Khan', 'Rahul Tewatia', 'Nishant Sindhu', 'Sherfane Rutherford', 'Mahipal Lomror', 'Dasun Shanaka', 'Rashid Khan', 'Ravisrinivasan Sai Kishore', 'Arshad Khan', 'Jayant Yadav', 'Karim Janat', 'Washington Sundar', 'Kumar Kushagra', 'Anuj Rawat', 'Kusal Mendis', 'Gerald Coetzee', 'Manav Suthar', 'Gurnoor Brar', 'Ishant Sharma', 'Kulwant Khejroliya', 'Prasidh Krishna', 'Mohammed Siraj', 'Himmat Singh', 'David Miller', 'Aiden Markram', 'Ayush Badoni', 'Mitchell Marsh', 'Abdul Samad', 'Arshin Kulkarni', 'Yuvraj Chaudhary', 'Shahbaz Ahmed', 'RS Hangargekar', 'Shardul Thakur', 'Matthew Breetzke', 'Nicholas Pooran', 'Aryan Juyal', 'Rishabh Pant', 'Ravi Bishnoi', 'Akash Deep', 'Manimaran Siddharth', 'Shamar Joseph', 'Avesh Khan', 'Prince Yadav', 'Akash Maharaj Singh', 'Digvesh Singh Rathi', 'William ORourke',

 # NEW 22 (appended exactly in order)
 'Ravichandran Smaran',
 'Lockie Ferguson',
 'Corbin Bosch',
 'Mayank Yadav',
 'Kagiso Rabada',
 'Marco Jansen',
 'Glenn Maxwell',
 'Sandeep Sharma',
 'Rajvardhan Hangargekar',
 'Nitish Rana',
 'Will Jacks',
 'Adam Zampa',
 'Rovman Powell',
 'Ryan Rickelton',
 'Glenn Phillips',
 'Devdutt Padikkal',
 'Mohsin Khan',
 'Mitchell Starc',
 'Jos Buttler',
 'Lungi Ngidi',
 'Jake Fraser-McGurk',
 'Mujeeb ur Rahman',
    # India (15)
    "Suryakumar Yadav (captain)",
    "Shubman Gill",
    "Abhishek Sharma",
    "Tilak Varma",
    "Hardik Pandya",
    "Shivam Dube",
    "Axar Patel",
    "Jitesh Sharma",
    "Sanju Samson",
    "Jasprit Bumrah",
    "Varun Chakravarthy",
    "Arshdeep Singh",
    "Kuldeep Yadav",
    "Harshit Rana",
    "Washington Sundar",

    # South Africa (17)
    "Aiden Markram (c)",
    "Ottneil Baartman",
    "Corbin Bosch",
    "Dewald Brevis",
    "Quinton de Kock (wk)",
    "Tony de Zorzi",
    "Donovan Ferreira",
    "Reeza Hendricks",
    "Marco Jansen",
    "George Linde",
    "Kwena Maphaka",
    "David Miller",
    "Lungi Ngidi",
    "Anrich Nortje",
    "Tristan Stubbs",
    "Keshav Maharaj",
    "Lutho Sipamla"
]

roles = ['BAT', 'BAT', 'BAT', 'BAT', 'BAT', 'BAT', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'WK', 'WK', 'WK', 'WK', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BAT', 'BAT', 'BAT', 'BAT', 'AR', 'AR', 'WK', 'WK', 'WK', 'WK', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BAT', 'BAT', 'BAT', 'BAT', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'WK', 'WK', 'WK', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BAT', 'BAT', 'BAT', 'BAT', 'BAT', 'AR', 'AR', 'AR', 'AR', 'AR', 'WK', 'WK', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BAT', 'BAT', 'BAT', 'BAT', 'BAT', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'WK', 'WK', 'WK', 'WK', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BAT', 'BAT', 'BAT', 'BAT', 'BAT', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'WK', 'WK', 'WK', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BAT', 'BAT', 'BAT', 'BAT', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'WK', 'WK', 'WK', 'WK', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BAT', 'BAT', 'BAT', 'BAT', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'WK', 'WK', 'WK', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BAT', 'BAT', 'BAT', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'AR', 'WK', 'WK', 'WK', 'WK', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL', 'BOWL',

 # appended (new players, in the same order you provided them)
 'BAT',   # Ravichandran Smaran
 'BOWL',  # Lockie Ferguson
 'AR',    # Corbin Bosch
 'BOWL',  # Mayank Yadav (corrected to BOWL)
 'BOWL',  # Kagiso Rabada
 'BOWL',  # Marco Jansen
 'AR',    # Glenn Maxwell
 'BOWL',  # Sandeep Sharma
 'AR',    # Rajvardhan Hangargekar (corrected to AR)
 'BAT',   # Nitish Rana
 'AR',    # Will Jacks
 'BOWL',  # Adam Zampa
 'BAT',   # Rovman Powell (corrected to BAT)
 'WK',    # Ryan Rickelton
 'WK',    # Glenn Phillips
 'BAT',   # Devdutt Padikkal
 'BOWL',  # Mohsin Khan
 'BOWL',  # Mitchell Starc
 'WK',    # Jos Buttler
 'BOWL',  # Lungi Ngidi
 'BAT',   # Jake Fraser-McGurk
 'BOWL',   # Mujeeb ur Rahman
    # India (15)
    "BAT",   # Suryakumar Yadav (captain)
    "BAT",   # Shubman Gill
    "BAT",   # Abhishek Sharma
    "BAT",   # Tilak Varma
    "AR",    # Hardik Pandya
    "AR",    # Shivam Dube
    "AR",    # Axar Patel
    "WK",    # Jitesh Sharma
    "WK",    # Sanju Samson
    "BOWL",  # Jasprit Bumrah
    "BOWL",  # Varun Chakravarthy
    "BOWL",  # Arshdeep Singh
    "BOWL",  # Kuldeep Yadav
    "BOWL",  # Harshit Rana
    "AR",    # Washington Sundar
    # South Africa (17)
    "BAT",   # Aiden Markram (c)
    "BOWL",  # Ottneil Baartman
    "AR",    # Corbin Bosch
    "BAT",   # Dewald Brevis
    "WK",    # Quinton de Kock (wk)
    "BAT",   # Tony de Zorzi
    "AR",    # Donovan Ferreira
    "BAT",   # Reeza Hendricks
    "AR",    # Marco Jansen
    "AR",    # George Linde
    "BOWL",  # Kwena Maphaka
    "BAT",   # David Miller
    "BOWL",  # Lungi Ngidi
    "BOWL",  # Anrich Nortje
    "BAT",   # Tristan Stubbs
    "BOWL",  # Keshav Maharaj
    "BOWL"   # Lutho Sipamla
]
squads = {
 'Chennai Super Kings': [
    'Ruturaj Gaikwad', 'Andre Siddarth C', 'Shaik Rasheed', 'Rahul Tripathi',
    'Ayush Mhatre', 'Dewald Brevis', 'Shivam Dube', 'Rachin Ravindra',
    'Deepak Hooda', 'Vijay Shankar', 'Ramakrishna Ghosh', 'Ravindra Jadeja',
    'Anshul Kamboj', 'Jamie Overton', 'Sam Curran', 'Ravichandran Ashwin',
    'Devon Conway', 'MS Dhoni', 'Vansh Bedi', 'Urvil Patel', 'Kamlesh Nagarkoti',
    'Shreyas Gopal', 'Matheesha Pathirana', 'Mukesh Choudhary', 'Nathan Ellis',
    'Noor Ahmad', 'Khaleel Ahmed'
 ],
 'Rajasthan Royals': [
    'Vaibhav Suryavanshi', 'Shimron Hetmyer', 'Yashasvi Jaiswal', 'Shubham Dubey',
    'Riyan Parag', 'Wanindu Hasaranga', 'Sanju Samson', 'Dhruv Jurel',
    'Kunal Singh Rathore', 'Lhuan-dre Pretorius', 'Yudhvir Singh Charak',
    'Tushar Deshpande', 'Kumar Kartikeya', 'Akash Madhwal', 'Kwena Maphaka',
    'Maheesh Theekshana', 'Fazalhaq Farooqi', 'Ashok Sharma', 'Jofra Archer',
    'Nandre Burger', 'Sandeep Sharma', 'Nitish Rana'
 ],
 'Kolkata Knight Riders': [
    'Manish Pandey', 'Ajinkya Rahane', 'Rinku Singh', 'Angkrish Raghuvanshi',
    'Anukul Roy', 'Ramandeep Singh', 'Venkatesh Iyer', 'Moeen Ali',
    'Sunil Narine', 'Andre Russell', 'Quinton de Kock', 'Rahmanullah Gurbaz',
    'Luvnith Sisodia', 'Varun Chakaravarthy', 'Mayank Markande', 'Vaibhav Arora',
    'Harshit Rana', 'Anrich Nortje', 'Spencer Johnson', 'Chetan Sakariya',
    'Shivam Shukla', 'Rovman Powell'
 ],
 'Sunrisers Hyderabad': [
    'Atharva Taide', 'Travis Head', 'Abhinav Manohar', 'Sachin Baby',
    'Aniket Verma', 'Nitish Kumar Reddy', 'Abhishek Sharma', 'Kamindu Mendis',
    'Wiaan Mulder', 'Harsh Dubey', 'Heinrich Klaasen', 'Ishan Kishan',
    'Zeeshan Ansari', 'Pat Cummins', 'Mohammed Shami', 'Harshal Patel',
    'Rahul Chahar', 'Simarjeet Singh', 'Eshan Malinga', 'Jaydev Unadkat',
    'Ravichandran Smaran', 'Adam Zampa'
 ],
 'Royal Challengers Bengaluru': [
    'Virat Kohli', 'Rajat Patidar', 'Swastik Chikara', 'Tim David',
    'Mayank Agarwal', 'Krunal Pandya', 'Liam Livingstone', 'Manoj Bhandage',
    'Romario Shepherd', 'Swapnil Singh', 'Mohit Rathee', 'Philip Salt',
    'Jitesh Sharma', 'Tim Seifert', 'Josh Hazlewood', 'Bhuvneshwar Kumar',
    'Rasikh Dar Salam', 'Suyash Sharma', 'Yash Dayal', 'Nuwan Thushara',
    'Abhinandan Singh', 'Blessing Muzarabani', 'Devdutt Padikkal', 'Lungi Ngidi'
 ],
 'Delhi Capitals': [
    'Faf du Plessis', 'Karun Nair', 'Sameer Rizvi', 'Sediqullah Atal',
    'Ashutosh Sharma', 'Tripurana Vijay', 'Axar Patel', 'Darshan Nalkande',
    'Ajay Jadav Mandal', 'Manvanth Kumar L', 'Madhav Tiwari', 'Tristan Stubbs',
    'Abishek Porel', 'Donovan Ferreira', 'KL Rahul', 'Vipraj Nigam',
    'Kuldeep Yadav', 'Dushmantha Chameera', 'Mohit Sharma', 'T Natarajan',
    'Mukesh Kumar', 'Mustafizur Rahman', 'Mitchell Starc', 'Jake Fraser-McGurk'
 ],
 'Punjab Kings': [
    'Nehal Wadhera', 'Harnoor Singh', 'Shreyas Iyer', 'Pyla Avinash',
    'Priyansh Arya', 'Musheer Khan', 'Marcus Stoinis', 'Aaron Hardie',
    'Suryansh Shedge', 'Shashank Singh', 'Mitchell Owen', 'Praveen Dubey',
    'Azmatullah Omarzai', 'Prabhsimran Singh', 'Josh Inglis', 'Vishnu Vinod',
    'Harpreet Brar', 'Arshdeep Singh', 'Yuzvendra Chahal', 'Vijaykumar Vyshak',
    'Kuldeep Sen', 'Yash Thakur', 'Xavier Bartlett', 'Kyle Jamieson',
    'Lockie Ferguson', 'Marco Jansen', 'Glenn Maxwell'
 ],
 'Mumbai Indians': [
    'Rohit Sharma', 'Suryakumar Yadav', 'Tilak Varma', 'Naman Dhir',
    'Bevon Jacobs', 'Hardik Pandya', 'Raj Bawa', 'Charith Asalanka',
    'Mitchell Santner', 'Arjun Tendulkar', 'Krishnan Shrijith', 'Robin Minz',
    'Jonny Bairstow', 'Jasprit Bumrah', 'Ashwani Kumar', 'Reece Topley',
    'Karn Sharma', 'Trent Boult', 'Satyanarayana Raju', 'Deepak Chahar',
    'Mujeeb Ur Rahman', 'Raghu Sharma', 'Richard Gleeson',
    'Corbin Bosch', 'Will Jacks', 'Ryan Rickelton', 'Mujeeb ur Rahman'
 ],
 'Gujarat Titans': [
    'Sai Sudharsan', 'Shubman Gill', 'Shahrukh Khan', 'Rahul Tewatia',
    'Nishant Sindhu', 'Sherfane Rutherford', 'Mahipal Lomror', 'Dasun Shanaka',
    'Rashid Khan', 'Ravisrinivasan Sai Kishore', 'Arshad Khan', 'Jayant Yadav',
    'Karim Janat', 'Washington Sundar', 'Kumar Kushagra', 'Anuj Rawat',
    'Kusal Mendis', 'Gerald Coetzee', 'Manav Suthar', 'Gurnoor Brar',
    'Ishant Sharma', 'Kulwant Khejroliya', 'Prasidh Krishna', 'Mohammed Siraj',
    'Kagiso Rabada', 'Glenn Phillips', 'Jos Buttler'
 ],
 'Lucknow Super Giants': [
    'Himmat Singh', 'David Miller', 'Aiden Markram', 'Ayush Badoni',
    'Mitchell Marsh', 'Abdul Samad', 'Arshin Kulkarni', 'Yuvraj Chaudhary',
    'Shahbaz Ahmed', 'RS Hangargekar', 'Shardul Thakur', 'Matthew Breetzke',
    'Nicholas Pooran', 'Aryan Juyal', 'Rishabh Pant', 'Ravi Bishnoi',
    'Akash Deep', 'Manimaran Siddharth', 'Shamar Joseph', 'Avesh Khan',
    'Prince Yadav', 'Akash Maharaj Singh', 'Digvesh Singh Rathi', 'William ORourke',
    'Mayank Yadav', 'Rajvardhan Hangargekar', 'Mohsin Khan'
 ],
    "India": [
        "Suryakumar Yadav (captain)",
        "Shubman Gill",
        "Abhishek Sharma",
        "Tilak Varma",
        "Hardik Pandya",
        "Shivam Dube",
        "Axar Patel",
        "Jitesh Sharma",
        "Sanju Samson",
        "Jasprit Bumrah",
        "Varun Chakravarthy",
        "Arshdeep Singh",
        "Kuldeep Yadav",
        "Harshit Rana",
        "Washington Sundar"
    ],

    "South Africa": [
        "Aiden Markram (c)",
        "Ottneil Baartman",
        "Corbin Bosch",
        "Dewald Brevis",
        "Quinton de Kock (wk)",
        "Tony de Zorzi",
        "Donovan Ferreira",
        "Reeza Hendricks",
        "Marco Jansen",
        "George Linde",
        "Kwena Maphaka",
        "David Miller",
        "Lungi Ngidi",
        "Anrich Nortje",
        "Tristan Stubbs",
        "Keshav Maharaj",
        "Lutho Sipamla"
    ]
}



def find_full_name(team, short_name):
    """
    Clean, safe, debug-friendly name resolver.
    No fuzzywuzzy. No unpack errors.
    """
    #print("Trying to find",short_name)
    try:
        # clean '(sub' and junk
        s = short_name.strip()
        s = re.sub(r'^\(sub\)?\s*', '', s, flags=re.IGNORECASE)
        s = s.strip("() ").strip()
        
        if s in team:
            #print(s,"found")
            return s

        # special manual fixes
        if "Varun Chak" in s:
            return "Varun Chakaravarthy"
        if "Reddy" in s and "Nitish" in s:
            return "Nitish Reddy"

        # your original logic
        for player in team:
            try:
                # option A: short inside long
                if len(player) > len(s):
                    count = 0
                    parts = split_camel_short(s)
                    for part in parts:
                        if part not in player:
                            count += 1
                    if count == 0:
                        #print(player,"found")
                        return player
                else:
                    # option B: long inside short
                    count = 0
                    parts = split_camel_short(player)
                    for part in parts:
                        if part not in s:
                            count += 1
                    if count == 0:
                        #print(player,"found")
                        return player
            except:
                continue

        # fallback: difflib
        team_lower = [p.lower() for p in team]
        matches = difflib.get_close_matches(s.lower(), team_lower, n=1, cutoff=0.7)

        if matches:
            idx = team_lower.index(matches[0])
            return team[idx]

        # FAILED → print debug + return None
        print(f"[DEBUG] No match: '{short_name}' -> returning None")
        return None

    except Exception as e:
        print(f"[DEBUG] Unexpected error for '{short_name}': {e}")
        return None


def split_camel_short(name):
    """Utility: split camel case or multi-part names reliably."""
    parts = []
    word = ""
    for i, ch in enumerate(name):
        word += ch
        if i == len(name)-1 or (i+1 < len(name) and name[i+1].isupper()):
            parts.append(word)
            word = ""
    return parts




class Score:
    
    def __init__(self, match_number: int):
        self.match_number = match_number
        self.squads = squads
        self.match_url, self.match_data = self.get_match_data(match_number)
        (
            self.full_player_list,
            self.player_list,
            self.winner,
            self.man_of_the_match,
            self.catchers,
            self.stumpers,
            self.main_runouters,
            self.secondary_runouters,
            self.bowled,
            self.lbw,
            self.innings_list,
            self.batsmen_list,
            self.bowlers_info
        ) = self._parse_json(self.match_data)
        
    def get_match_data(self,match_id):
        payload = {}
        headers = {
            'x-apihub-key': 'NFrLWAL6waj1iYNyxjW594CFWp0KHzlZVJIMOAaEX7t3OOdbfk',
            'x-apihub-host': 'Cricbuzz-Official-Cricket-API.allthingsdev.co',
            'x-apihub-endpoint': '5f260335-c228-4005-9eec-318200ca48d6'
        }
        try:
            match_number_string = str(match_id)
            url = "https://Cricbuzz-Official-Cricket-API.proxy-production.allthingsdev.co/match/"+match_number_string+"/scorecard"
            response = requests.request("GET", url, headers=headers, data=payload)
            match = response.json()
            match_url = match['appindex']['weburl']
            return match_url, match
        except:
            return None,None
    
    #def player_list_generator(self):

    def _parse_dismissal(self, outdec: str) -> Dict[str, Any]:
        """
        Parse a dismissal string (outdec) and return a dict with:
        - mode: 'not out'/'caught'/'stumped'/'bowled'/'lbw'/'run out'/...
        - catcher, stumper, main_ro, secondary_ro, bowler_bowled, bowler_lbw
        Guarantees keys exist and are strings (empty string when not present).
        """
        import re

        outdec = (outdec or "").strip()

        # Quick: not out / empty
        if not outdec or outdec.lower() == 'not out':
            return {
                'mode': 'not out',
                'catcher': '',
                'stumper': '',
                'main_ro': '',
                'secondary_ro': '',
                'bowler_bowled': '',
                'bowler_lbw': ''
            }

        # default result
        result = {
            'mode': 'other',
            'catcher': '',
            'stumper': '',
            'main_ro': '',
            'secondary_ro': '',
            'bowler_bowled': '',
            'bowler_lbw': ''
        }

        # c and b X  -> "c and b X" (caught and bowled)
        if outdec.startswith('c and b '):
            x = outdec.replace('c and b ', '').strip()
            result.update({'mode': 'caught', 'catcher': x})
            return result

        # c X b Y  -> caught by X, bowled by Y (we only record catcher here as before)
        if outdec.startswith('c ') and ' b ' in outdec:
            try:
                parts = outdec.split(' b ', 1)
                fielder = parts[0].replace('c ', '').strip()
                # bowler = parts[1].strip()  # bowler available if needed
                result.update({'mode': 'caught', 'catcher': fielder})
            except Exception:
                # fallback: leave as other but keep raw
                result.update({'mode': 'caught'})
            return result

        # st X b Y  -> stumped by X
        if outdec.startswith('st ') and ' b ' in outdec:
            try:
                parts = outdec.split(' b ', 1)
                fielder = parts[0].replace('st ', '').strip()
                result.update({'mode': 'stumped', 'stumper': fielder})
            except Exception:
                result.update({'mode': 'stumped'})
            return result

        # b X  -> bowled (pure) — must NOT contain 'lbw'
        if outdec.startswith('b ') and 'lbw' not in outdec.lower():
            bowler = outdec.replace('b ', '').strip()
            result.update({'mode': 'bowled', 'bowler_bowled': bowler})
            return result

        # lbw b X  -> lbw (bowler)
        if 'lbw b ' in outdec.lower():
            # prefer to extract text after the last ' b ' occurrence, preserving case if possible
            try:
                # find lowercase pattern position
                low = outdec.lower()
                idx = low.rfind('lbw b ')
                bowler = outdec[idx + len('lbw b '):].strip()
                # fallback: use last ' b ' if above fails
                if not bowler:
                    bowler = outdec.split(' b ')[-1].strip()
            except Exception:
                bowler = outdec.split(' b ')[-1].strip() if ' b ' in outdec else ''
            result.update({'mode': 'lbw', 'bowler_lbw': bowler})
            return result

        # run out (X) or run out (X/Y) — robust handling for subs, nested parentheses, 3 names etc.
        if 'run out' in outdec.lower():
            main_clean = ""
            sec_clean = ""

            # Extract text from first '(' to last ')'
            i = outdec.find('(')
            j = outdec.rfind(')')
            raw = ""
            if i != -1 and j != -1 and j > i:
                raw = outdec[i+1:j].strip()
            else:
                # fallback to first parenthesised group
                m = re.search(r'\(([^)]*)\)', outdec)
                if m:
                    raw = m.group(1).strip()

            if raw:
                # Remove repeated (sub) markers and stray parens but preserve names and slashes
                raw = re.sub(r'\(sub\)', '', raw, flags=re.IGNORECASE)
                raw = raw.replace('(', '').replace(')', '').strip()

                # Split on '/', drop empty parts
                parts = [p.strip() for p in raw.split('/') if p.strip()]

                # If 3+ parts: ignore the first, take the last two (as you requested)
                if len(parts) >= 3:
                    chosen = parts[-2:]
                elif len(parts) == 2:
                    chosen = parts
                elif len(parts) == 1:
                    chosen = [parts[0]]
                else:
                    chosen = []

                def _sanitize_name(tok):
                    if not tok:
                        return ""
                    t = tok.strip()
                    t = re.sub(r'^\(?sub\)?\s*', '', t, flags=re.IGNORECASE)  # leading sub markers
                    t = t.strip("() ").strip()
                    return t

                if len(chosen) == 2:
                    main_clean = _sanitize_name(chosen[0])
                    sec_clean = _sanitize_name(chosen[1])
                elif len(chosen) == 1:
                    main_clean = _sanitize_name(chosen[0])
                    sec_clean = ""
            else:
                # couldn't extract party list — debug
                print(f"[DEBUG][RUNOUT] Could not parse run out parties from outdec: {outdec!r}")

            result.update({'mode': 'run out', 'main_ro': main_clean, 'secondary_ro': sec_clean})
            return result

        # If none of the above matched, return 'other' (keep the raw outdec in callers if needed)
        return result

    def _parse_json(self, match_Dict: [str, Any]) -> Tuple[
        List[str], Dict[str, List[str]], str, str,
        List[str], List[str], List[str], List[str],
        List[str], List[str],
        List[str], pd.DataFrame, pd.DataFrame
    ]:
        # Lists — NO deduplication, preserve order & frequency
        catchers, stumpers = [], []
        main_runouters, secondary_runouters = [], []
        bowled, lbw = [], []

        innings_list = []
        player_list = {}
        batsmen_rows, bowlers_rows = [], []
        
        dots_info,man_of_the_match,player_list,full_player_list = self.match_dots_mom()

        for idx, inn in enumerate(self.match_data.get('scorecard', [])):
            inn_num = idx + 1
            bat_team = inn['batteamname']
            teams_known = list(player_list.keys())
            bowl_team = next((t for t in teams_known if t != bat_team), "")
            
            bat_players = player_list[bat_team]
            bowl_players = player_list[bowl_team]
            
            innings_list.append(bat_team)


            # --- Batsmen ---
            for b in inn.get('batsman', []):
                if b['balls'] == 0 and not b.get('outdec'):
                    continue
                name = b['name'].strip()
                name = find_full_name(bat_players,name)

                outdec = b.get('outdec', 'not out') or 'not out'
                dis = self._parse_dismissal(outdec)
                #print(outdec,dis)

                # Append exactly once per dismissal — no dedup
                if dis['catcher']:
                    dismisser = dis['catcher']
                    dismisser = find_full_name(bowl_players,dismisser)
                    catchers.append(dismisser)
                if dis['stumper']:
                    dismisser = dis['stumper']
                    dismisser = find_full_name(bowl_players,dismisser)
                    stumpers.append(dismisser)
                if dis['main_ro'] != '' and dis['secondary_ro'] == '':
                    dismisser = dis['main_ro']
                    dismisser = find_full_name(bowl_players,dismisser)
                    main_runouters.append(dismisser)
                elif dis['main_ro'] != '' and dis['secondary_ro'] != '':
                    dismisser = dis['main_ro']
                    dismisser = find_full_name(bowl_players,dismisser)
                    main_runouters.append(dismisser)
                    dismisser = dis['secondary_ro']
                    dismisser = find_full_name(bowl_players,dismisser)
                    secondary_runouters.append(dismisser)
                if dis['bowler_bowled']:
                    dismisser = dis['bowler_bowled']
                    dismisser = find_full_name(bowl_players,dismisser)
                    bowled.append(dismisser)
                if dis['bowler_lbw']:
                    dismisser = dis['bowler_lbw']
                    dismisser = find_full_name(bowl_players,dismisser)
                    lbw.append(dismisser)

                sr = float(b['strkrate']) if b['strkrate'] and b['strkrate'] != '0' else 0.0
                batsmen_rows.append({
                    'Innings Number': inn_num,
                    'Innings Name': bat_team,
                    'Batsman': name,
                    'Dismissal': outdec,
                    'Runs': int(b['runs']),
                    'Balls': int(b['balls']),
                    '4s': int(b['fours']),
                    '6s': int(b['sixes']),
                    'Strike Rate': sr
                })

            # --- Bowlers ---
            for blr in inn.get('bowler', []):
                name = blr['name'].strip()
                name = find_full_name(bowl_players,name)

                # Overs: preserve as X.Y (e.g., '2.2' → 2.2, not 2.333)
                o_str = blr['overs']
                try:
                    if '.' in o_str:
                        whole, frac = o_str.split('.')
                        # Clamp fractional part to 1 digit (0–5)
                        frac = frac[:1]
                        # Ensure valid ball count (0–5)
                        if frac.isdigit() and 0 <= int(frac) <= 5:
                            overs_val = float(f"{whole}.{frac}")
                        else:
                            overs_val = float(whole)
                    else:
                        overs_val = float(o_str)
                except:
                    overs_val = 0.0
                
                bowlers_rows.append({
                    'Innings Number': inn_num,
                    'Innings Name': bat_team,
                    'Bowler': name,
                    'Overs': overs_val,
                    'Maidens': int(blr['maidens']),
                    'Runs': int(blr['runs']),
                    'Wickets': int(blr['wickets']),
                    'Economy': float(blr['economy']),
                    '0s': dots_info.get(name, {}).get('dots', 0)
                })


        batsmen_df = pd.DataFrame(batsmen_rows) if batsmen_rows else pd.DataFrame(columns=[
            'Innings Number', 'Innings Name', 'Batsman', 'Dismissal',
            'Runs', 'Balls', '4s', '6s', 'Strike Rate'
        ])
        bowlers_df = pd.DataFrame(bowlers_rows) if bowlers_rows else pd.DataFrame(columns=[
            'Innings Number', 'Innings Name', 'Bowler',
            'Overs', 'Maidens', 'Runs', 'Wickets', 'Economy','0s'
        ])

        # Winner
        status = self.match_data.get('status', '')
        winner = ""
        if 'won by' in status:
            winner = status.split(' won by ')[0].strip()
            for team in innings_list:
                if winner in team or team in winner:
                    winner = team
                    break
        elif ' won the' in status:
            winner = status.split(' won the')[0].split('(')[1].strip()
        

        return (
            full_player_list,
            player_list,
            winner,
            man_of_the_match,
            catchers,
            stumpers,
            main_runouters,
            secondary_runouters,
            bowled,
            lbw,
            innings_list,
            batsmen_df,
            bowlers_df
        )

    def printing_scorecard(self):
        print("Player List:")
        print(self.player_list)
        print()
        for innings in self.innings_list:
            print("-" * 140)
            print(f"{innings}:")
            print()

            print("Batsmen:")
            batsmen_inn = self.batsmen_list[self.batsmen_list['Innings Name'] == innings]
            if not batsmen_inn.empty:
                df = batsmen_inn.drop(columns=['Innings Number', 'Innings Name']).copy()
                print(df.to_string(index=False))
            else:
                print("(No data)")
            print()

            print("Bowlers:")
            bowlers_inn = self.bowlers_info[self.bowlers_info['Innings Name'] == innings]
            if not bowlers_inn.empty:
                df = bowlers_inn.drop(columns=['Innings Number', 'Innings Name']).copy()
                # Format Overs to show exactly one decimal place (e.g., 2.2, 4.0)
                df['Overs'] = df['Overs'].apply(lambda x: f"{x:.1f}")
                print(df.to_string(index=False))
            else:
                print("(No data)")
            print()

        print("Catchers:")
        print(self.catchers)
        print()

        print("Stumpings:")
        print(self.stumpers)
        print()

        print("Main Run Outs:")
        print(self.main_runouters)
        print()

        print("Secondary Run Outs:")
        print(self.secondary_runouters)
        print()

        print("Bowled:")
        print(self.bowled)
        print()

        print("LBW:")
        print(self.lbw)
        print()
        print("-" * 140)
        print("Winner: ", self.winner)
        print()
        print("Man of the Match: ", self.man_of_the_match)

    def match_dots_mom(self):

        def calculate_dots(over_summary: str) -> int:
            toks = [t for t in over_summary.strip().split(' ') if t != '']
            dots = 0
            for ball in toks:
                # same semantics you used earlier
                if ball == '0' or ball == 'W' or 'L' in ball or 'B' in ball:
                    dots += 1
            return dots

        state_path = Path(f"dots_state_{self.match_number}.pkl")

        if state_path.exists():
            try:
                with state_path.open("rb") as fh:
                    state = pickle.load(fh)
            except Exception:
                state = {"processed_overs": [], "cumulative": {}}
        else:
            state = {"processed_overs": [], "cumulative": {}}

        processed = set(tuple(x) for x in state.get("processed_overs", []))
        cumulative = dict(state.get("cumulative", {}))

        # fetch overs endpoint
        payload = {}
        headers_overs = {
            'x-apihub-key': 'NFrLWAL6waj1iYNyxjW594CFWp0KHzlZVJIMOAaEX7t3OOdbfk',
            'x-apihub-host': 'Cricbuzz-Official-Cricket-API.allthingsdev.co',
            'x-apihub-endpoint': '5db6b2f0-86b9-44b5-bfc4-8c2888acd4de'
        }
        match_number_string = str(self.match_number)
        url_overs = f"https://Cricbuzz-Official-Cricket-API.proxy-production.allthingsdev.co/match/{match_number_string}/overs"
        try:
            resp = requests.request("GET", url_overs, headers=headers_overs, data=payload, timeout=15)
            overs_score = resp.json()
        except Exception:
            # On fetch failure, return existing cumulative in the expected shape
            bowlers_info = {b: {"dots": v} for b, v in cumulative.items()}
            # attempt to build player_list/full_player_list as before (best-effort)
            try:
                team1 = overs_score.get("matchheaders", {}).get("team1", {}).get("teamname", "")
                team2 = overs_score.get("matchheaders", {}).get("team2", {}).get("teamname", "")
            except Exception:
                team1 = team2 = ""
            player_list = {}
            try:
                global squads
                if team1:
                    player_list[team1] = list(squads.get(team1, []))
                if team2:
                    player_list[team2] = list(squads.get(team2, []))
            except Exception:
                pass
            full_player_list = []
            if team1:
                full_player_list.extend(player_list.get(team1, []))
            if team2:
                full_player_list.extend(player_list.get(team2, []))
            man_of_the_match = ""
            if isinstance(overs_score, dict):
                man_of_the_match = overs_score.get("matchheaders", {}).get("momplayers", {}).get("player", [{}])[0].get("name", "")
            return bowlers_info, man_of_the_match, player_list, full_player_list

        oversep = overs_score.get('overseplist', {}).get('oversep', []) or []

        # prepare commentary endpoint details (we'll fetch only when needed)
        headers_comm = {
            'x-apihub-key': 'NFrLWAL6waj1iYNyxjW594CFWp0KHzlZVJIMOAaEX7t3OOdbfk',
            'x-apihub-host': 'Cricbuzz-Official-Cricket-API.allthingsdev.co',
            'x-apihub-endpoint': '8cb69a0f-bcaa-45b5-a016-229a2e7594f6'
        }
        url_commentary = f"https://Cricbuzz-Official-Cricket-API.proxy-production.allthingsdev.co/match/{match_number_string}/commentary"

        # helper: build ball-level bowler sequence for a given overnum using commentary
        def get_ball_bowlers_for_over(target_overnum):
            """
            Returns a list of bowler names in chronological ball order for the given overnum float (e.g., 17.3, 17.4 etc).
            Strategy:
            - Fetch commentary once per function call when needed.
            - Filter commentary entries where commentary["commentary"]["overnum"] has same integer and fractional part equal to target's fractional part.
            - Deduplicate repeated comments for the same ball by keeping first unique (bowler,batsman) tuple.
            - Return the sequence of bowlers in the order seen (trimmed to number of balls needed by oversummary).
            """
            try:
                resp = requests.request("GET", url_commentary, headers=headers_comm, data={}, timeout=15)
                comm = resp.json()
            except Exception:
                return []

            comwrapper = comm.get("comwrapper", []) or []

            seq = []
            seen = set()
            # iterate in the provided order; commentary is usually chronological
            for entry in comwrapper:
                c = entry.get("commentary", {}) or {}
                overnum = c.get("overnum")
                if overnum is None:
                    continue
                # normalize and compare with some tolerance: match exact float representation
                try:
                    overnum_f = float(overnum)
                except Exception:
                    continue
                # float compare: require exact equality of float value
                if abs(overnum_f - float(target_overnum)) > 1e-6:
                    continue
                commtxt = c.get("commtxt", "") or ""
                # parse "Bowler to Batsman" from the start (guarded)
                part = commtxt.split(',')[0].strip()
                if ' to ' not in part:
                    # skip entries that don't follow expected format
                    continue
                try:
                    bowler_name, batsman_name = [x.strip() for x in part.split(' to ', 1)]
                except Exception:
                    continue
                key = (bowler_name, batsman_name)
                if key in seen:
                    continue
                seen.add(key)
                seq.append(bowler_name)
            return seq

        for over in oversep:
            inningsid = over.get('inningsid')
            overnum = over.get('overnum')
            oversummary = (over.get('oversummary') or "").strip()
            ovrbowlnames = over.get('ovrbowlnames') or []

            try:
                overnum_f = float(overnum)
            except Exception:
                continue

            over_int = int(overnum_f)
            frac = overnum_f - over_int
            if frac + 1e-9 < 0.6:
                continue

            over_key = (int(inningsid), over_int)
            if over_key in processed:
                continue

            toks = [t for t in oversummary.split(' ') if t != '']
            n_tokens = len(toks)
            if n_tokens == 0:
                # nothing to attribute; mark processed to avoid repeated work
                processed.add(over_key)
                continue

            if len(ovrbowlnames) == 1:
                # single bowler: same as before
                bowler = ovrbowlnames[0]
                dots = 0
                for ball in toks:
                    if ball == '0' or ball == 'W' or 'L' in ball or 'B' in ball:
                        dots += 1
                if dots:
                    cumulative[bowler] = cumulative.get(bowler, 0) + dots
                processed.add(over_key)
            else:
                # multiple bowlers - try to resolve per-ball attribution using commentary
                # Build ball-level bowler sequence for this over
                ball_bowlers = get_ball_bowlers_for_over(overnum_f)

                # If we couldn't get enough ball-level entries, don't process this over now
                if len(ball_bowlers) < n_tokens:
                    # leave unprocessed so future scrape can attempt again
                    continue

                # align first n_tokens ball bowlers with tokens
                ball_bowlers = ball_bowlers[:n_tokens]

                # attribute dots per ball
                for token, bowler in zip(toks, ball_bowlers):
                    if token == '0' or token == 'W' or 'L' in token or 'B' in token:
                        cumulative[bowler] = cumulative.get(bowler, 0) + 1

                processed.add(over_key)

        # persist updated state
        state_out = {
            "processed_overs": list(processed),
            "cumulative": cumulative
        }
        print(processed)
        try:
            with state_path.open("wb") as fh:
                pickle.dump(state_out, fh)
        except Exception:
            pass

        bowlers_info = {b: {"dots": v} for b, v in cumulative.items()}
        

        man_of_the_match = overs_score.get("matchheaders", {}).get("momplayers", {}).get("player", [{}]) 
        if man_of_the_match:
            man_of_the_match = man_of_the_matchs[0].get("name", "")
        else:
            man_of_the_match = ""

        team1 = overs_score.get("matchheaders", {}).get("team1", {}).get("teamname", "")
        team2 = overs_score.get("matchheaders", {}).get("team2", {}).get("teamname", "")

        player_list = {}
        if team1:
            player_list[team1] = list(self.squads.get(team1, []))
        if team2:
            player_list[team2] = list(self.squads.get(team2, []))

        full_player_list = []
        if team1:
            full_player_list.extend(player_list.get(team1, []))
        if team2:
            full_player_list.extend(player_list.get(team2, []))

        return bowlers_info, man_of_the_match, player_list, full_player_list

            
    # def match_dots_mom(self):
    #     def calculate_dots(over_summary):
    #         over = over_summary.split(' ')
    #         dots = 0
    #         for ball in over:
    #             if ball == '0' or ball == 'W' or 'L' in ball or 'B' in ball:
    #                 dots += 1
    #         return dots

    #     payload = {}
    #     headers = {
    #         'x-apihub-key': 'NFrLWAL6waj1iYNyxjW594CFWp0KHzlZVJIMOAaEX7t3OOdbfk',
    #         'x-apihub-host': 'Cricbuzz-Official-Cricket-API.allthingsdev.co',
    #         'x-apihub-endpoint': '5db6b2f0-86b9-44b5-bfc4-8c2888acd4de'
    #     }

    #     match_number_string = str(self.match_number)
    #     url = "https://Cricbuzz-Official-Cricket-API.proxy-production.allthingsdev.co/match/"+match_number_string+"/overs"
    #     response = requests.request("GET", url, headers=headers, data=payload)
    #     overs_score = response.json()  

    #     bowlers_info = {}
    #     for over in overs_score['overseplist']['oversep']:
    #         bowler_names = over['ovrbowlnames']
    #         over_summary = over['oversummary']
    #         if len(bowler_names) == 1:
    #             bowler = bowler_names[0]
    #             if bowler not in bowlers_info.keys():
    #                 bowlers_info[bowler] = {'dots': 0}
    #                 dots = calculate_dots(over_summary)
    #                 bowlers_info[bowler]['dots'] += dots
    #             else:
    #                 dots = calculate_dots(over_summary)
    #                 bowlers_info[bowler]['dots'] += dots
    #         else:
    #             for bowler in bowler_names:
    #                 x=2 #fill this later
        
    #     man_of_the_match = overs_score["matchheaders"]["momplayers"]["player"][0]["name"]
        
    #     team1 = overs_score["matchheaders"]["team1"]["teamname"]
    #     team2 = overs_score["matchheaders"]["team2"]["teamname"]
    #     print(team1, team2)

    #     player_list = {}
    #     player_list[team1] = squads[team1].copy()   # make copies
    #     player_list[team2] = squads[team2].copy()

    #     # full player list should also be a copy, not a reference
    #     full_player_list = squads[team1].copy()     # copy, not reference
    #     full_player_list.extend(squads[team2])      # now safe

    #     return bowlers_info,man_of_the_match,player_list,full_player_list

class Series:
    def __init__(
        self,
        series_number: int = 9237,
        database_name: str = "IPL2025.pkl",
    ):
        """
        End behaviour matches your old Series:
        - self.match_objects: dict[match_id] -> Score instance
        - self.match_links: list of match_ids in this series
        - loads/stores objects via dill in database_name
        """
        self.series_number = series_number
        self.database_name = database_name

        self.base_url = (
            "https://Cricbuzz-Official-Cricket-API.proxy-production.allthingsdev.co/series/"
        )

        self.match_objects = {}
        self.match_links = []

        # Get all match IDs for this series
        match_links = self.match_id_generator()

        # Try loading existing DB
        try:
            with open(self.database_name, "rb") as file:
                stored = dill.load(file)
        except Exception:
            stored = {}

        match_objects = stored
        match_links_list = list(stored.keys())

        # Mirror your original logic:
        if len(match_links) >= len(match_links_list):
            if len(match_links) != 0:
                for match in match_links:
                    # Same condition: new match or last match (to refresh)
                    if match not in match_links_list or match == match_links[-1]:
                        print("Attempting to scrape:", match)
                        # Score now takes match_id instead of URL
                        match_object = Score(match)
                        print("Scraping Successful")
                        match_object.printing_scorecard()
                        match_objects[match] = match_object
                        print("Added:", match)

                # After loop, same success/fail logic
                if len(list(match_objects.keys())) == len(match_links):
                    self.match_links = match_links
                    self.match_objects = match_objects
                    with open(self.database_name, "wb") as file:
                        dill.dump(match_objects, file)
                    print("LOADING SUCCESSFUL")
                else:
                    print("LOADING FAILED")
                    print("No. of match objects", len(match_objects))
                    print("Number of extracted links", len(match_links))
                    self.match_objects = match_objects
                    self.match_links = match_links
                    print("Missing Links:")
                    for match_id in match_links:
                        if match_id not in list(match_objects.keys()):
                            print(match_id)
                    if len(match_objects) > len(stored):
                        with open(self.database_name, "wb") as file:
                            dill.dump(match_objects, file)
        else:
            print("DATA UP TO DATE")
            self.match_objects = match_objects
            self.match_links = match_links

    def match_id_generator(self):
        url = f"{self.base_url}{self.series_number}"
        headers = {
            "x-apihub-key": "NFrLWAL6waj1iYNyxjW594CFWp0KHzlZVJIMOAaEX7t3OOdbfk",
            "x-apihub-host": "Cricbuzz-Official-Cricket-API.allthingsdev.co",
            "x-apihub-endpoint": "661c6b89-b558-41fa-9553-d0aca64fcb6f",
        }

        try:
            resp = requests.get(url, headers=headers, timeout=15)
            resp.raise_for_status()
            series_data = resp.json()
        except Exception as e:
            print("Error fetching series data:", e)
            return []

        match_ids = []
        for block in series_data.get("matchDetails", []):
            mmap = block.get("matchDetailsMap", {})
            match_list = mmap.get("match", [])
            for m in match_list:
                info = m.get("matchInfo", {})
                match_id = info.get("matchId")
                if match_id:
                    match_ids.append(match_id)

        return match_ids

if __name__ == "__main__":
    #ipl2025 = Series()
    #match = Score(115140)
    match = Score(117416)
    match.printing_scorecard()

{(2, 2), (1, 0), (1, 6), (2, 5), (1, 3), (1, 9), (1, 12), (1, 15), (2, 4), (1, 2), (2, 1), (1, 5), (1, 11), (1, 8), (1, 14), (1, 17), (1, 1), (2, 0), (1, 4), (2, 3), (1, 7), (1, 13), (2, 6), (1, 10), (1, 16)}
Player List:
{'India': ['Suryakumar Yadav (captain)', 'Shubman Gill', 'Abhishek Sharma', 'Tilak Varma', 'Hardik Pandya', 'Shivam Dube', 'Axar Patel', 'Jitesh Sharma', 'Sanju Samson', 'Jasprit Bumrah', 'Varun Chakravarthy', 'Arshdeep Singh', 'Kuldeep Yadav', 'Harshit Rana', 'Washington Sundar'], 'South Africa': ['Aiden Markram (c)', 'Ottneil Baartman', 'Corbin Bosch', 'Dewald Brevis', 'Quinton de Kock (wk)', 'Tony de Zorzi', 'Donovan Ferreira', 'Reeza Hendricks', 'Marco Jansen', 'George Linde', 'Kwena Maphaka', 'David Miller', 'Lungi Ngidi', 'Anrich Nortje', 'Tristan Stubbs', 'Keshav Maharaj', 'Lutho Sipamla']}

--------------------------------------------------------------------------------------------------------------------------------------------
South Africa:

Batsmen:
       